# ------------- Mohammad Amin Kiani 4003613052 -------------
## Final Project _ Emotion-Based Music Generation _ ui.ac.ir 400-404

# تحلیل متن    

In [1]:
# نصب کتابخانه‌های موردنیاز برای NLP و فاین‌تیون مدل‌ها
!pip install -q transformers datasets sentencepiece scikit-learn langdetect accelerate


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 23.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 75.2 MB/s eta 0:00:00


In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from transformers import DataCollatorWithPadding
from datasets import load_dataset, Dataset
from sklearn.metrics import accuracy_score, f1_score
from langdetect import detect
import torch
import numpy as np


In [ ]:
def detect_language(text):
    try:
        lang = detect(text)
        return 'fa' if lang == 'fa' else 'en'
    except:
        return 'unknown'


In [ ]:
# مدل احساسات فارسی (ParsBERT)
fa_model_name = "HooshvareLab/bert-fa-base-uncased"
fa_tokenizer = AutoTokenizer.from_pretrained(fa_model_name)
fa_model = AutoModelForSequenceClassification.from_pretrained(fa_model_name, num_labels=7)  # فعلاً 7 برچسب احساس

# مدل احساسات انگلیسی (BERT روی GoEmotions یا مشابه)
# برخلاف روبوبرت از قبل روی دیتاست مرحله بعدی فاین تیون شده و نیازی به اموزش ندارد
en_model_name = "nateraw/bert-base-uncased-emotion"
en_tokenizer = AutoTokenizer.from_pretrained(en_model_name)
en_model = AutoModelForSequenceClassification.from_pretrained(en_model_name)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/440 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/654M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/654M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/768 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [ ]:
def predict_emotion(text):
    lang = detect_language(text)

    if lang == 'fa':
        tokenizer = fa_tokenizer
        model = fa_model
    elif lang == 'en':
        tokenizer = en_tokenizer
        model = en_model
    else:
        return "Unknown language"

    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        max_length=128              # مشخص کردن max_length برای جلوگیری از هشدار
    )
    with torch.no_grad():
        logits = model(**inputs).logits
    probs = torch.nn.functional.softmax(logits, dim=1)
    pred_class = torch.argmax(probs, dim=1).item()

    return f"زبان تشخیص‌داده‌شده: {lang} | کلاس احساس: {pred_class}"


In [ ]:
# تست: متن فارسی
print(predict_emotion("!خیلی ناراحتم"))

# تست: متن انگلیسی
print(predict_emotion("I'm feeling excited and happy today!"))


model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


زبان تشخیص‌داده‌شده: fa | کلاس احساس: 2
زبان تشخیص‌داده‌شده: en | کلاس احساس: 1


## --------------------------------------

## فاین تیون کردن

### متن فارسی

In [3]:
#  سلول کلون کردن مخزن و بررسی محتویات
!git clone https://github.com/Arman-Rayan-Sharif/arman-text-emotion.git
!ls arman-text-emotion/dataset

# from datasets import load_dataset

# # بارگیری دیتاست ArmanEmo
# dataset_fa = load_dataset("sobhanmoosavi/arman-emo")

# # بررسی چند نمونه اول
# dataset_fa['train'][0]


Cloning into 'arman-text-emotion'...
remote: Enumerating objects: 52, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 52 (delta 10), reused 9 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (52/52), 572.39 KiB | 10.60 MiB/s, done.
Resolving deltas: 100% (10/10), done.
test.tsv  train.tsv


In [4]:
import pandas as pd

# فایل‌ها هیچ هدر ندارند، پس header=None و نام‌گذاری ستون‌ها
train_df = pd.read_csv(
    "arman-text-emotion/dataset/train.tsv",
    sep="\t",
    header=None,
    names=["text","label"]
)
test_df = pd.read_csv(
    "arman-text-emotion/dataset/test.tsv",
    sep="\t",
    header=None,
    names=["text","label"]
)

# حذف ردیف‌های ناقص (در صورت وجود)
train_df.dropna(subset=["text","label"], inplace=True)
test_df.dropna(subset=["text","label"], inplace=True)

print("Train sample:")
print(train_df.head(), "\n")
print("Test sample:")
print(test_df.head())


Train sample:
                                                text  label
0  خیلی کوچیک هستن و سایزشون بدرد نمیخوره میخوام ...    SAD
1     از صدای پرنده دم دمای صبح متنفرم متنفرم متنفرم   HATE
2   "کیفیتش خیلی خوبه با شک خریدم ولی واقعا راضیم...    SAD
3  چون همش با دوربین ثبت شده ، ایا میشه اعتراض زد...  OTHER
4              اين وضع ب طرز خنده داري گريه داره ...    SAD 

Test sample:
                                                text  label
0  اين شايد اولين عزاى عمومى واقعى است كه ياد دار...    SAD
1  دیشب بعد از ارسال تویت مربوط به آثار باستانی ت...  HAPPY
2   کدوم شعبه پول نداده، بگو الان برات آمار دقیق ...  OTHER
3  امروز وسط یه بحث با بابا مامانم گفتم آدم باید ...  HAPPY
4  امشب گفت نامزدی دوستش که ادم روشنفکری است بهم ...    SAD


In [5]:
import os
os.environ["WANDB_DISABLED"] = "true"

#از لاگ‌گیری Weights & Biases (wandb) جلوگیری کنید و دیگر از شما API Key نپرسد

In [6]:

# 1. نصب احتمالی آپدیت transformers (در صورت نیاز)
!pip install -q --upgrade transformers

import pandas as pd
import glob
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding, TrainingArguments, Trainer
import numpy as np
import torch
from sklearn.metrics import accuracy_score, f1_score

# 2. بارگذاری داده‌های TSV با نام‌گذاری ستون‌ها
train_df = pd.read_csv(
    "arman-text-emotion/dataset/train.tsv",
    sep="\t", header=None, names=["text","label"]
)
test_df = pd.read_csv(
    "arman-text-emotion/dataset/test.tsv",
    sep="\t", header=None, names=["text","label"]
)
train_df.dropna(subset=["text","label"], inplace=True)
test_df.dropna(subset=["text","label"], inplace=True)

# 3. ساخت DatasetDict از pandas
dataset_fa = DatasetDict({
    "train": Dataset.from_pandas(train_df),
    "test":  Dataset.from_pandas(test_df)
})

# 4. نگاشت برچسب‌های فارسی به عدد و زیر کلید 'labels'
label_list = sorted(train_df["label"].unique().tolist())
label2id   = {l:i for i,l in enumerate(label_list)}
id2label   = {i:l for l,i in label2id.items()}

def encode_label(ex):
    return {"labels": label2id[ex["label"]]}

# فقط ستون 'label' را حذف می‌کنیم
dataset_fa = dataset_fa.map(encode_label, remove_columns=["label"])

# 5. بارگذاری توکنایزر و مدل (ParsBERT)
fa_model_name = "HooshvareLab/bert-fa-base-uncased"
fa_tokenizer  = AutoTokenizer.from_pretrained(fa_model_name)
fa_model      = AutoModelForSequenceClassification.from_pretrained(
    fa_model_name,
    num_labels=len(label_list)
)

# 6. توکن‌سازی
def tokenize_fn(ex):
    return fa_tokenizer(
        ex["text"],
        truncation=True,
        padding="max_length",
        max_length=128
    )

dataset_fa = dataset_fa.map(tokenize_fn, batched=True, remove_columns=["text"])

# 7. تنظیم فرمت PyTorch
dataset_fa.set_format(
    type="torch",
    columns=["input_ids","attention_mask","labels"]
)

# 8. DataCollator
data_collator = DataCollatorWithPadding(tokenizer=fa_tokenizer)

# 9. متریک‌ها
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1":       f1_score(labels, preds, average="macro")
    }

from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./parsbert-emotion-fa",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=2e-5,
    weight_decay=0.01,

    # لاگ‌گیری
    logging_dir="./logs",
    logging_steps=100,    # هر ۱۰۰ step یک لاگ

    # فعال‌سازی ارزیابی و ذخیره‌سازی در طول آموزش
    do_train=True,
    do_eval=True,
    eval_steps=500,       # هر ۵۰۰ step ارزیابی کن
    save_steps=500,       # هر ۵۰۰ step یک checkpoint ذخیره کن
    save_total_limit=2,   # حداکثر ۲ checkpoint نگه دار

    # (حذف load_best_model_at_end چون نیازمند evaluation_strategy است)
)

trainer_fa = Trainer(
    model=fa_model,
    args=training_args,
    train_dataset=dataset_fa["train"],
    eval_dataset=dataset_fa["test"],
    tokenizer=fa_tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer_fa.train()


# 13. ذخیره مدل
fa_model.save_pretrained("./final_parsbert_emotion")
fa_tokenizer.save_pretrained("./final_parsbert_emotion")
print(" مدل فارسی فاین‌تیون شد و ذخیره شد.")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 62.1 MB/s eta 0:00:00


Map:   0%|          | 0/6125 [00:00<?, ? examples/s]

Map:   0%|          | 0/1151 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/440 [00:00<?, ?B/s]

...

Map:   0%|          | 0/6125 [00:00<?, ? examples/s]

Map:   0%|          | 0/1151 [00:00<?, ? examples/s]

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/tmp/ipython-input-1923072073.py:103: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_fa = Trainer(


Step,Training Loss
100,1.680700
200,1.197100
300,1.022000
400,0.940200
500,0.718600
600,0.740800
700,0.748000
800,0.592700
900,0.495800
1000,0.474800


 مدل فارسی فاین‌تیون شد و ذخیره شد.


In [ ]:
!zip -r final_parsbert_emotion.zip final_parsbert_emotion

from google.colab import files
files.download("final_parsbert_emotion.zip")


  adding: final_parsbert_emotion/ (stored 0%)
  adding: final_parsbert_emotion/model.safetensors (deflated 8%)
  adding: final_parsbert_emotion/special_tokens_map.json (deflated 42%)
  adding: final_parsbert_emotion/vocab.txt (deflated 62%)
  adding: final_parsbert_emotion/tokenizer.json (deflated 72%)
  adding: final_parsbert_emotion/config.json (deflated 56%)
  adding: final_parsbert_emotion/tokenizer_config.json (deflated 74%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
print("All labels:", label_list)
print("Mapping id2label:", id2label)


All labels: ['ANGRY', 'FEAR', 'HAPPY', 'HATE', 'OTHER', 'SAD', 'SURPRISE']
Mapping id2label: {0: 'ANGRY', 1: 'FEAR', 2: 'HAPPY', 3: 'HATE', 4: 'OTHER', 5: 'SAD', 6: 'SURPRISE'}


In [ ]:
from transformers import AutoConfig

# ۱. بارگذاری کانفیگ و افزودن نگاشت
config = AutoConfig.from_pretrained(model_dir)
config.id2label   = id2label
config.label2id   = label2id

# ۲. بارگذاری مدل با کانفیگ سفارشی
model = AutoModelForSequenceClassification.from_pretrained(model_dir, config=config)
tokenizer = AutoTokenizer.from_pretrained(model_dir)

# ۳. ساخت pipeline
clf = pipeline("text-classification", model=model, tokenizer=tokenizer)

# ۴. تست
print(clf("خیلی ناراحتم و غمگینم"))


Device set to use cuda:0


[{'label': 'SAD', 'score': 0.9119796752929688}]


In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification

# بارگذاری مدل و pipeline
model_dir = "/content/final_parsbert_emotion"
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model     = AutoModelForSequenceClassification.from_pretrained(model_dir)
clf = pipeline("text-classification", model=model, tokenizer=tokenizer)


def predict_with_threshold(text, threshold=0.7):
    out = clf(text, top_k=None)[0]
    prob = out['score']
    idx  = int(out['label'].split("_")[-1])
    label = id2label[idx]
    if prob < threshold:
        label = 'OTHER'
    return {"label": label, "score": prob}

# تست
print(predict_with_threshold("رفتم بازار خرید کنم", threshold=0.7))

Device set to use cuda:0


{'label': 'OTHER', 'score': 0.9068320393562317}


### متن انگلیسی

In [ ]:
import numpy as np
import torch
from datasets import load_dataset, DatasetDict
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer,
    pipeline
)
from sklearn.metrics import f1_score, precision_score, recall_score


In [ ]:
# GoEmotions دیتاستِ تشخیص احساس چندجسمانی ۲۷ برچسبه


# ۱. دیتاست خام را دوباره بارگذاری کن تا text و id داشته باشد
raw_en = load_dataset("go_emotions")

# تقسیم train/validation/test را می‌بینیم
print(dataset_en)

# برچسب‌ها (نام‌های ۲۷کلاسه)
label_names = dataset_en["train"].features["labels"].feature.names
print("Labels:", label_names)
num_labels = len(label_names)


DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 43410
    })
    validation: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 5426
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 5427
    })
})
Labels: ['admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise', 'neutral']


In [ ]:
model_name = "roberta-base"  #یا هر مدل پایهٔ دیگری
tokenizer_en = AutoTokenizer.from_pretrained(model_name)

# مشکل multi-label: از problem_type="multi_label_classification" استفاده می‌کنیم
model_en = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
    problem_type="multi_label_classification"
)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from datasets import load_dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from transformers import DataCollatorWithPadding, TrainingArguments, Trainer
from sklearn.metrics import f1_score, precision_score, recall_score

# 1) بارگذاری دیتاست خام
raw = load_dataset("go_emotions")
dataset_en = DatasetDict({
    "train":      raw["train"],
    "validation": raw["validation"],
    "test":       raw["test"]
})

# 2) آماده‌سازی توکنایزر و مدل
tokenizer_en = AutoTokenizer.from_pretrained("roberta-base")
model_en  = AutoModelForSequenceClassification.from_pretrained(
    "roberta-base",
    num_labels=len(raw["train"].features["labels"].feature.names),
    problem_type="multi_label_classification"
)
num_labels = model_en.config.num_labels

# 3) تابع preprocessing که مستقیماً لیبل‌های float می‌سازد
def preprocess(ex):
    enc = tokenizer_en(
        ex["text"],
        truncation=True,
        padding="max_length",
        max_length=128
    )
    # اینجا label_vec از float ساخته می‌شود
    label_vec = [0.0] * num_labels
    for idx in ex["labels"]:
        label_vec[idx] = 1.0
    enc["labels"] = label_vec
    return enc

# 4) توکنایز + حذف ستون‌های زائد
dataset_en = dataset_en.map(
    preprocess,
    batched=False,
    remove_columns=["text", "id"]
)

# 5) قالب‌دهی مستقیم برای PyTorch (labels اکنون لیست float است)
dataset_en.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "labels"]
)

# ادامه Trainer و training_args مثل قبل:
data_collator_en = DataCollatorWithPadding(tokenizer=tokenizer_en)

def compute_metrics_en(eval_pred):
    logits, labels = eval_pred
    probs = torch.sigmoid(torch.tensor(logits))
    preds = (probs.numpy() >= 0.5).astype(int)
    labels = labels.numpy()
    return {
        "f1":        f1_score(labels, preds, average="macro", zero_division=0),
        "precision": precision_score(labels, preds, average="macro", zero_division=0),
        "recall":    recall_score(labels, preds, average="macro", zero_division=0),
    }

training_args_en = TrainingArguments(
    output_dir="./roberta-go_emotions",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_dir="./logs_go",
    logging_steps=500,
    do_train=True,
    do_eval=True,
    eval_steps=1000,
    save_steps=1000,
    save_total_limit=2,
    report_to="none",
    run_name=None,
)

import torch
import torch.nn as nn
from transformers import Trainer

loss_fn = nn.BCEWithLogitsLoss()

class MultiLabelTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs["labels"].float()
        outputs = model(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
        )
        logits = outputs.logits
        loss = loss_fn(logits, labels)
        return (loss, outputs) if return_outputs else loss


# حالا از این Trainerِ جدید استفاده کنید:
trainer_en = MultiLabelTrainer(
    model=model_en,
    args=training_args_en,
    train_dataset=dataset_en["train"],
    eval_dataset=dataset_en["validation"],
    tokenizer=tokenizer_en,
    data_collator=data_collator_en,
    compute_metrics=compute_metrics_en,
)


trainer_en.train()
trainer_en.save_model("./final_go_emotions")
tokenizer_en.save_pretrained("./final_go_emotions")


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/43410 [00:00<?, ? examples/s]

Map:   0%|          | 0/5426 [00:00<?, ? examples/s]

Map:   0%|          | 0/5427 [00:00<?, ? examples/s]

/tmp/ipython-input-1721164868.py:103: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `MultiLabelTrainer.__init__`. Use `processing_class` instead.
  trainer_en = MultiLabelTrainer(
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Step,Training Loss
500,0.179200
1000,0.117400
1500,0.105300
2000,0.099000
2500,0.095200
3000,0.089600
3500,0.084900
4000,0.085400
4500,0.084800
5000,0.081800


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/usr/local/lib/python3.1

('./final_go_emotions/tokenizer_config.json',
 './final_go_emotions/special_tokens_map.json',
 './final_go_emotions/vocab.json',
 './final_go_emotions/merges.txt',
 './final_go_emotions/added_tokens.json',
 './final_go_emotions/tokenizer.json')

In [ ]:
# بارگذاری مدل فاین‌تیون‌شده
tokenizer_en = AutoTokenizer.from_pretrained("./final_go_emotions")
model_en     = AutoModelForSequenceClassification.from_pretrained("./final_go_emotions")

# pipeline چندتایی با ارزیابی احتمالات
clf_en = pipeline(
    "text-classification",
    model=model_en,
    tokenizer=tokenizer_en,
    function_to_apply="sigmoid",
    top_k=None  # همه برچسب‌ها را برمی‌گرداند
)

# تست
print(clf_en("I am feeling very excited and joyful today!"))
print(clf_en("I am sad and depressed."))


Device set to use cuda:0
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


[[{'label': 'LABEL_13', 'score': 0.7790734767913818}, {'label': 'LABEL_17', 'score': 0.21510884165763855}, {'label': 'LABEL_27', 'score': 0.05265502259135246}, {'label': 'LABEL_0', 'score': 0.0437777116894722}, {'label': 'LABEL_26', 'score': 0.04254526272416115}, {'label': 'LABEL_7', 'score': 0.036241091787815094}, {'label': 'LABEL_8', 'score': 0.027362694963812828}, {'label': 'LABEL_4', 'score': 0.019663216546177864}, {'label': 'LABEL_18', 'score': 0.019117606803774834}, {'label': 'LABEL_20', 'score': 0.015489795245230198}, {'label': 'LABEL_22', 'score': 0.013442710041999817}, {'label': 'LABEL_1', 'score': 0.012346770614385605}, {'label': 'LABEL_15', 'score': 0.01202961802482605}, {'label': 'LABEL_14', 'score': 0.011980804614722729}, {'label': 'LABEL_21', 'score': 0.011965765617787838}, {'label': 'LABEL_3', 'score': 0.011479067616164684}, {'label': 'LABEL_2', 'score': 0.008294496685266495}, {'label': 'LABEL_25', 'score': 0.008273624815046787}, {'label': 'LABEL_6', 'score': 0.007423783

In [ ]:
from transformers import pipeline

clf_en = pipeline(
    "text-classification",
    model="nateraw/bert-base-uncased-emotion",
    return_all_scores=False
)

print(clf_en("I feel really happy today!"))
# [{'label': 'joy', 'score': 0.99}]


Device set to use cuda:0


[{'label': 'joy', 'score': 0.9966766834259033}]


/usr/local/lib/python3.11/dist-packages/transformers/pipelines/text_classification.py:111: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


##متن نهایی

In [ ]:
#  نصب Gradio (اگر هنوز نصب نکرده‌اید)
!pip install -q gradio

import gradio as gr
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
from langdetect import detect

# ۱) آماده‌سازی مدل‌ها و pipelineهای دو‌‌زبانه
# فارسی
fa_tokenizer = AutoTokenizer.from_pretrained("/content/final_parsbert_emotion")
fa_model     = AutoModelForSequenceClassification.from_pretrained("/content/final_parsbert_emotion")
pipe_fa      = pipeline("text-classification", model=fa_model, tokenizer=fa_tokenizer)

FA_LABELS = ['ANGRY','FEAR','HAPPY','HATE','OTHER','SAD','SURPRISE']
fa_map = { idx: ("ANGRY" if lab=="HATE" else lab)
           for idx,lab in enumerate(FA_LABELS) }

# انگلیسی single-label
pipe_en = pipeline(
    "text-classification",
    model="nateraw/bert-base-uncased-emotion",
    top_k=1
)

def map_en(label: str):
    if label in ["joy","love"]: return "HAPPY"
    if label=="sadness":         return "SAD"
    if label=="anger":           return "ANGRY"
    if label=="fear":            return "FEAR"
    if label=="surprise":        return "SURPRISE"
    return "OTHER"

# ۲) نگاشت ایموجی برای نمایش
EMOJI_MAP = {
    "ANGRY":    "😠",
    "FEAR":     "😨",
    "HAPPY":    "😃",
    "SAD":      "😢",
    "SURPRISE": "😲",
    "OTHER":    "😐"
}

# ۳) تابع تحلیل احساس با آستانه
def analyze_interface(text, threshold=0.7):
    lang = detect(text)
    if lang=="fa":
        out = pipe_fa(text)[0]
        idx = int(out['label'].split("_")[-1])
        emotion = fa_map[idx]
        score   = out['score']
    else:
        out = pipe_en(text)[0]
        emotion = map_en(out['label'])
        score   = out['score']
    if score < threshold:
        emotion = "OTHER"
    emoji = EMOJI_MAP.get(emotion, "")
    return emotion, round(score,3), emoji

# ۴) ساخت رابط کاربری با Gradio
with gr.Blocks(css="""
    #root {max-width:600px;margin:auto;padding:20px;}
    .output-box {font-size:1.2rem;text-align:center;margin-top:10px;}
""") as demo:
    gr.Markdown("## 🎵 AI Emotion Detector")
    gr.Markdown("متن خود را وارد کنید تا «حس» شما را تشخیص دهد، همراه با امتیاز اطمینان و ایموجی.")
    with gr.Row():
        txt     = gr.Textbox(label="متن ورودی", placeholder="اینجا بنویسید...", lines=3)
        thresh  = gr.Slider(0,1,value=0.7,step=0.01,label="آستانه برای OTHER")
    btn = gr.Button("تحلیل کن", variant="primary")
    with gr.Row():
        label_out = gr.Textbox(label="احساس شناسایی‌شده", interactive=False, elem_classes="output-box")
        score_out = gr.Textbox(label="درصد اطمینان",       interactive=False, elem_classes="output-box")
        emoji_out = gr.Textbox(label="ایموجی",              interactive=False, elem_classes="output-box")
    btn.click(fn=analyze_interface, inputs=[txt, thresh], outputs=[label_out, score_out, emoji_out])

demo.launch(share=False)


Device set to use cuda:0


config.json:   0%|          | 0.00/768 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cuda:0


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

#تحلیل تصویر

## آماده

In [ ]:
!pip install opencv-python keras numpy tensorflow


In [ ]:
!pip install --upgrade tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 3.1 MB/s eta 0:00:00


In [ ]:

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"  # اجرای روی CPU

import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array

# دانلود مدل
!wget https://github.com/oarriaga/face_classification/raw/master/trained_models/emotion_models/fer2013_mini_XCEPTION.102-0.66.hdf5 -O emotion_model.h5

# لود مدل بدون کامپایل
model = load_model('emotion_model.h5', compile=False)

emotion_labels = ['ANGRY', 'FEAR', 'HAPPY', 'SAD', 'SURPRISE', 'OTHER']

img_path = "/content/face4.jpg"
frame = cv2.imread(img_path)
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
faces = face_classifier.detectMultiScale(gray, 1.3, 5)

for (x, y, w, h) in faces:
    roi_gray = gray[y:y+h, x:x+w]
    roi_gray = cv2.resize(roi_gray, (64, 64), interpolation=cv2.INTER_AREA)

    if np.sum([roi_gray]) != 0:
        roi = roi_gray.astype('float32') / 255.0
        roi = img_to_array(roi)
        roi = np.expand_dims(roi, axis=0)
        roi = roi.reshape((1, 64, 64, 1))

        preds = model.predict(roi)[0]
        label = emotion_labels[np.argmax(preds)]
        confidence = np.max(preds)
        print(f"🧠 Detected Emotion: {label} ({confidence:.2f})")
    else:
        print("🚫 No face detected.")


--2025-08-04 18:54:37--  https://github.com/oarriaga/face_classification/raw/master/trained_models/emotion_models/fer2013_mini_XCEPTION.102-0.66.hdf5
Resolving github.com (github.com)... 140.82.116.3
Connecting to github.com (github.com)|140.82.116.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/oarriaga/face_classification/master/trained_models/emotion_models/fer2013_mini_XCEPTION.102-0.66.hdf5 [following]
--2025-08-04 18:54:37--  https://raw.githubusercontent.com/oarriaga/face_classification/master/trained_models/emotion_models/fer2013_mini_XCEPTION.102-0.66.hdf5
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 872856 (852K) [application/octet-stream]
Saving to: ‘emotion_model.h5’

emotion_m

In [ ]:
import numpy as np
import cv2
from tensorflow.keras.models import load_model
import os
# مطمئن بشویم فایل هست و بارگذاری می‌شود
print("Exists emotion_model.h5:", os.path.exists("emotion_model.h5"))
emotion_cnn = load_model("emotion_model.h5", compile=False)
print("Loaded model, summary:")
emotion_cnn.summary()  # حداقل ببینیم مدل واقعاً لود شد


Exists emotion_model.h5: True
Loaded model, summary:


Model: "model_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_1             │ (None, 64, 64, 1) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 62, 62, 8) │         72 │ input_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 62, 62, 8) │         32 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 62, 62, 8) │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 60, 60, 8) │        576 │ activation_1[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 60, 60, 8) │         32 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 60, 60, 8) │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ separable_conv2d_1  │ (None, 60, 60,    │        200 │ activation_2[0][… │
│ (SeparableConv2D)   │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 60, 60,    │         64 │ separable_conv2d… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 60, 60,    │          0 │ batch_normalizat… │
│ (Activation)        │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ separable_conv2d_2  │ (None, 60, 60,    │        400 │ activation_3[0][… │
│ (SeparableConv2D)   │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 60, 60,    │         64 │ separable_conv2d… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 30, 30,    │        128 │ activation_2[0][… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 30, 30,    │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 30, 30,    │         64 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 30, 30,    │          0 │ max_pooling2d_1[… │
│                     │ 16)               │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ separable_conv2d_3  │ (None, 30, 30,    │        656 │ add_1[0][0]       │
│ (SeparableConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 58,423 (228.21 KB)

 Trainable params: 56,951 (222.46 KB)

 Non-trainable params: 1,472 (5.75 KB)

## فاین تیون دستی

In [ ]:
# ─── سلول ۱: کلون دیتاست FER-2013 ────────────────────────────────
!git clone https://huggingface.co/datasets/Jeneral/fer-2013


Cloning into 'fer-2013'...
remote: Enumerating objects: 18, done.
remote: Total 18 (delta 0), reused 0 (delta 0), pack-reused 18 (from 1)
Unpacking objects: 100% (18/18), 2.78 KiB | 568.00 KiB/s, done.
Filtering content: 100% (2/2), 55.17 MiB | 19.06 MiB/s, done.


In [ ]:
# ─── سلول ۲: تبدیل .pt → DatasetDict + نگاشت لیبل‌ها ─────────────────

import torch, pickle
from datasets import Dataset, DatasetDict

# ۱) بارگذاری لیست نمونه‌ها با pickle
with open("fer-2013/train.pt","rb") as f:
    train_list = pickle.load(f)
with open("fer-2013/test.pt","rb") as f:
    test_list  = pickle.load(f)

# ۲) تبدیل لیست به Dataset
train_ds = Dataset.from_list(train_list)
test_ds  = Dataset.from_list(test_list)

# ۳) همین‌جا DatasetDict را بسازید (قبل از هیچ map دیگری)
ds = DatasetDict({
    "train": train_ds,
    "test":  test_ds
})

# ۴) نگاشت برچسب‌های رشته‌ای به عدد
label_list = ["angry","disgust","fear","happy","neutral","sad","surprise"]
label2id   = {l:i for i,l in enumerate(label_list)}
id2label   = {i:l for i,l in enumerate(label_list)}

def label_str_to_int(example):
    return {"labels": label2id[example["labels"]]}

ds = ds.map(label_str_to_int, batched=False)

# ۵) دیکد کردن بایت‌ها به تصویر PIL
from PIL import Image
import io

def decode_bytes(example):
    example["image"] = Image.open(io.BytesIO(example["img_bytes"])).convert("RGB")
    return example

ds = ds.map(decode_bytes, remove_columns=["img_bytes"])

print(ds)


# ─── سلول ۳: دیکد کردن بایت‌ها به تصویر PIL ────────────────────────

print(ds["train"][0])
# {'labels': 0, 'image': <PIL.Image.Image image mode=RGB size=48x48>}

# ─── سلول ۴: FeatureExtractor و مدل ViT ───────────────────────────

# ─── Config و FeatureExtractor ──────────────────────
import torch
from transformers import AutoConfig, AutoFeatureExtractor, AutoModelForImageClassification

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# ۷ کلاس پروژه
label_list = ["angry","disgust","fear","happy","neutral","sad","surprise"]
label2id   = {l:i for i,l in enumerate(label_list)}
id2label   = {i:l for i,l in enumerate(label_list)}

# ۴.۱) Config
config = AutoConfig.from_pretrained(
    "WinKawaks/vit-tiny-patch16-224",
    num_labels=len(label_list),
    id2label=id2label,
    label2id=label2id
)

# ۴.۲) FeatureExtractor
feature_extractor = AutoFeatureExtractor.from_pretrained(
    "WinKawaks/vit-tiny-patch16-224",
    size=224
)

# ۴.۳) لود مدل با ignore_mismatched_sizes
model = AutoModelForImageClassification.from_pretrained(
    "WinKawaks/vit-tiny-patch16-224",
    config=config,
    ignore_mismatched_sizes=True
).to(device)

# ۴.۴) فریز کردن همه لایه‌ها به جز head
for name, param in model.named_parameters():
    if not name.startswith("classifier"):
        param.requires_grad = False

print("Trainable params:", sum(p.numel() for p in model.parameters() if p.requires_grad))



# ─── سلول ۵: پیش‌پردازش دسته‌ای و قالب‌دهی ────────────────────────
def preprocess_batch(batch):
    # extractor همزمان لیست تصاویر را resize+normalize می‌کند
    enc = feature_extractor(images=batch["image"], return_tensors="pt")
    enc["labels"] = torch.tensor(batch["labels"], dtype=torch.long)
    return enc

# اعمال به کل دیتاست
ds = ds.map(preprocess_batch, batched=True, remove_columns=["image"])
ds.set_format(type="torch", columns=["pixel_values","labels"])


# ─── سلول ۶: تنظیم Trainer و شروع آموزش ────────────────────────────
# معیار دقت
from transformers import TrainingArguments, Trainer
import numpy as np
import evaluate

metric = evaluate.load("accuracy")
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=-1)
    return metric.compute(predictions=preds, references=p.label_ids)

training_args = TrainingArguments(
    output_dir="./face_emotion_vit",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=5,

    per_device_train_batch_size=4,       # ← خیلی کم
    per_device_eval_batch_size=4,        # ← خیلی کم

    gradient_accumulation_steps=4,       # ← effective batch = 4×4 =16
    gradient_checkpointing=True,         # ← reduce memory for activations

    learning_rate=2e-4,                  # ← اندکی بیشتر برای head
    weight_decay=0.0,

    load_best_model_at_end=True,
    metric_for_best_model="accuracy",

    fp16=torch.cuda.is_available(),      # ← mixed precision
    dataloader_num_workers=2,
    logging_steps=100
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds["train"],
    eval_dataset=ds["test"],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics
)

trainer.train()

trainer.save_model("./face_emotion_vit")
feature_extractor.save_pretrained("./face_emotion_vit")


# ─── سلول ۷: تست نهایی (inference) ────────────────────────────────
from transformers import pipeline
pipe = pipeline(
    "image-classification",
    model="./face_emotion_vit",
    feature_extractor="./face_emotion_vit",
    device=0
)
from PIL import Image
print(pipe(Image.open("face4.jpg"), top_k=3))


Map:   0%|          | 0/28709 [00:00<?, ? examples/s]

Map:   0%|          | 0/7178 [00:00<?, ? examples/s]

Map:   0%|          | 0/28709 [00:00<?, ? examples/s]

Map:   0%|          | 0/7178 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['labels', 'image'],
        num_rows: 28709
    })
    test: Dataset({
        features: ['labels', 'image'],
        num_rows: 7178
    })
})
{'labels': 0, 'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=48x48 at 0x7E144DBEDED0>}
Using device: cuda


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/models/vit/feature_extraction_vit.py:30: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(
Some weights of ViTForImageClassification were not initialized from the model checkpoint at WinKawaks/vit-tiny-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size

Trainable params: 1351


Map:   0%|          | 0/28709 [00:00<?, ? examples/s]

به علت محدودیت های اجرایی برای تصویر و صوت نمیتوان فاین تیون انجام داد و صرفا پایپ لاین انجام می شود...

# تحلیل صوت

In [ ]:
from transformers import pipeline
from pydub import AudioSegment

# لود مدل تحلیل احساس از صوت
model_id = "superb/wav2vec2-base-superb-er"
emotion_pipeline = pipeline("audio-classification", model=model_id)

# تبدیل mp3 به wav با تنظیمات درست
audio = AudioSegment.from_file("/content/input2.wav") #shad
audio = audio.set_frame_rate(16000).set_channels(1)
audio.export("/content/sample.wav", format="wav")

# تست مدل روی فایل صوتی wav
audio_path = "/content/sample.wav"
results = emotion_pipeline(audio_path)
print(results)

Device set to use cuda:0


[{'score': 0.8742848038673401, 'label': 'hap'}, {'score': 0.11109784245491028, 'label': 'sad'}, {'score': 0.014354194514453411, 'label': 'neu'}, {'score': 0.00026313262060284615, 'label': 'ang'}]


# تحلیل احساسات مولتی مودال

In [ ]:
# ─── سلول ۱: نصب کتابخانه‌های مورد نیاز ───────────────────────────────----------------------------------------------------
!pip install -q gradio transformers[sentencepiece] soundfile pillow


# ─── سلول ۲: بارگذاری و آماده‌سازی مدل‌ها ────────────────────────────-------------------------------------------------------
import gradio as gr
from transformers import pipeline
from langdetect import detect
import soundfile as sf
import tempfile

# تعریف ایموجی و فضای ۶ کلاسه
EMOJI = {
    "ANGRY":"😠", "FEAR":"😨", "HAPPY":"😃",
    "SAD":"😢",   "SURPRISE":"😲","OTHER":"😐"
}

# ── Pipeline متن فارسی ──
pipe_fa = pipeline(
    "text-classification",
    model="/content/final_parsbert_emotion",
    tokenizer="/content/final_parsbert_emotion",
    device=0
)

# ── Pipeline متن انگلیسی single-label ──
pipe_en = pipeline(
    "text-classification",
    model="nateraw/bert-base-uncased-emotion",
    tokenizer="nateraw/bert-base-uncased-emotion",
    device=0,
    return_all_scores=False
)

def analyze_text(text, threshold):
    # تشخیص زبان
    lang = detect(text)
    if lang == "fa":
        out = pipe_fa(text)[0]
        idx = int(out["label"].split("_")[-1])
        # نگاشت HATE→ANGRY
        FA = ['ANGRY','FEAR','HAPPY','HATE','OTHER','SAD','SURPRISE']
        label = FA[idx] if FA[idx]!="HATE" else "ANGRY"
        score = out["score"]
    else:
        out = pipe_en(text)[0]
        raw = out["label"].lower()
        # نگاشت برچسب‌های انگلیسی به فضای مشترک
        MAP = {
            "joy":"HAPPY","love":"HAPPY",
            "sadness":"SAD","anger":"ANGRY",
            "fear":"FEAR","surprise":"SURPRISE"
        }
        label = MAP.get(raw, "OTHER")
        score = out["score"]

    if score < threshold:
        label = "OTHER"
    return label, round(score,3), EMOJI[label]



# —— Pipeline ابزار FER برای تصویر ——
!wget https://github.com/oarriaga/face_classification/raw/master/trained_models/emotion_models/fer2013_mini_XCEPTION.102-0.66.hdf5 -O emotion_model.h5

# سلول ۰: مطمئن شدن از نصب tensorflow و opencv
!pip install -q tensorflow opencv-python-headless

import numpy as np
import cv2
from tensorflow.keras.models import load_model

# حتماً بعد از نصب tensorflow اجرا کنید:
emotion_cnn = load_model("emotion_model.h5", compile=False)
FER_LABELS = ['angry','disgust','fear','happy','sad','surprise','neutral']
IMG_MAP = {
  "angry":"ANGRY","disgust":"ANGRY","fear":"FEAR",
  "happy":"HAPPY","sad":"SAD","surprise":"SURPRISE",
  "neutral":"OTHER"
}

face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
)

def analyze_image(img, threshold):
    try:
        gray = cv2.cvtColor(np.array(img.convert("RGB")), cv2.COLOR_RGB2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)

        if len(faces)==0:
            # کراپ از وسط وقتی چهره پیدا نشد
            h,w = gray.shape
            sz = min(h,w)//2
            x = (w-sz)//2; y = (h-sz)//2
            roi = gray[y:y+sz, x:x+sz]
        else:
            x,y,w,h = faces[0]
            roi = gray[y:y+h, x:x+w]

        face = cv2.resize(roi, (64,64), interpolation=cv2.INTER_AREA)
        face = face.astype("float32")/255.0
        face = face.reshape(1,64,64,1)

        preds = emotion_cnn.predict(face, verbose=0)[0]
        idx   = int(np.argmax(preds))
        score = float(preds[idx])
        raw   = FER_LABELS[idx]
        label = IMG_MAP.get(raw, "OTHER")

        # بدون صفر کردن با threshold
        return label, round(score,3), EMOJI[label]
    except Exception:
        return "OTHER", 0.0, EMOJI["OTHER"]



# ── Pipeline صوت (Superb) ──
audio_pipe = pipeline(
    "audio-classification",
    model="superb/wav2vec2-base-superb-er",
    device=0,
    top_k=1
)
def analyze_audio(filepath, threshold):
    # ممکن است فرمت متفاوت باشد؛ مجدد روی 16kHz بنویسیم
    arr, sr = sf.read(filepath)
    tmp = tempfile.NamedTemporaryFile(suffix=".wav", delete=False)
    sf.write(tmp.name, arr, sr)
    out = audio_pipe(tmp.name)[0]
    raw = out["label"].lower()
    score = out["score"]
    AUD_MAP = {
      "hap":"HAPPY","sad":"SAD","ang":"ANGRY",
      "fea":"FEAR","sur":"SURPRISE","neu":"OTHER"
    }
    label = AUD_MAP.get(raw[:3], "OTHER")
    if score < threshold:
        label = "OTHER"
    return label, round(float(score),3), EMOJI[label]




# ─── سلول ۳: تابع چندمودال برای Gradio ──────────────────────────────--------------------------------------------------------------
def analyze_all(text, image, audio_path, threshold):
    if text and text.strip() != "":
        return analyze_text(text, threshold)
    if image is not None:
        return analyze_image(image, threshold)
    if audio_path:
        return analyze_audio(audio_path, threshold)
    return "OTHER", 0.0, EMOJI["OTHER"]





# ─── سلول ۴: ساخت رابط Gradio ───────────────────────────────────────----------------------------------------------------------------
with gr.Blocks(css="""
    #root {max-width:600px;margin:auto;padding:20px;}
    .out {font-size:1.3rem;text-align:center;}
""") as demo:
    gr.Markdown("## 🌈 دستیار چند‌مودال تشخیص احساس")
    gr.Markdown("متن، تصویر یا صوت آپلود کنید تا یکی از ۶ کلاس اصلی را همراه درصد اطمینان و ایموجی ببینید.")
    with gr.Row():
        txt = gr.Textbox(label="متن (فارسی/انگلیسی)", lines=2, placeholder="مثال: من خیلی خوشحالم!")
        img = gr.Image(label="تصویر", type="pil")
        aud = gr.Audio(label="صوت (wav/mp3)", type="filepath")
    thr = gr.Slider(0,1,0.7,step=0.01, label="آستانه OTHER")
    btn = gr.Button("تحلیل کن", variant="primary")
    with gr.Row():
        o1 = gr.Textbox(label="احساس",    interactive=False, elem_classes="out")
        o2 = gr.Textbox(label="اعتماد",    interactive=False, elem_classes="out")
        o3 = gr.Textbox(label="ایموجی",    interactive=False, elem_classes="out")
    btn.click(analyze_all, [txt, img, aud, thr], [o1, o2, o3])

demo.launch(share=False)


Device set to use cpu
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/768 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

...

preprocessor_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

Device set to use cpu


model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

# تولید موسیقی از احساس تحلیل شده

## آماده

In [ ]:
!pip install -q "numpy<2"


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 110.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [ ]:
# ─── سلول A: پاک کردن همه نسخه‌های قبلی ─────────────────────────
!pip uninstall -y torch torchvision torchaudio audiocraft

# ─── سلول B: نصب نسخه‌ی CPU-only PyTorch و وابستگی‌ها ───────────
# نصب نسخه‌ی CPU-only PyTorch که با NumPy<2 همخوانی داره
!pip install -q \
    torch==2.1.0+cpu torchvision==0.16.0+cpu torchaudio==2.1.0+cpu \
    --extra-index-url https://download.pytorch.org/whl/cpu

# نصب مجدد Audiocraft
!pip install -q audiocraft soundfile


Found existing installation: torch 2.6.0+cu124
Uninstalling torch-2.6.0+cu124:
  Successfully uninstalled torch-2.6.0+cu124
Found existing installation: torchvision 0.21.0+cu124
Uninstalling torchvision-0.21.0+cu124:
  Successfully uninstalled torchvision-0.21.0+cu124
Found existing installation: torchaudio 2.6.0+cu124
Uninstalling torchaudio-2.6.0+cu124:
  Successfully uninstalled torchaudio-2.6.0+cu124
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.9/184.9 MB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 54.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 73.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 635.7/635.7 kB 17.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.4/72.4 kB 7.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━

In [ ]:
import numpy as np, torch, torchaudio
from audiocraft.models import MusicGen

print("numpy:", np.__version__)
print("torch:", torch.__version__)
print("torchaudio:", torchaudio.__version__)

# تست بارگذاری MusicGen روی CPU
device = "cpu"
model = MusicGen.get_pretrained("facebook/musicgen-small").to(device)
print("✅ MusicGen loaded successfully")



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-package

AttributeError: module 'torch' has no attribute 'uint64'

In [ ]:
# ─── سلول ۵: نصب Audiocraft (MusicGen) ─────────────────────────────────────
!pip install -q audiocraft soundfile

# ─── سلول ۶: بارگذاری و پیکربندی MusicGen ─────────────────────────────────
import torch
from audiocraft.models import MusicGen
import soundfile as sf

# ۱) استفاده از GPU در صورت موجود بودن
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

# ۲) بارگذاری مدل پیش‌آموزش‌دیده (small یا medium یا large)
model = MusicGen.get_pretrained("facebook/musicgen-small").to(device)

# ۳) تنظیمات کلی تولید: طول نمونه و تنوع
model.set_generation_params(
    duration=20.0,   # طول صدای خروجی به ثانیه
    top_k=250        # تنوع نتایج
)

# ─── سلول ۷: نگاشت احساس → پرامپت برای موسیقی ───────────────────────────────
EMOTION_TO_PROMPT = {
    "HAPPY":    "An upbeat, joyful pop instrumental, lively tempo, major key",
    "SAD":      "A slow, melancholic piano melody with soft strings, minor key",
    "ANGRY":    "A heavy rock track with distorted electric guitar and strong drums",
    "FEAR":     "An eerie ambient soundscape with dissonant synth textures and low drones",
    "SURPRISE": "A bright cinematic fanfare with brass and uplifting percussion",
    "OTHER":    "A calm ambient background piece with gentle pads and soft textures"
}

def generate_music_from_emotion(emotion_label: str, out_path: str="generated.wav"):
    # انتخاب پرامپت
    prompt = EMOTION_TO_PROMPT.get(emotion_label, EMOTION_TO_PROMPT["OTHER"])
    # تولید موسیقی
    wav = model.generate([prompt])[0]  # خروجی: Tensor شکل (time,)
    # ذخیره روی دیسک
    sf.write(out_path, wav.cpu().numpy(), samplerate=model.sample_rate)
    return out_path

# تست سریع
print("Producing a 10s HAPPY track …")
test_path = generate_music_from_emotion("HAPPY")
print("Saved to", test_path)

# ─── سلول ۸: اتصال به Gradio (تک سلول) ────────────────────────────────────
import gradio as gr

def gradio_generate(label, threshold_dummy=0.0):
    # اینجا label همان خروجی analyze_all است
    path = generate_music_from_emotion(label)
    return path

with gr.Blocks() as music_demo:
    gr.Markdown("## 🎶 تولید موسیقی از روی احساس")
    lbl = gr.Textbox(label="احساس دریافتی (یکی از: HAPPY,SAD,ANGRY,FEAR,SURPRISE,OTHER)")
    btn = gr.Button("تولید موسیقی")
    out_audio = gr.Audio(label="موسیقی تولیدشده")
    btn.click(fn=gradio_generate, inputs=[lbl], outputs=[out_audio])

music_demo.launch(share=False)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 635.7/635.7 kB 10.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.4/72.4 kB 6.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 45.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 76.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.1/87.1 kB 8.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━

OSError: /usr/local/lib/python3.11/dist-packages/torchaudio/lib/libtorchaudio.so: undefined symbol: _ZN2at4_ops10zeros_like4callERKNS_6TensorEN3c108optionalINS5_10ScalarTypeEEENS6_INS5_6LayoutEEENS6_INS5_6DeviceEEENS6_IbEENS6_INS5_12MemoryFormatEEE

## دستی

In [ ]:
# FluidSynth (رندرکننده‌ی MIDI → WAV)
!apt-get -y install fluidsynth

# ماژول پایتونی pyFluidSynth + ابزارها
!pip install -q pyFluidSynth==1.3.3 pretty_midi music21 librosa soundfile numpy midi2audio

# یک SoundFont جنرال خوب (FluidR3)
!wget -q https://github.com/pepaslabs/fluidsynth-soundfonts/raw/master/FluidR3_GM/FluidR3_GM.sf2 -O /content/FluidR3_GM.sf2 \
  || wget -q https://github.com/urish/cyberbotics/raw/master/projects/robotbenchmark/resources/fluidr3/FluidR3_GM.sf2 -O /content/FluidR3_GM.sf2


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fluid-soundfont-gm libevdev2 libfluidsynth3 libgudev-1.0-0 libinput-bin
  libinput10 libinstpatch-1.0-2 libmd4c0 libmtdev1 libqt5core5a libqt5dbus5
  libqt5gui5 libqt5network5 libqt5svg5 libqt5widgets5 libwacom-bin
  libwacom-common libwacom9 libxcb-icccm4 libxcb-image0 libxcb-keysyms1
  libxcb-render-util0 libxcb-util1 libxcb-xinerama0 libxcb-xinput0 libxcb-xkb1
  libxkbcommon-x11-0 qsynth qt5-gtk-platformtheme qttranslations5-l10n
  timgm6mb-soundfont
Suggested packages:
  fluid-soundfont-gs qt5-image-formats-plugins qtwayland5 jackd
The following NEW packages will be installed:
  fluid-soundfont-gm fluidsynth libevdev2 libfluidsynth3 libgudev-1.0-0
  libinput-bin libinput10 libinstpatch-1.0-2 libmd4c0 libmtdev1 libqt5core5a
  libqt5dbus5 libqt5gui5 libqt5network5 libqt5svg5 libqt5widgets5 libwacom-bin
  libwacom-common libwacom9 libx

In [ ]:
# import numpy as np, random, pretty_midi

# SF2_PATH = "/content/FluidR3_GM.sf2"

# EMO_TO_PARAMS = {
#     "HAPPY":    dict(mode="major", bpm=128, bars=8, chord_prog=["I","V","vi","IV"],
#                      melody_density=0.70, drum=True, key=60,   comment="شاد و پرانرژی"),
#     "SAD":      dict(mode="minor", bpm=72,  bars=8, chord_prog=["i","VI","III","VII"],
#                      melody_density=0.35, drum=False, key=57,  comment="کند و احساسی"),
#     "ANGRY":    dict(mode="minor", bpm=150, bars=8, chord_prog=["i","bVI","bVII","i"],
#                      melody_density=0.85, drum=True, key=50,   comment="تهاجمی و کوبنده"),
#     "FEAR":     dict(mode="minor", bpm=60,  bars=8, chord_prog=["i","bII°","i","bVI"],
#                      melody_density=0.30, drum=False, key=57,  comment="مرموز و تیره"),
#     "SURPRISE": dict(mode="major", bpm=132, bars=8, chord_prog=["I","V","IV","♭VII"],
#                      melody_density=0.75, drum=True, key=65,   comment="جهشی/سینکوپ"),
#     "OTHER":    dict(mode="major", bpm=96,  bars=8, chord_prog=["I","ii","IV","V"],
#                      melody_density=0.50, drum=False, key=60,  comment="خنثی/پس‌زمینه"),
# }

# MAJOR_ROMAN = {"I":(0,"maj"),"ii":(2,"min"),"iii":(4,"min"),"IV":(5,"maj"),
#                "V":(7,"maj"),"vi":(9,"min"),"♭VII":(10,"maj")}
# MINOR_ROMAN = {"i":(0,"min"),"iv":(5,"min"),"v":(7,"min"),"VI":(8,"maj"),
#                "VII":(10,"maj"),"bVI":(8,"maj"),"bVII":(10,"maj"),"bII°":(1,"dim")}
# def chord_intervals(q): return dict(maj=[0,4,7], min=[0,3,7], dim=[0,3,6])[q]

# def generate_midi_for_emotion(emotion:str, seed:int=42, out_mid="gen.mid"):
#     assert emotion in EMO_TO_PARAMS, f"unknown emo {emotion}"
#     p = EMO_TO_PARAMS[emotion]
#     random.seed(seed); np.random.seed(seed)

#     pm = pretty_midi.PrettyMIDI()
#     beats_per_bar = 4

#     inst_har = pretty_midi.Instrument(program=48)  # Strings
#     inst_mel = pretty_midi.Instrument(program=0)   # Piano
#     pm.instruments += [inst_har, inst_mel]
#     inst_drm = pretty_midi.Instrument(program=0, is_drum=True) if p["drum"] else None
#     if inst_drm: pm.instruments.append(inst_drm)

#     ROM = MAJOR_ROMAN if p["mode"]=="major" else MINOR_ROMAN
#     def add_chord(bar_idx, roman):
#         off, qual = ROM[roman]
#         root = (p["key"] + off) % 128
#         ints = chord_intervals(qual)
#         start = bar_idx * beats_per_bar * (60.0/p["bpm"])
#         end   = (bar_idx+1) * beats_per_bar * (60.0/p["bpm"])
#         for n in [root+i for i in ints]:
#             inst_har.notes.append(pretty_midi.Note(velocity=70, pitch=int(n), start=start, end=end))

#     def add_drums(bar_idx):
#         if not inst_drm: return
#         bar_start = bar_idx * beats_per_bar * (60.0/p["bpm"])
#         beat_dur  = (60.0/p["bpm"])
#         for b in range(beats_per_bar):
#             t = bar_start + b*beat_dur
#             # kick/snare
#             inst_drm.notes.append(pretty_midi.Note(100 if b%2==0 else 90,
#                                                    pitch=36 if b%2==0 else 38, start=t, end=t+0.05))
#             # hats
#             inst_drm.notes.append(pretty_midi.Note(60, pitch=42, start=t, end=t+0.03))
#             inst_drm.notes.append(pretty_midi.Note(60, pitch=42, start=t+beat_dur/2, end=t+beat_dur/2+0.03))

#     scale = [0,2,4,5,7,9,11] if p["mode"]=="major" else [0,2,3,5,7,8,10]
#     for i in range(p["bars"]):
#         add_chord(i, p["chord_prog"][i % len(p["chord_prog"])])
#         add_drums(i)

#     density = p["melody_density"]
#     total_sub = p["bars"]*beats_per_bar*2  # 8th notes
#     tonic = p["key"]
#     lo, hi = tonic-5, tonic+(14 if p["mode"]=="major" else 12)
#     t = 0.0; step = 0.5*(60.0/p["bpm"]); last_pitch = tonic+7
#     for s in range(total_sub):
#         if np.random.rand() < density:
#             pc = random.choice(scale)
#             base = tonic + pc
#             cand = base + random.choice([-12,0,12])
#             pitch = int(np.clip(int(0.7*last_pitch + 0.3*cand), lo, hi))
#             dur = step if np.random.rand()<0.7 else step*2
#             inst_mel.notes.append(pretty_midi.Note(velocity=80, pitch=pitch, start=t, end=t+dur))
#             last_pitch = pitch
#         t += step

#     pm.write(out_mid)
#     return out_mid, p


import time, os, numpy as np, random, pretty_midi
import soundfile as sf
from midi2audio import FluidSynth

SF2_PATH = "/content/FluidR3_GM.sf2"   # اگر SoundFont دیگری داری همینجا عوضش کن

# محدوده‌های موسیقایی برای هر احساس (برای تنوع هر بار یک نمونه تصادفی انتخاب می‌کنیم)
EMO_PROFILE = {
    "HAPPY": {
        "mode": "major",
        "tempo_range": (115, 140),
        "base_keys":  [60, 62, 65, 67],                 # C, D, F, G
        "progressions": [
            ["I","V","vi","IV"], ["I","IV","V","I"], ["I","vi","IV","V"]
        ],
        "melody_density_range": (0.55, 0.85),
        "drums": True,
        "comment": "شاد و پرانرژی"
    },
    "SAD": {
        "mode": "minor",
        "tempo_range": (60, 85),
        "base_keys":  [57, 55, 52],                     # A, G, E
        "progressions": [
            ["i","VI","III","VII"], ["i","iv","i","VII"]
        ],
        "melody_density_range": (0.25, 0.45),
        "drums": False,
        "comment": "کند و احساسی"
    },
    "ANGRY": {
        "mode": "minor",
        "tempo_range": (130, 170),
        "base_keys":  [50, 48, 53],                     # D, C, F
        "progressions": [
            ["i","bVI","bVII","i"], ["i","bII°","bVII","i"]
        ],
        "melody_density_range": (0.7, 0.95),
        "drums": True,
        "comment": "تهاجمی و کوبنده"
    },
    "FEAR": {
        "mode": "minor",
        "tempo_range": (50, 75),
        "base_keys":  [57, 58, 55],                     # A, Bb, G
        "progressions": [
            ["i","bII°","i","bVI"], ["i","iv","bII°","i"]
        ],
        "melody_density_range": (0.2, 0.4),
        "drums": False,
        "comment": "مرموز و تیره"
    },
    "SURPRISE": {
        "mode": "major",
        "tempo_range": (120, 145),
        "base_keys":  [65, 67, 69],                     # F, G, A
        "progressions": [
            ["I","V","IV","♭VII"], ["I","IV","V","♭VII"]
        ],
        "melody_density_range": (0.6, 0.9),
        "drums": True,
        "comment": "جهشی/سینکوپ"
    },
    "OTHER": {
        "mode": "major",
        "tempo_range": (85, 110),
        "base_keys":  [60, 62],
        "progressions": [
            ["I","ii","IV","V"], ["I","IV","ii","V"]
        ],
        "melody_density_range": (0.4, 0.65),
        "drums": False,
        "comment": "خنثی/پس‌زمینه"
    },
}

MAJOR_ROMAN = {"I":(0,"maj"),"ii":(2,"min"),"iii":(4,"min"),"IV":(5,"maj"),
               "V":(7,"maj"),"vi":(9,"min"),"♭VII":(10,"maj")}
MINOR_ROMAN = {"i":(0,"min"),"iv":(5,"min"),"v":(7,"min"),"VI":(8,"maj"),
               "VII":(10,"maj"),"bVI":(8,"maj"),"bVII":(10,"maj"),"bII°":(1,"dim")}
QUAL2INTS = {"maj":[0,4,7], "min":[0,3,7], "dim":[0,3,6]}

def _choose(profile, rng):
    bpm = rng.randint(*profile["tempo_range"])
    key = rng.choice(profile["base_keys"])
    prog = rng.choice(profile["progressions"])
    density = rng.uniform(*profile["melody_density_range"])
    return bpm, key, prog, density

def _scale(mode):
    return [0,2,4,5,7,9,11] if mode=="major" else [0,2,3,5,7,8,10]

def _add_chord(inst, start, end, root, qual, human_vel=70, rng=None):
    ints = QUAL2INTS[qual]
    for i in ints:
        vel = int(np.clip(human_vel + (rng.randint(-6,6) if rng else 0), 40, 100))
        inst.notes.append(pretty_midi.Note(velocity=vel, pitch=int(root+i), start=start, end=end))

def _drum_bar(inst_drm, bar_start, beat_dur, rng):
    for b in range(4):
        t = bar_start + b*beat_dur
        # Kick/Snare
        inst_drm.notes.append(pretty_midi.Note(100 if b%2==0 else 90,
                                               pitch=36 if b%2==0 else 38,
                                               start=t, end=t+0.05))
        # Hats (با کمی جیتتر)
        for off in [0, beat_dur/2]:
            jit = rng.uniform(-0.01, 0.01)
            inst_drm.notes.append(pretty_midi.Note(60, pitch=42, start=t+off+jit, end=t+off+jit+0.03))

def _drum_fill(inst_drm, bar_start, beat_dur, rng):
    # یک فیل ساده تام‌ها در انتهای سکشن
    for i, pitch in enumerate([47,45,43,41]):  # toms
        t = bar_start + i*(beat_dur/2)
        inst_drm.notes.append(pretty_midi.Note(85+rng.randint(-5,5), pitch=pitch, start=t, end=t+0.12))

def _melody_motif(scale, rng, length=4):
    return [rng.choice(scale) for _ in range(length)]

def _add_melody(inst, start, bars, bpm, mode, key, density, rng):
    scale = _scale(mode)
    step = 0.5*(60.0/bpm)       # 8th notes
    total_sub = int(bars*4*2)
    t = start
    last_pitch = key + 7
    motif = _melody_motif(scale, rng, length=rng.choice([3,4,5]))

    lo, hi = key-5, key+(14 if mode=="major" else 12)
    for s in range(total_sub):
        if rng.random() < density:
            if rng.random() < 0.25:
                deg = rng.choice(scale)            # گاهی خارج از موتیف
            else:
                deg = motif[s % len(motif)]
            base = key + deg + rng.choice([-12,0,12])
            pitch = int(np.clip(int(0.7*last_pitch + 0.3*base), lo, hi))
            jitter = rng.uniform(-0.02, 0.02)
            dur = step if rng.random()<0.7 else step*2
            vel = int(np.clip(75 + rng.randint(-12,12), 40, 110))
            inst.notes.append(pretty_midi.Note(velocity=vel, pitch=pitch, start=t+jitter, end=t+jitter+dur))
            last_pitch = pitch
        t += step

def _roman_dict(mode): return MAJOR_ROMAN if mode=="major" else MINOR_ROMAN

def generate_full_track(emotion: str, minutes: float = 2.5, seed: int | None = None,
                        out_mid: str | None = None, out_wav: str | None = None):
    assert emotion in EMO_PROFILE, f"unknown emotion {emotion}"
    rng = random.Random(seed if seed is not None else int(time.time()*1000) % 2_147_483_647)
    prof = EMO_PROFILE[emotion]
    bpm, key, progression, density = _choose(prof, rng)

    # محاسبه تعداد میزان‌ها برای زمان هدف
    sec_per_bar = 4*(60.0/bpm)
    total_bars = max(16, int((minutes*60)/sec_per_bar))   # حداقل 16 میزان برای ساختار
    # ساختار ساده: A (40%) – B (40%) – A' (20%)
    bars_A = int(total_bars*0.4)
    bars_B = int(total_bars*0.4)
    bars_A2 = total_bars - (bars_A+bars_B)

    pm = pretty_midi.PrettyMIDI()
    inst_har = pretty_midi.Instrument(program=48)    # Strings pad
    inst_mel = pretty_midi.Instrument(program=0)     # Piano
    pm.instruments += [inst_har, inst_mel]
    inst_drm = pretty_midi.Instrument(program=0, is_drum=True) if prof["drums"] else None
    if inst_drm: pm.instruments.append(inst_drm)

    ROM = _roman_dict(prof["mode"])
    beat_dur = (60.0/bpm)

    def add_section(start_bar, num_bars, key_this, progression_this, density_mul=1.0, final_fill=False):
        for i in range(num_bars):
            roman = progression_this[i % len(progression_this)]
            off, qual = ROM[roman]
            root = (key_this + off) % 128
            start = (start_bar+i)*4*beat_dur
            end   = (start_bar+i+1)*4*beat_dur
            _add_chord(inst_har, start, end, root, qual, rng=rng)
            if inst_drm: _drum_bar(inst_drm, start, beat_dur, rng)
        _add_melody(inst_mel, start_bar*4*beat_dur, num_bars, bpm, prof["mode"], key_this, density*density_mul, rng)
        if final_fill and inst_drm:
            _drum_fill(inst_drm, (start_bar+num_bars-1)*4*beat_dur, beat_dur, rng)

    # سکشن A
    add_section(0, bars_A, key, progression, density_mul=1.0, final_fill=True)
    # سکشن B با تغییرات: احتمال مدولاسیون + تغییر پروگرشن و دانسیته
    mod_key = key + rng.choice([-2, 2, 0])   # مقداری غافلگیری
    new_prog = rng.choice(prof["progressions"])
    add_section(bars_A, bars_B, mod_key, new_prog, density_mul=rng.uniform(0.9, 1.2), final_fill=True)
    # سکشن A’ (بازگشت با کمی تغییر ملودی)
    add_section(bars_A+bars_B, bars_A2, key, progression, density_mul=rng.uniform(0.8, 1.1), final_fill=False)

    # خروجی‌ها
    stamp = int(time.time())
    out_mid = out_mid or f"{emotion.lower()}_{minutes:.1f}min_{stamp}.mid"
    out_wav = out_wav or f"{emotion.lower()}_{minutes:.1f}min_{stamp}.wav"
    pm.write(out_mid)

    # رندر به WAV (اول تلاش با pyfluidsynth؛ در صورت خطا → midi2audio)
    try:
        audio = pm.fluidsynth(fs=22050, sf2_path=SF2_PATH)
        sf.write(out_wav, audio, 22050)
    except Exception as e:
        print("[render] fallback to midi2audio:", e)
        FluidSynth(sound_font=SF2_PATH, sample_rate=22050).midi_to_audio(out_mid, out_wav)

    meta = {
        "emotion": emotion,
        "minutes": minutes,
        "bpm": bpm,
        "mode": prof["mode"],
        "key_midi": key,
        "progression_A": progression,
        "progression_B": new_prog,
        "drums": bool(inst_drm),
        "comment": prof["comment"],
        "bars_total": total_bars
    }
    return out_mid, out_wav, meta


In [ ]:
import soundfile as sf
from midi2audio import FluidSynth

def midi_to_wav(mid_path, wav_path="gen.wav", sr=22050, sf2_path=SF2_PATH):
    # مسیر A: استفاده از pretty_midi.fluidsynth (نیاز به pyFluidSynth)
    try:
        audio = pretty_midi.PrettyMIDI(mid_path).fluidsynth(fs=sr, sf2_path=sf2_path)
        sf.write(wav_path, audio, sr)
        return wav_path
    except Exception as e:
        print("[midi_to_wav] fallback to midi2audio:", e)
        # مسیر B: midi2audio (از باینری fluidsynth استفاده می‌کند)
        FluidSynth(sound_font=sf2_path, sample_rate=sr).midi_to_audio(mid_path, wav_path)
        return wav_path


In [ ]:
import librosa

TARGETS = {
    "HAPPY":    dict(mode="major", tempo=(115,140), density=(6,10), want_dissonance=False, want_syncopation=True),
    "SAD":      dict(mode="minor", tempo=(60,85),   density=(2,5),  want_dissonance=False, want_syncopation=False),
    "ANGRY":    dict(mode="minor", tempo=(130,170), density=(8,14), want_dissonance=True,  want_syncopation=True),
    "FEAR":     dict(mode="minor", tempo=(50,75),   density=(1,4),  want_dissonance=True,  want_syncopation=False),
    "SURPRISE": dict(mode="major", tempo=(115,145), density=(6,12), want_dissonance=False, want_syncopation=True),
    "OTHER":    dict(mode="major", tempo=(85,110),  density=(4,8),  want_dissonance=False, want_syncopation=False),
}

def analyze_midi(mid_path, emotion):
    pm = pretty_midi.PrettyMIDI(mid_path)
    mel = next((ins for ins in pm.instruments if not ins.is_drum), None)
    note_count = len(mel.notes) if mel else 0
    bars = EMO_TO_PARAMS[emotion]["bars"]
    density = note_count / max(1,bars)

    bpm_guess = EMO_TO_PARAMS[emotion]["bpm"]

    has_dim_like = any((n.end-n.start)<0.15 for ins in pm.instruments for n in ins.notes if not ins.is_drum)
    step = 60.0/bpm_guess/2
    sync_onsets = 0
    if mel:
        for n in mel.notes:
            frac = (n.start/step) % 2
            if 0.4 < frac < 1.6: sync_onsets += 1
    sync_ratio = (sync_onsets / max(1, len(mel.notes))) if mel else 0.0

    target = TARGETS[emotion]
    mode_match = 1.0 if (EMO_TO_PARAMS[emotion]["mode"] == target["mode"]) else 0.0
    tempo_fit  = 1.0 if (target["tempo"][0] <= bpm_guess <= target["tempo"][1]) else 0.0
    d_lo, d_hi = target["density"]
    if density<d_lo: density_fit = max(0.0, 1 - (d_lo-density)/d_lo)
    elif density>d_hi: density_fit = max(0.0, 1 - (density-d_hi)/d_hi)
    else: density_fit = 1.0
    disson_fit = 1.0 if (target["want_dissonance"] and has_dim_like) or (not target["want_dissonance"] and not has_dim_like) else 0.5
    sync_fit   = min(1.0, sync_ratio*1.5) if target["want_syncopation"] else (1.0 - min(1.0, sync_ratio*1.5))

    score = 0.25*mode_match + 0.25*tempo_fit + 0.25*density_fit + 0.15*disson_fit + 0.10*sync_fit
    return {
        "bpm": bpm_guess, "bars": bars,
        "note_density_per_bar": round(density,2),
        "mode_match": round(mode_match,2),
        "tempo_fit": round(tempo_fit,2),
        "density_fit": round(density_fit,2),
        "dissonance_fit": round(disson_fit,2),
        "syncopation_fit": round(sync_fit,2),
        "overall_structural_score": round(float(score),3),
    }


In [ ]:
# from transformers import pipeline

# # کلاس‌فایر احساس صوت
# audio_clf = pipeline("audio-classification", model="superb/wav2vec2-base-superb-er", top_k=None)

# AUD_MAP = {
#   "hap":"HAPPY","sad":"SAD","ang":"ANGRY",
#   "fea":"FEAR","sur":"SURPRISE","neu":"OTHER"
# }

# def evaluate_with_audio_classifier(wav_path, target_emotion:str):
#     out = audio_clf(wav_path)
#     top = max(out, key=lambda x:x["score"])
#     raw = top["label"].lower()
#     mapped = AUD_MAP.get(raw[:3], "OTHER")
#     prob = float(top["score"])
#     match = 1.0 if mapped==target_emotion else 0.0
#     return {
#         "predicted_emotion_audio": mapped,
#         "predicted_confidence_audio": round(prob,3),
#         "target_emotion": target_emotion,
#         "audio_match": match
#     }


from transformers import pipeline

# ارزیابی صوتی با superb (اختیاری: اگر نمی‌خوای هر بار دانلود شه، یک بار بالا صداش بزن)
audio_clf = pipeline("audio-classification", model="superb/wav2vec2-base-superb-er", top_k=None)
AUD_MAP = { "hap":"HAPPY","sad":"SAD","ang":"ANGRY","fea":"FEAR","sur":"SURPRISE","neu":"OTHER" }

def analyze_midi_struct(mid_path, meta):
    pm = pretty_midi.PrettyMIDI(mid_path)
    mel = next((ins for ins in pm.instruments if not ins.is_drum), None)
    note_count = len(mel.notes) if mel else 0
    density = note_count / max(1, meta["bars_total"])
    bpm = meta["bpm"]
    # سیمپل همون فرمول ساده‌ی قبل
    has_dim_like = any((n.end-n.start)<0.15 for ins in pm.instruments for n in ins.notes if not ins.is_drum)
    step = 60.0/bpm/2
    sync_onsets = 0
    if mel:
        for n in mel.notes:
            frac = (n.start/step) % 2
            if 0.4 < frac < 1.6: sync_onsets += 1
    sync_ratio = (sync_onsets / max(1, len(mel.notes))) if mel else 0.0

    # حدود هدف
    TARGETS = {
        "HAPPY":    dict(mode="major", tempo=(115,140), density=(6,10), want_dissonance=False, want_syncopation=True),
        "SAD":      dict(mode="minor", tempo=(60,85),   density=(2,5),  want_dissonance=False, want_syncopation=False),
        "ANGRY":    dict(mode="minor", tempo=(130,170), density=(8,14), want_dissonance=True,  want_syncopation=True),
        "FEAR":     dict(mode="minor", tempo=(50,75),   density=(1,4),  want_dissonance=True,  want_syncopation=False),
        "SURPRISE": dict(mode="major", tempo=(115,145), density=(6,12), want_dissonance=False, want_syncopation=True),
        "OTHER":    dict(mode="major", tempo=(85,110),  density=(4,8),  want_dissonance=False, want_syncopation=False),
    }
    target = TARGETS[meta["emotion"]]
    mode_match = 1.0 if (meta["mode"] == target["mode"]) else 0.0
    tempo_fit  = 1.0 if (target["tempo"][0] <= bpm <= target["tempo"][1]) else 0.0
    d_lo, d_hi = target["density"]
    if density<d_lo: density_fit = max(0.0, 1 - (d_lo-density)/max(d_lo,1))
    elif density>d_hi: density_fit = max(0.0, 1 - (density-d_hi)/max(d_hi,1))
    else: density_fit = 1.0
    disson_fit = 1.0 if (target["want_dissonance"] and has_dim_like) or (not target["want_dissonance"] and not has_dim_like) else 0.5
    sync_fit   = min(1.0, sync_ratio*1.5) if target["want_syncopation"] else (1.0 - min(1.0, sync_ratio*1.5))
    score = 0.25*mode_match + 0.25*tempo_fit + 0.25*density_fit + 0.15*disson_fit + 0.10*sync_fit

    return {
        "bpm": bpm,
        "note_density_per_bar": round(density,2),
        "mode_match": round(mode_match,2),
        "tempo_fit": round(tempo_fit,2),
        "density_fit": round(density_fit,2),
        "dissonance_fit": round(disson_fit,2),
        "syncopation_fit": round(sync_fit,2),
        "overall_structural_score": round(float(score),3),
    }

def evaluate_audio_emotion(wav_path, target_emotion):
    out = audio_clf(wav_path)
    top = max(out, key=lambda x:x["score"])
    mapped = AUD_MAP.get(top["label"].lower()[:3], "OTHER")
    return {
        "audio_pred": mapped,
        "audio_conf": round(float(top["score"]),3),
        "audio_match": 1.0 if mapped==target_emotion else 0.0
    }

def generate_and_evaluate_full(emotion:str, minutes:float=3.0, seed:int|None=None):
    mid, wav, meta = generate_full_track(emotion, minutes=minutes, seed=seed)
    struct = analyze_midi_struct(mid, meta)
    audio  = evaluate_audio_emotion(wav, emotion)
    final_score = round(0.7*struct["overall_structural_score"] + 0.3*audio["audio_match"], 3)
    report = dict(meta); report.update(struct); report.update(audio); report["final_score"] = final_score
    return wav, report


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/configuration_utils.py:334: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/378M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
def generate_and_evaluate(emotion:str, seed:int=42, out_mid="gen.mid", out_wav="gen.wav"):
    mid, params = generate_midi_for_emotion(emotion, seed=seed, out_mid=out_mid)
    wav = midi_to_wav(mid, wav_path=out_wav, sr=22050, sf2_path=SF2_PATH)
    report = analyze_midi(mid, emotion)
    return wav, report

# مثال اجرا: از لیبل خروجی طبقه‌بندی‌ات استفاده کن
wav_path, report = generate_and_evaluate("HAPPY", seed=123)
print("WAV:", wav_path)
print(report)
# حالا wav را پخش کن یا دانلود کن


WAV: gen.wav
{'bpm': 128, 'bars': 8, 'note_density_per_bar': 3.0, 'mode_match': 1.0, 'tempo_fit': 1.0, 'density_fit': 0.5, 'dissonance_fit': 1.0, 'syncopation_fit': 0.0, 'overall_structural_score': 0.775}


In [ ]:
import gradio as gr

def ui_make_music(label, minutes, seed):
    if not label:
        return None, {"error":"برچسب احساس خالی است!"}
    label = label.strip().upper()
    if label not in EMO_PROFILE:
        return None, {"error": f"برچسب نامعتبر: {label}"}
    wav, rep = generate_and_evaluate_full(label, minutes=minutes, seed=None if seed=="" else int(seed))
    return wav, rep

with gr.Blocks() as music_ui:
    gr.Markdown("### 🎵 ساخت موسیقی از روی احساس")
    emo = gr.Dropdown(choices=list(EMO_PROFILE.keys()), value="HAPPY", label="احساس")
    mins = gr.Slider(1.0, 6.0, value=3.0, step=0.5, label="مدت (دقیقه)")
    seed = gr.Textbox(value="", label="Seed (خالی = خروجی متنوع)")
    go = gr.Button("بساز!")
    audio_out = gr.Audio(label="خروجی موزیک", type="filepath")
    rep_out = gr.JSON(label="گزارش ارزیابی")
    go.click(ui_make_music, [emo, mins, seed], [audio_out, rep_out])

music_ui.launch(share=False)


# کل پروژه از تحلیل احساس تا تولید موسیقی

## فقط درامز و پیانو

In [ ]:
# =======================  یک سلولِ یکپارچه  ==========================
# نصب وابستگی‌ها
!apt-get -qq -y install fluidsynth
!pip install -q gradio transformers[sentencepiece] soundfile pillow pyFluidSynth==1.3.3 \
                pretty_midi music21 librosa midi2audio langdetect tensorflow opencv-python-headless

# SoundFont باکیفیت
!wget -q https://github.com/pepaslabs/fluidsynth-soundfonts/raw/master/FluidR3_GM/FluidR3_GM.sf2 -O /content/FluidR3_GM.sf2 \
  || wget -q https://github.com/urish/cyberbotics/raw/master/projects/robotbenchmark/resources/fluidr3/FluidR3_GM.sf2 -O /content/FluidR3_GM.sf2

# مدل FER (mini_XCEPTION)
!wget -q https://raw.githubusercontent.com/oarriaga/face_classification/master/trained_models/emotion_models/fer2013_mini_XCEPTION.102-0.66.hdf5 -O /content/emotion_model.h5

# ------------------ Import & Config (بستن GPU برای TF) ----------------
import os, io, time, math, random, json
os.environ["CUDA_VISIBLE_DEVICES"] = os.environ.get("CUDA_VISIBLE_DEVICES", "")  # برای Torch می‌گذاریم به حال خودش
# ولی TensorFlow را مجبور به CPU می‌کنیم تا cuDNN ارور نده
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES_TF"] = "-1"   # مارکر محلی (فقط برای وضوح)

import numpy as np
import gradio as gr
import soundfile as sf
from PIL import Image

import torch
from transformers import pipeline

import tensorflow as tf
try:
    tf.config.set_visible_devices([], 'GPU')
except Exception:
    pass

import cv2
from tensorflow.keras.models import load_model

import pretty_midi
from midi2audio import FluidSynth
from langdetect import detect

# ----------------------- Device برای Transformer ----------------------
DEVICE = 0 if torch.cuda.is_available() else -1
print("Device for Transformers:", "cuda:0" if DEVICE==0 else "cpu")
SF2_PATH = "/content/FluidR3_GM.sf2"

# ----------------------------- Emoji & Maps ---------------------------
EMOJI = {"ANGRY":"😠","FEAR":"😨","HAPPY":"😃","SAD":"😢","SURPRISE":"😲","OTHER":"😐"}

# ------------------------- Pipelines متن/صوت --------------------------
pipe_fa = pipeline(
    "text-classification",
    model="/content/final_parsbert_emotion",
    tokenizer="/content/final_parsbert_emotion",
    device=DEVICE
)
pipe_en = pipeline(
    "text-classification",
    model="nateraw/bert-base-uncased-emotion",
    tokenizer="nateraw/bert-base-uncased-emotion",
    device=DEVICE,
    return_all_scores=False
)
def map_en(label):
    l = label.lower()
    if l in ["joy","love"]: return "HAPPY"
    if l == "sadness":      return "SAD"
    if l == "anger":        return "ANGRY"
    if l == "fear":         return "FEAR"
    if l == "surprise":     return "SURPRISE"
    return "OTHER"

audio_clf = pipeline("audio-classification", model="superb/wav2vec2-base-superb-er", device=DEVICE, top_k=None)
AUD_MAP = {"hap":"HAPPY","sad":"SAD","ang":"ANGRY","fea":"FEAR","sur":"SURPRISE","neu":"OTHER"}

# ------------------------- مدل تصویر (TF روی CPU) ---------------------
emotion_cnn = load_model("/content/emotion_model.h5", compile=False)
FER_LABELS = ['angry','disgust','fear','happy','sad','surprise','neutral']
IMG_MAP = {
    "angry":"ANGRY","disgust":"ANGRY","fear":"FEAR",
    "happy":"HAPPY","sad":"SAD","surprise":"SURPRISE","neutral":"OTHER"
}
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# -------------------------- تشخیص احساس‌ها ----------------------------
def analyze_text(text, threshold):
    lang = "fa"
    try:
        lang = detect(text)
    except:
        pass
    if lang == "fa":
        out = pipe_fa(text)[0]
        idx = int(out["label"].split("_")[-1])
        FA = ['ANGRY','FEAR','HAPPY','HATE','OTHER','SAD','SURPRISE']
        label = FA[idx] if FA[idx]!="HATE" else "ANGRY"
        score = float(out["score"])
    else:
        out = pipe_en(text)[0]
        label = map_en(out["label"])
        score = float(out["score"])
    if score < threshold:
        label = "OTHER"
    return label, round(score,3), EMOJI[label]

def analyze_image(img, threshold):
    # بدون threshold صفرکن؛ فقط خروجی واقعی مدل را بده
    try:
        gray = cv2.cvtColor(np.array(img.convert("RGB")), cv2.COLOR_RGB2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)
        if len(faces)==0:
            h,w = gray.shape
            sz = min(h,w)//2
            x = (w-sz)//2; y = (h-sz)//2
            roi = gray[y:y+sz, x:x+sz]
        else:
            x,y,w,h = faces[0]
            roi = gray[y:y+h, x:x+w]
        face = cv2.resize(roi, (64,64), interpolation=cv2.INTER_AREA)
        face = face.astype("float32")/255.0
        face = face.reshape(1,64,64,1)
        preds = emotion_cnn.predict(face, verbose=0)[0]
        idx   = int(np.argmax(preds))
        score = float(preds[idx])
        raw   = FER_LABELS[idx]
        label = IMG_MAP.get(raw, "OTHER")
        return label, round(score,3), EMOJI[label]
    except Exception as e:
        # لاگ کوچک برای دیباگ
        print("[analyze_image error]", repr(e))
        return "OTHER", 0.0, EMOJI["OTHER"]

def analyze_audio(filepath, threshold):
    try:
        out = audio_clf(filepath)
        if not out:
            return "OTHER", 0.0, EMOJI["OTHER"]
        top = max(out, key=lambda x: x["score"])
        raw = top["label"].lower()
        score = float(top["score"])
        label = AUD_MAP.get(raw[:3], "OTHER")
        if score < threshold:
            label = "OTHER"
        return label, round(score,3), EMOJI[label]
    except Exception as e:
        print("[analyze_audio error]", repr(e))
        return "OTHER", 0.0, EMOJI["OTHER"]

def analyze_all(text, image, audio_path, threshold):
    if text and str(text).strip():
        return analyze_text(text, threshold)
    if image is not None:
        return analyze_image(image, threshold)
    if audio_path:
        return analyze_audio(audio_path, threshold)
    return "OTHER", 0.0, EMOJI["OTHER"]

# --------------------- تنظیمات ساخت موسیقی (Rule-based) ----------------
EMO_PROFILE = {
    "HAPPY":    {"mode":"major", "tempo_range":(115,140), "base_keys":[60,62,65,67],
                 "progressions":[["I","V","vi","IV"],["I","IV","V","I"],["I","vi","IV","V"]],
                 "melody_density_range":(0.55,0.85), "drums":True,  "comment":"شاد و پرانرژی"},
    "SAD":      {"mode":"minor", "tempo_range":(60,85),   "base_keys":[57,55,52],
                 "progressions":[["i","VI","III","VII"],["i","iv","i","VII"]],
                 "melody_density_range":(0.25,0.45), "drums":False, "comment":"کند و احساسی"},
    "ANGRY":    {"mode":"minor", "tempo_range":(130,170), "base_keys":[50,48,53],
                 "progressions":[["i","bVI","bVII","i"],["i","bII°","bVII","i"]],
                 "melody_density_range":(0.7,0.95),  "drums":True,  "comment":"تهاجمی و کوبنده"},
    "FEAR":     {"mode":"minor", "tempo_range":(50,75),   "base_keys":[57,58,55],
                 "progressions":[["i","bII°","i","bVI"],["i","iv","bII°","i"]],
                 "melody_density_range":(0.2,0.4),   "drums":False, "comment":"مرموز و تیره"},
    "SURPRISE": {"mode":"major", "tempo_range":(120,145), "base_keys":[65,67,69],
                 "progressions":[["I","V","IV","♭VII"],["I","IV","V","♭VII"]],
                 "melody_density_range":(0.6,0.9),   "drums":True,  "comment":"جهشی/سینکوپ"},
    "OTHER":    {"mode":"major", "tempo_range":(85,110),  "base_keys":[60,62],
                 "progressions":[["I","ii","IV","V"],["I","IV","ii","V"]],
                 "melody_density_range":(0.4,0.65),  "drums":False, "comment":"خنثی/پس‌زمینه"},
}
MAJOR_ROMAN = {"I":(0,"maj"),"ii":(2,"min"),"iii":(4,"min"),"IV":(5,"maj"),"V":(7,"maj"),"vi":(9,"min"),"♭VII":(10,"maj")}
MINOR_ROMAN = {"i":(0,"min"),"iv":(5,"min"),"v":(7,"min"),"VI":(8,"maj"),"VII":(10,"maj"),"bVI":(8,"maj"),"bVII":(10,"maj"),"bII°":(1,"dim")}
QUAL2INTS = {"maj":[0,4,7], "min":[0,3,7], "dim":[0,3,6]}

def _scale(mode): return [0,2,4,5,7,9,11] if mode=="major" else [0,2,3,5,7,8,10]
def _choose(prof, rng):
    bpm = rng.randint(*prof["tempo_range"])
    key = rng.choice(prof["base_keys"])
    prog = rng.choice(prof["progressions"])
    density = rng.uniform(*prof["melody_density_range"])
    return bpm, key, prog, density

def _add_chord(inst, start, end, root, qual, rng=None):
    for i in QUAL2INTS[qual]:
        vel = int(np.clip(72 + (rng.randint(-6,6) if rng else 0), 40, 100))
        inst.notes.append(pretty_midi.Note(velocity=vel, pitch=int(root+i), start=start, end=end))

def _drum_bar(inst_drm, bar_start, beat_dur, rng):
    for b in range(4):
        t = bar_start + b*beat_dur
        inst_drm.notes.append(pretty_midi.Note(100 if b%2==0 else 90, pitch=36 if b%2==0 else 38, start=t, end=t+0.06))
        for off in [0, beat_dur/2]:
            jit = rng.uniform(-0.01, 0.01)
            inst_drm.notes.append(pretty_midi.Note(62, pitch=42, start=t+off+jit, end=t+off+jit+0.03))

def _drum_fill(inst_drm, bar_start, beat_dur, rng):
    for i, pitch in enumerate([47,45,43,41]):  # toms
        t = bar_start + i*(beat_dur/2)
        inst_drm.notes.append(pretty_midi.Note(86+rng.randint(-5,5), pitch=pitch, start=t, end=t+0.12))

def _add_melody(inst, start, bars, bpm, mode, key, density, rng):
    scale = _scale(mode); step = 0.5*(60.0/bpm)
    total_sub = int(bars*4*2)
    t = start; last_pitch = key+7
    lo, hi = key-5, key+(14 if mode=="major" else 12)
    motif = [rng.choice(scale) for _ in range(rng.choice([3,4,5]))]
    for s in range(total_sub):
        if rng.random() < density:
            deg = (motif[s % len(motif)]) if rng.random()<0.75 else rng.choice(scale)
            base = key + deg + rng.choice([-12,0,12])
            pitch = int(np.clip(int(0.7*last_pitch + 0.3*base), lo, hi))
            jitter = rng.uniform(-0.02, 0.02)
            dur = step if rng.random()<0.7 else step*2
            vel = int(np.clip(78 + rng.randint(-12,12), 40, 110))
            inst.notes.append(pretty_midi.Note(velocity=vel, pitch=pitch, start=t+jitter, end=t+jitter+dur))
            last_pitch = pitch
        t += step

def generate_full_track(emotion: str, minutes: float = 3.0, seed: int | None = None):
    assert emotion in EMO_PROFILE, f"unknown emotion {emotion}"
    rng = random.Random(seed if seed is not None else int(time.time()*1000) % 2_147_483_647)
    prof = EMO_PROFILE[emotion]
    bpm, key, progression, density = _choose(prof, rng)
    sec_per_bar = 4*(60.0/bpm)
    total_bars = max(16, int((minutes*60)/sec_per_bar))
    bars_A = int(total_bars*0.4); bars_B = int(total_bars*0.4); bars_A2 = total_bars - (bars_A+bars_B)

    pm = pretty_midi.PrettyMIDI()
    inst_har = pretty_midi.Instrument(program=rng.choice([48,49,50,51,52]))
    inst_mel = pretty_midi.Instrument(program=rng.choice([0,1,4,5,6,16,24]))
    pm.instruments += [inst_har, inst_mel]
    inst_drm = pretty_midi.Instrument(program=0, is_drum=True) if prof["drums"] else None
    if inst_drm: pm.instruments.append(inst_drm)
    beat_dur = (60.0/bpm)
    ROM = MAJOR_ROMAN if prof["mode"]=="major" else MINOR_ROMAN

    def add_section(start_bar, num_bars, key_this, progression_this, density_mul=1.0, final_fill=False):
        for i in range(num_bars):
            roman = progression_this[i % len(progression_this)]
            off, qual = ROM[roman]
            root = (key_this + off) % 128
            start = (start_bar+i)*4*beat_dur
            end   = (start_bar+i+1)*4*beat_dur
            _add_chord(inst_har, start, end, root, qual, rng)
            if inst_drm: _drum_bar(inst_drm, start, beat_dur, rng)
        _add_melody(inst_mel, start_bar*4*beat_dur, num_bars, bpm, prof["mode"], key_this, density*density_mul, rng)
        if final_fill and inst_drm:
            _drum_fill(inst_drm, (start_bar+num_bars-1)*4*beat_dur, beat_dur, rng)

    add_section(0, bars_A, key, progression, density_mul=1.0, final_fill=True)
    mod_key = key + rng.choice([-2, 2, 0])
    new_prog = rng.choice(prof["progressions"])
    add_section(bars_A, bars_B, mod_key, new_prog, density_mul=random.uniform(0.9, 1.2), final_fill=True)
    add_section(bars_A+bars_B, bars_A2, key, progression, density_mul=random.uniform(0.8, 1.1), final_fill=False)

    stamp = int(time.time())
    out_mid = f"{emotion.lower()}_{minutes:.1f}min_{stamp}.mid"
    out_wav = f"{emotion.lower()}_{minutes:.1f}min_{stamp}.wav"
    pm.write(out_mid)

    # رندر WAV
    try:
        audio = pm.fluidsynth(fs=22050, sf2_path=SF2_PATH)
        sf.write(out_wav, audio, 22050)
    except Exception as e:
        print("[render fallback]", e)
        FluidSynth(sound_font=SF2_PATH, sample_rate=22050).midi_to_audio(out_mid, out_wav)

    meta = {"emotion": emotion, "minutes": minutes, "bpm": bpm, "mode": prof["mode"],
            "key_midi": key, "progression_A": progression, "progression_B": new_prog,
            "drums": bool(inst_drm), "comment": prof["comment"], "bars_total": total_bars}
    return out_mid, out_wav, meta

# ---------------------------- ارزیابی موسیقی --------------------------
TARGETS = {
    "HAPPY":    dict(mode="major", tempo=(115,140), density=(6,10), want_dissonance=False, want_syncopation=True),
    "SAD":      dict(mode="minor", tempo=(60,85),   density=(2,5),  want_dissonance=False, want_syncopation=False),
    "ANGRY":    dict(mode="minor", tempo=(130,170), density=(8,14), want_dissonance=True,  want_syncopation=True),
    "FEAR":     dict(mode="minor", tempo=(50,75),   density=(1,4),  want_dissonance=True,  want_syncopation=False),
    "SURPRISE": dict(mode="major", tempo=(115,145), density=(6,12), want_dissonance=False, want_syncopation=True),
    "OTHER":    dict(mode="major", tempo=(85,110),  density=(4,8),  want_dissonance=False, want_syncopation=False),
}

def analyze_midi_struct(mid_path, emotion, build_params):
    pm = pretty_midi.PrettyMIDI(mid_path)
    mel = next((ins for ins in pm.instruments if not ins.is_drum), None)
    note_count = len(mel.notes) if mel else 0
    bars = build_params["bars_total"]
    density = note_count / max(1,bars)
    bpm_guess = build_params["bpm"]
    has_dim_like = any((n.end-n.start) < (60.0/bpm_guess)/4 for ins in pm.instruments for n in ins.notes if not ins.is_drum)
    step = 60.0/bpm_guess/2
    sync_onsets = 0
    if mel and mel.notes:
        for n in mel.notes:
            frac = (n.start/step) % 2
            if 0.4 < frac < 1.6: sync_onsets += 1
    sync_ratio = (sync_onsets / max(1, len(mel.notes))) if (mel and mel.notes) else 0.0

    target = TARGETS[emotion]
    mode_match = 1.0 if (build_params["mode"] == target["mode"]) else 0.0
    tempo_fit  = 1.0 if (target["tempo"][0] <= bpm_guess <= target["tempo"][1]) else 0.0
    d_lo, d_hi = target["density"]
    if density<d_lo: density_fit = max(0.0, 1 - (d_lo-density)/max(1e-6,d_lo))
    elif density>d_hi: density_fit = max(0.0, 1 - (density-d_hi)/max(1e-6,d_hi))
    else: density_fit = 1.0
    sync_fit   = min(1.0, sync_ratio*1.5) if target["want_syncopation"] else (1.0 - min(1.0, sync_ratio*1.5))
    disson_fit = 1.0 if (target["want_dissonance"] and has_dim_like) or (not target["want_dissonance"] and not has_dim_like) else 0.5

    score = 0.25*mode_match + 0.25*tempo_fit + 0.25*density_fit + 0.15*disson_fit + 0.10*sync_fit
    return {
        "bpm": bpm_guess, "bars": bars, "note_density_per_bar": round(density,2),
        "mode_match": round(mode_match,2), "tempo_fit": round(tempo_fit,2),
        "density_fit": round(density_fit,2), "dissonance_fit": round(disson_fit,2),
        "syncopation_fit": round(sync_fit,2), "overall_structural_score": round(float(score),3),
        "build_params": build_params,
    }

def evaluate_with_audio_classifier(wav_path, target_emotion:str):
    try:
        out = audio_clf(wav_path)
        top = max(out, key=lambda x:x["score"])
        mapped = AUD_MAP.get(top["label"].lower()[:3], "OTHER")
        return {"predicted_emotion_audio": mapped,
                "predicted_confidence_audio": round(float(top["score"]),3),
                "target_emotion": target_emotion,
                "audio_match": 1.0 if mapped==target_emotion else 0.0}
    except Exception as e:
        print("[audio_eval error]", repr(e))
        return {"predicted_emotion_audio": None,"predicted_confidence_audio":0.0,
                "target_emotion": target_emotion,"audio_match":0.0}

def generate_and_evaluate_full(emotion:str, minutes:float=3.0, seed=None):
    mid_path, wav_path, meta = generate_full_track(emotion, minutes=minutes, seed=seed)
    struct = analyze_midi_struct(mid_path, emotion, meta)
    audio_eval = evaluate_with_audio_classifier(wav_path, emotion)
    final_score = round(0.75*struct["overall_structural_score"] + 0.25*audio_eval["audio_match"], 3)
    report = {"emotion": emotion, "structural": struct, "audio_eval": audio_eval, "final_score": final_score}
    return wav_path, report

# ----------------------------- Gradio UI ------------------------------
def analyze_and_make_music(text, image, audio_path, threshold, minutes, seed):
    label, score, emoji = analyze_all(text, image, audio_path, threshold)
    try:
        seed_int = None if (seed is None or str(seed).strip()=="") else int(seed)
    except:
        seed_int = None
    wav, rep = generate_and_evaluate_full(label, minutes=float(minutes), seed=seed_int)
    rep["detected_label"] = label
    rep["detected_confidence"] = score
    return label, score, emoji, wav, rep

with gr.Blocks(css="""
  #root {max-width:980px;margin:auto;padding:16px;}
  .out {font-size:1.1rem;text-align:center;}
""") as app:
    gr.Markdown("## 🎛️ تشخیص احساس (متن/تصویر/صوت) → 🎵 ساخت موسیقی چند‌دقیقه‌ای مطابق احساس + گزارش ارزیابی")
    with gr.Row():
        with gr.Column():
            txt = gr.Textbox(label="متن (فارسی/انگلیسی)", lines=3, placeholder="مثال: امروز خیلی خوشحالم!")
            img = gr.Image(label="تصویر چهره", type="pil")
            aud = gr.Audio(label="صوت (wav/mp3)", type="filepath")
        with gr.Column():
            thr  = gr.Slider(0,1,0.7,step=0.01, label="آستانه OTHER برای متن/صوت")
            mins = gr.Slider(1.0, 6.0, value=3.0, step=0.5, label="مدت موسیقی (دقیقه)")
            seed = gr.Textbox(value="", label="Seed (خالی = هر بار متفاوت)")
            go   = gr.Button("🔎 تحلیل و 🎵 ساخت موسیقی", variant="primary")

    with gr.Row():
        o1 = gr.Textbox(label="احساس",    interactive=False, elem_classes="out")
        o2 = gr.Textbox(label="اعتماد",   interactive=False, elem_classes="out")
        o3 = gr.Textbox(label="ایموجی",   interactive=False, elem_classes="out")

    audio_out = gr.Audio(label="🎧 موسیقی خروجی", type="filepath")
    rep_out   = gr.JSON(label="📊 گزارش ارزیابی (ساختاری + کلاسیفایر صوت روی WAV)")

    go.click(analyze_and_make_music, [txt, img, aud, thr, mins, seed], [o1, o2, o3, audio_out, rep_out])

app.launch(share=False)
# =====================================================================


Extracting templates from packages: 100%
Selecting previously unselected package libqt5core5a:amd64.
(Reading database ... 126284 files and directories currently installed.)
Preparing to unpack .../00-libqt5core5a_5.15.3+dfsg-2ubuntu0.2_amd64.deb ...
Unpacking libqt5core5a:amd64 (5.15.3+dfsg-2ubuntu0.2) ...
Selecting previously unselected package libevdev2:amd64.
Preparing to unpack .../01-libevdev2_1.12.1+dfsg-1_amd64.deb ...
Unpacking libevdev2:amd64 (1.12.1+dfsg-1) ...
Selecting previously unselected package libmtdev1:amd64.
Preparing to unpack .../02-libmtdev1_1.1.6-1build4_amd64.deb ...
Unpacking libmtdev1:amd64 (1.1.6-1build4) ...
Selecting previously unselected package libgudev-1.0-0:amd64.
Preparing to unpack .../03-libgudev-1.0-0_1%3a237-2build1_amd64.deb ...
Unpacking libgudev-1.0-0:amd64 (1:237-2build1) ...
Selecting previously unselected package libwacom-common.
Preparing to unpack .../04-libwacom-common_2.2.0-1_all.deb ...
Unpacking libwacom-common (2.2.0-1) ...
Selecting 

Device set to use cuda:0


config.json:   0%|          | 0.00/768 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

...

preprocessor_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

Device set to use cuda:0


model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

## مولتی ساز

In [ ]:
# =======================  یک سلولِ یکپارچه  ==========================
# نصب وابستگی‌ها
!apt-get -qq -y install fluidsynth
!pip install -q gradio transformers[sentencepiece] soundfile pillow pyFluidSynth==1.3.3 \
                pretty_midi music21 librosa midi2audio langdetect tensorflow opencv-python-headless

# SoundFont باکیفیت
!wget -q https://github.com/pepaslabs/fluidsynth-soundfonts/raw/master/FluidR3_GM/FluidR3_GM.sf2 -O /content/FluidR3_GM.sf2 \
  || wget -q https://github.com/urish/cyberbotics/raw/master/projects/robotbenchmark/resources/fluidr3/FluidR3_GM.sf2 -O /content/FluidR3_GM.sf2

# مدل FER (mini_XCEPTION)
!wget -q https://raw.githubusercontent.com/oarriaga/face_classification/master/trained_models/emotion_models/fer2013_mini_XCEPTION.102-0.66.hdf5 -O /content/emotion_model.h5

# ------------------ Import & Config (بستن GPU برای TF) ----------------
import os, io, time, math, random, json
os.environ["CUDA_VISIBLE_DEVICES"] = os.environ.get("CUDA_VISIBLE_DEVICES", "")  # برای Torch می‌گذاریم به حال خودش
# ولی TensorFlow را مجبور به CPU می‌کنیم تا cuDNN ارور نده
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES_TF"] = "-1"   # مارکر محلی (فقط برای وضوح)

import numpy as np
import gradio as gr
import soundfile as sf
from PIL import Image

import torch
from transformers import pipeline

import tensorflow as tf
try:
    tf.config.set_visible_devices([], 'GPU')
except Exception:
    pass

import cv2
from tensorflow.keras.models import load_model

import pretty_midi
from midi2audio import FluidSynth
from langdetect import detect

# ----------------------- Device برای Transformer ----------------------
DEVICE = 0 if torch.cuda.is_available() else -1
print("Device for Transformers:", "cuda:0" if DEVICE==0 else "cpu")
SF2_PATH = "/content/FluidR3_GM.sf2"

def _normalize_limit(x, peak=0.98):
    if x.ndim == 1:
        x = x[:, None]
    # حذف NaN/Inf
    x = np.nan_to_num(x, nan=0.0, posinf=0.0, neginf=0.0)
    # DC offset
    x = x - np.mean(x, axis=0, keepdims=True)
    # نرمالایز
    mx = np.max(np.abs(x)) + 1e-9
    x = x / mx
    # سافت-لیمیت (tanh) و مقیاس قله
    x = np.tanh(1.2 * x) * (peak / np.max(np.abs(np.tanh(1.2 * x)) + 1e-9))
    return x.squeeze()

def _render_wav_robust(pm, out_mid, out_wav, sf2_path=SF2_PATH, sr=22050):
    # تلاش ۱: pyfluidsynth
    try:
        audio = pm.fluidsynth(fs=sr, sf2_path=sf2_path)
        audio = _normalize_limit(audio)
        rms = np.sqrt(np.mean(audio**2))
        if (rms < 1e-4) or (not np.isfinite(rms)):
            raise ValueError("silent_or_invalid_audio")
        sf.write(out_wav, audio, sr)
        return out_wav, sr
    except Exception as e:
        print("[render] fallback to midi2audio:", e)
        # تلاش ۲: midi2audio (fl﻿uidsynth CLI)
        pm.write(out_mid)
        FluidSynth(sound_font=sf2_path, sample_rate=sr).midi_to_audio(out_mid, out_wav)
        y, sr2 = sf.read(out_wav, always_2d=False)
        y = _normalize_limit(y)
        sf.write(out_wav, y, sr2)
        return out_wav, sr2

# ----------------------------- Emoji & Maps ---------------------------
EMOJI = {"ANGRY":"😠","FEAR":"😨","HAPPY":"😃","SAD":"😢","SURPRISE":"😲","OTHER":"😐"}

# ------------------------- Pipelines متن/صوت --------------------------
pipe_fa = pipeline(
    "text-classification",
    model="/content/final_parsbert_emotion",
    tokenizer="/content/final_parsbert_emotion",
    device=DEVICE
)
pipe_en = pipeline(
    "text-classification",
    model="nateraw/bert-base-uncased-emotion",
    tokenizer="nateraw/bert-base-uncased-emotion",
    device=DEVICE,
    return_all_scores=False
)
def map_en(label):
    l = label.lower()
    if l in ["joy","love"]: return "HAPPY"
    if l == "sadness":      return "SAD"
    if l == "anger":        return "ANGRY"
    if l == "fear":         return "FEAR"
    if l == "surprise":     return "SURPRISE"
    return "OTHER"

audio_clf = pipeline("audio-classification", model="superb/wav2vec2-base-superb-er", device=DEVICE, top_k=None)
AUD_MAP = {"hap":"HAPPY","sad":"SAD","ang":"ANGRY","fea":"FEAR","sur":"SURPRISE","neu":"OTHER"}

# ------------------------- مدل تصویر (TF روی CPU) ---------------------
emotion_cnn = load_model("/content/emotion_model.h5", compile=False)
FER_LABELS = ['angry','disgust','fear','happy','sad','surprise','neutral']
IMG_MAP = {
    "angry":"ANGRY","disgust":"ANGRY","fear":"FEAR",
    "happy":"HAPPY","sad":"SAD","surprise":"SURPRISE","neutral":"OTHER"
}
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# -------------------------- تشخیص احساس‌ها ----------------------------
def analyze_text(text, threshold):
    lang = "fa"
    try:
        lang = detect(text)
    except:
        pass
    if lang == "fa":
        out = pipe_fa(text)[0]
        idx = int(out["label"].split("_")[-1])
        FA = ['ANGRY','FEAR','HAPPY','HATE','OTHER','SAD','SURPRISE']
        label = FA[idx] if FA[idx]!="HATE" else "ANGRY"
        score = float(out["score"])
    else:
        out = pipe_en(text)[0]
        label = map_en(out["label"])
        score = float(out["score"])
    if score < threshold:
        label = "OTHER"
    return label, round(score,3), EMOJI[label]

def analyze_image(img, threshold):
    # بدون threshold صفرکن؛ فقط خروجی واقعی مدل را بده
    try:
        gray = cv2.cvtColor(np.array(img.convert("RGB")), cv2.COLOR_RGB2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)
        if len(faces)==0:
            h,w = gray.shape
            sz = min(h,w)//2
            x = (w-sz)//2; y = (h-sz)//2
            roi = gray[y:y+sz, x:x+sz]
        else:
            x,y,w,h = faces[0]
            roi = gray[y:y+h, x:x+w]
        face = cv2.resize(roi, (64,64), interpolation=cv2.INTER_AREA)
        face = face.astype("float32")/255.0
        face = face.reshape(1,64,64,1)
        preds = emotion_cnn.predict(face, verbose=0)[0]
        idx   = int(np.argmax(preds))
        score = float(preds[idx])
        raw   = FER_LABELS[idx]
        label = IMG_MAP.get(raw, "OTHER")
        return label, round(score,3), EMOJI[label]
    except Exception as e:
        # لاگ کوچک برای دیباگ
        print("[analyze_image error]", repr(e))
        return "OTHER", 0.0, EMOJI["OTHER"]

def analyze_audio(filepath, threshold):
    try:
        out = audio_clf(filepath)
        if not out:
            return "OTHER", 0.0, EMOJI["OTHER"]
        top = max(out, key=lambda x: x["score"])
        raw = top["label"].lower()
        score = float(top["score"])
        label = AUD_MAP.get(raw[:3], "OTHER")
        if score < threshold:
            label = "OTHER"
        return label, round(score,3), EMOJI[label]
    except Exception as e:
        print("[analyze_audio error]", repr(e))
        return "OTHER", 0.0, EMOJI["OTHER"]

def analyze_all(text, image, audio_path, threshold):
    if text and str(text).strip():
        return analyze_text(text, threshold)
    if image is not None:
        return analyze_image(image, threshold)
    if audio_path:
        return analyze_audio(audio_path, threshold)
    return "OTHER", 0.0, EMOJI["OTHER"]

# --------------------- تنظیمات ساخت موسیقی (Rule-based) ----------------
# GM Programs (0-based)
GM = dict(
    piano=[0,1,4,5,6],                        # Piano family
    guitar_ac=[24,25], guitar_el=[26,27,28],  # Acoustic & Electric Guitars
    guitar_drive=[29,30],                     # Overdrive/Distortion
    bass=[32,33,34,39],                       # Acoustic/Electric/Synth Bass
    strings=[48,49,50],                       # Strings / SynthStrings
    brass=[60,61],                             # French Horn / Brass Section
    mallet=[11,12,13,14],                      # Vibraphone/Marimba/Xylo
    pad=[88,89,91,94],                         # Warm/Poly/Choir pads
    lead=[80,81,82,84]                         # Lead synths
)

EMO_PROFILE = {
    "HAPPY": {
        "mode":"major", "tempo_range":(118,140), "base_keys":[60,62,65,67],
        "progressions":[["I","V","vi","IV"],["I","vi","IV","V"],["I","IV","V","I"]],
        "melody_density_range":(0.6,0.9), "drums":True,  "comment":"شاد و پرانرژی",
        "roles_pool": ["pad","piano","guitar_ac","guitar_el","bass","strings","lead","mallet"]
    },
    "SAD": {
        "mode":"minor", "tempo_range":(60,82), "base_keys":[57,55,52],
        "progressions":[["i","VI","III","VII"],["i","iv","i","VII"]],
        "melody_density_range":(0.25,0.5), "drums":False, "comment":"کند و احساسی",
        "roles_pool": ["piano","strings","pad","mallet","bass"]
    },
    "ANGRY": {
        "mode":"minor", "tempo_range":(135,170), "base_keys":[50,48,53],
        "progressions":[["i","bVI","bVII","i"],["i","bII°","bVII","i"]],
        "melody_density_range":(0.7,0.95), "drums":True, "comment":"تهاجمی و کوبنده",
        "roles_pool": ["guitar_drive","guitar_el","bass","lead","pad"]
    },
    "FEAR": {
        "mode":"minor", "tempo_range":(52,72), "base_keys":[57,58,55],
        "progressions":[["i","bII°","i","bVI"],["i","iv","bII°","i"]],
        "melody_density_range":(0.2,0.45), "drums":False, "comment":"مرموز و تیره",
        "roles_pool": ["pad","strings","mallet","bass","lead"]
    },
    "SURPRISE": {
        "mode":"major", "tempo_range":(122,145), "base_keys":[65,67,69],
        "progressions":[["I","V","IV","♭VII"],["I","IV","V","♭VII"]],
        "melody_density_range":(0.6,0.95), "drums":True, "comment":"جهشی/سینکوپ",
        "roles_pool": ["piano","guitar_ac","lead","brass","mallet","bass","pad"]
    },
    "OTHER": {
        "mode":"major", "tempo_range":(90,110), "base_keys":[60,62],
        "progressions":[["I","ii","IV","V"],["I","IV","ii","V"]],
        "melody_density_range":(0.4,0.65), "drums":False, "comment":"خنثی/پس‌زمینه",
        "roles_pool": ["piano","pad","strings","mallet","bass"]
    },
}
MAJOR_ROMAN = {"I":(0,"maj"),"ii":(2,"min"),"iii":(4,"min"),"IV":(5,"maj"),"V":(7,"maj"),"vi":(9,"min"),"♭VII":(10,"maj")}
MINOR_ROMAN = {"i":(0,"min"),"iv":(5,"min"),"v":(7,"min"),"VI":(8,"maj"),"VII":(10,"maj"),"bVI":(8,"maj"),"bVII":(10,"maj"),"bII°":(1,"dim")}
QUAL2INTS = {"maj":[0,4,7], "min":[0,3,7], "dim":[0,3,6]}

def _scale(mode): return [0,2,4,5,7,9,11] if mode=="major" else [0,2,3,5,7,8,10]

def _choose(prof, rng):
    bpm = rng.randint(*prof["tempo_range"])
    key = rng.choice(prof["base_keys"])
    prog = rng.choice(prof["progressions"])
    density = rng.uniform(*prof["melody_density_range"])
    # هر بار مجموعه‌ای از نقش‌ها را تصادفی فعال می‌کنیم
    roles = prof["roles_pool"][:]
    rng.shuffle(roles)
    # حداقل 3 نقش، حداکثر 6 (به همراه هارمونی و ملودی تضمینی)
    roles = roles[:rng.randint(3, min(6, len(roles)))]
    return bpm, key, prog, density, roles

def _root_pitch(key, roman, ROM):
    off, qual = ROM[roman]
    return (key + off) % 128, qual

def _add_chord(inst, start, end, root, qual, rng, spread=0):
    # آکورد بسته + اختیاری اسپرد (اکتاو پایین/بالا)
    ints = QUAL2INTS[qual]
    for i in ints:
        p = int(root+i + spread)
        vel = int(np.clip(72 + rng.randint(-8,8), 40, 105))
        inst.notes.append(pretty_midi.Note(velocity=vel, pitch=p, start=start, end=end))

def _strum_guitar(inst, start, end, root, qual, rng, beat_dur, direction="down"):
    # شبیه‌سازی استرام: نت‌های آکورد با فاصله‌های خیلی کوتاه
    ints = QUAL2INTS[qual][:]
    if direction=="up": ints = list(reversed(ints))
    t = start
    for i in ints:
        p = int(root+i)
        d = min(0.25*beat_dur, end-start)  # کوتاه
        vel = int(np.clip(76 + rng.randint(-10,10), 45, 110))
        jit = rng.uniform(-0.01,0.01)
        inst.notes.append(pretty_midi.Note(velocity=vel, pitch=p, start=t+jit, end=min(t+jit+d, end)))
        t += 0.04 + rng.uniform(0.0, 0.02)

def _arp_pattern(inst, start, end, root, qual, rng, step):
    # آرپژ روی 8th/16th ها
    ints = QUAL2INTS[qual]
    seq = ints + ints[::-1]               # up & down
    t = start
    idx = rng.randint(0,len(seq)-1)
    while t < end:
        p = int(root + seq[idx % len(seq)])
        dur = step * (1.0 if rng.random()<0.7 else 0.5)
        vel = int(np.clip(74 + rng.randint(-12,12), 40, 110))
        jit = rng.uniform(-0.015,0.015)
        inst.notes.append(pretty_midi.Note(velocity=vel, pitch=p, start=t+jit, end=min(t+jit+dur, end)))
        t += step
        idx += 1

def _bass_pattern(inst, start, end, root, bpm, rng, style="eighths"):
    beat = 60.0/bpm
    if style=="long":
        p = max(0, root-12)
        inst.notes.append(pretty_midi.Note(velocity=78, pitch=p, start=start, end=end))
    elif style=="syncop":
        # رو الگوی off-beat
        for k in [0.5, 1.5, 2.5, 3.5]:
            t = start + k*beat + rng.uniform(-0.01,0.01)
            p = max(0, root-12 + rng.choice([0, -5, 7])//7*0)  # بیشتر روت
            inst.notes.append(pretty_midi.Note(velocity=85+rng.randint(-6,6), pitch=max(0,root-12), start=t, end=min(t+0.3, end)))
    else:  # eighths
        t = start
        while t < end:
            p = max(0, root-12)
            d = 0.45*beat
            inst.notes.append(pretty_midi.Note(velocity=82+rng.randint(-8,8), pitch=p, start=t, end=min(t+d, end)))
            t += 0.5*beat

def _lead_melody(inst, start, bars, bpm, mode, key, density, rng):
    scale = _scale(mode)
    step = 0.5*(60.0/bpm)       # 8th
    total_sub = int(bars*4*2)
    t = start
    last_pitch = key + 7 + rng.choice([-12,0,12])
    lo, hi = key-5, key+(16 if mode=="major" else 13)
    motif = [rng.choice(scale) for _ in range(rng.choice([3,4,5,6]))]
    for s in range(total_sub):
        if rng.random() < density:
            deg = motif[s % len(motif)] if rng.random()<0.75 else rng.choice(scale)
            base = key + deg + rng.choice([-12,0,12])
            pitch = int(np.clip(int(0.65*last_pitch + 0.35*base), lo, hi))
            dur = step if rng.random()<0.65 else step*2
            jit = rng.uniform(-0.02,0.02)
            vel = int(np.clip(80 + rng.randint(-14,14), 40, 115))
            inst.notes.append(pretty_midi.Note(velocity=vel, pitch=pitch, start=t+jit, end=t+jit+dur))
            last_pitch = pitch
        t += step

def _drum_bar(inst_drm, bar_start, beat_dur, rng, style="straight"):
    for b in range(4):
        t = bar_start + b*beat_dur
        # Kick/Snare
        inst_drm.notes.append(pretty_midi.Note(100 if b%2==0 else 92, pitch=36 if b%2==0 else 38, start=t, end=t+0.06))
        # Hats
        if style=="swing":
            inst_drm.notes.append(pretty_midi.Note(62, pitch=42, start=t, end=t+0.03))
            inst_drm.notes.append(pretty_midi.Note(62, pitch=42, start=t+beat_dur*2/3, end=t+beat_dur*2/3+0.03))
        else:
            inst_drm.notes.append(pretty_midi.Note(62, pitch=42, start=t, end=t+0.03))
            inst_drm.notes.append(pretty_midi.Note(62, pitch=42, start=t+beat_dur/2, end=t+beat_dur/2+0.03))

def _drum_fill(inst_drm, bar_start, beat_dur, rng):
    for i, pitch in enumerate([47,45,43,41]):  # toms
        t = bar_start + i*(beat_dur/2)
        inst_drm.notes.append(pretty_midi.Note(86+rng.randint(-6,6), pitch=pitch, start=t, end=t+0.12))

def _make_inst(program_choices, rng):
    return pretty_midi.Instrument(program=rng.choice(program_choices))

def _cc(inst, cc, value, time=0.0):
    inst.control_changes.append(pretty_midi.ControlChange(cc, int(value), time))

def _power_chord_riff(inst_L, inst_R, start, end, root, bpm, rng):
    """ریفی ساده بر پایه روت+۵ام+اکتاو؛ دابل‌تِرک L/R"""
    beat = 60.0/bpm
    t = start
    while t < end:
        dur = min(0.45*beat, end-t)
        velL = int(np.clip(96 + rng.randint(-8,8), 60, 127))
        velR = int(np.clip(94 + rng.randint(-8,8), 60, 127))
        for off in [0,7,12]:
            inst_L.notes.append(pretty_midi.Note(velocity=velL, pitch=int(root+off),
                                                 start=t+rng.uniform(-0.005,0.005),
                                                 end=min(t+dur, end)))
            inst_R.notes.append(pretty_midi.Note(velocity=velR, pitch=int(root+off),
                                                 start=t+rng.uniform(-0.005,0.005),
                                                 end=min(t+dur, end)))
        t += 0.5*beat  # 8th drive

def generate_full_track(emotion: str, minutes: float = 3.0, seed: int | None = None):
    assert emotion in EMO_PROFILE, f"unknown emotion {emotion}"
    rng = random.Random(seed if seed is not None else int(time.time()*1000) % 2_147_483_647)
    prof = EMO_PROFILE[emotion]
    bpm, key, progression, density, roles = _choose(prof, rng)
    ROM = MAJOR_ROMAN if prof["mode"]=="major" else MINOR_ROMAN
    sec_per_bar = 4*(60.0/bpm)
    total_bars = max(16, int((minutes*60)/sec_per_bar))
    bars_A = int(total_bars*0.4); bars_B = int(total_bars*0.4); bars_A2 = total_bars - (bars_A+bars_B)
    beat_dur = (60.0/bpm)

    pm = pretty_midi.PrettyMIDI()

    # Harmony bed
    if "pad" in roles:
        inst_har = _make_inst(GM["pad"]+GM["strings"], rng)
    elif "strings" in roles:
        inst_har = _make_inst(GM["strings"], rng)
    else:
        inst_har = _make_inst(GM["piano"], rng)
    _cc(inst_har, 7, 96)   # Volume
    _cc(inst_har, 10, 64)  # Pan
    pm.instruments.append(inst_har)

    # Optional roles with mix
    insts = {}
    def _add(name, progs, vol, pan):
        insts[name] = _make_inst(progs, rng)
        _cc(insts[name], 7, vol); _cc(insts[name], 10, pan)
        pm.instruments.append(insts[name])

    if "piano" in roles:        _add("piano", GM["piano"],        100, 64)
    if "guitar_ac" in roles:    _add("gtr_ac", GM["guitar_ac"],    96,  54)
    if "guitar_el" in roles:    _add("gtr_el", GM["guitar_el"],    96,  74)
    if "guitar_drive" in roles:
        _add("gtrL", GM["guitar_drive"], 110, 32)  # Left
        _add("gtrR", GM["guitar_drive"], 110, 96)  # Right
    if "bass" in roles:         _add("bass", GM["bass"],          110, 64)
    if "strings" in roles:      _add("str2", GM["strings"],         92, 80)
    if "brass" in roles:        _add("brs", GM["brass"],            96, 70)
    if "mallet" in roles:       _add("mlt", GM["mallet"],           92, 58)
    if "lead" in roles:         _add("lead", GM["lead"],           104, 64)

    inst_drm = pretty_midi.Instrument(program=0, is_drum=True) if prof["drums"] else None
    if inst_drm: pm.instruments.append(inst_drm)

    def render_section(start_bar, num_bars, key_this, prog_this, dens_mul=1.0, final_fill=False):
        for i in range(num_bars):
            roman = prog_this[i % len(prog_this)]
            root, qual = _root_pitch(key_this, roman, ROM)
            start = (start_bar+i)*4*beat_dur
            end   = (start_bar+i+1)*4*beat_dur

            # Harmony bed
            _add_chord(inst_har, start, end, root, qual, rng, spread=rng.choice([0,12,-12,0]))

            # FEAR: ارگان کلیسا لانگ‌نوت (در صورت حضور lead → نقش lead با ارگان)
            if emotion=="FEAR":
                org = pretty_midi.Instrument(program=19)  # Church Organ (0-based)
                _cc(org,7,100); _cc(org,10,64)
                pm.instruments.append(org)
                insts["fear_org"] = org
                org_note = pretty_midi.Note(velocity=84, pitch=int(root), start=start, end=end)
                org.notes.append(org_note)

            # Guitar strum / Power-chords
            has_drive = ("gtrL" in insts and "gtrR" in insts)
            if has_drive and emotion=="ANGRY":
                _power_chord_riff(insts["gtrL"], insts["gtrR"], start, end, root, bpm, rng)
            else:
                gtr = insts.get("gtr_el") or insts.get("gtr_ac")
                if gtr: _strum_guitar(gtr, start, end, root, qual, rng, beat_dur, direction=rng.choice(["down","up"]))

            # Piano arpeggio
            if "piano" in insts and rng.random()<0.85:
                _arp_pattern(insts["piano"], start, end, root, qual, rng, step=0.5*beat_dur)

            # Brass/Mallet hits
            if "brs" in insts and rng.random()<0.4 and emotion in ["HAPPY","SURPRISE"]:
                _add_chord(insts["brs"], start, min(start+beat_dur, end), root+12, qual, rng)
            if "mlt" in insts and rng.random()<0.5:
                _arp_pattern(insts["mlt"], start, min(start+2*beat_dur, end), root+12, qual, rng, step=0.25*beat_dur)

            # Bass
            if "bass" in insts:
                style = "eighths"
                if emotion in ["SAD","FEAR"]: style = "long"
                if emotion in ["HAPPY","SURPRISE"]: style = rng.choice(["eighths","syncop"])
                if emotion=="ANGRY": style = "eighths"
                _bass_pattern(insts["bass"], start, end, root, bpm, rng, style=style)

            # Drums
            if inst_drm:
                groove = "swing" if (emotion=="SURPRISE" and rng.random()<0.4) else "straight"
                _drum_bar(inst_drm, start, beat_dur, rng, style=groove)

        # Lead
        lead_inst = insts.get("lead") or insts.get("piano") or inst_har
        _lead_melody(lead_inst, start_bar*4*beat_dur, num_bars, bpm, EMO_PROFILE[emotion]["mode"], key_this, density*dens_mul, rng)

        if final_fill and inst_drm:
            _drum_fill(inst_drm, (start_bar+num_bars-1)*4*beat_dur, beat_dur, rng)

    # ساختار A–B–A′
    render_section(0, bars_A, key, progression, dens_mul=1.0, final_fill=True)
    mod_key = key + rng.choice([-2, 2, 0])
    new_prog = rng.choice(prof["progressions"])
    render_section(bars_A, bars_B, mod_key, new_prog, dens_mul=random.uniform(0.9,1.25), final_fill=True)
    render_section(bars_A+bars_B, bars_A2, key, progression, dens_mul=random.uniform(0.85,1.15), final_fill=False)

    stamp = int(time.time())
    out_mid = f"{emotion.lower()}_{minutes:.1f}min_{stamp}.mid"
    out_wav = f"{emotion.lower()}_{minutes:.1f}min_{stamp}.wav"

    #  رندر محکم و نرمال‌سازی‌شده
    wav_path, sr = _render_wav_robust(pm, out_mid, out_wav, sf2_path=SF2_PATH, sr=22050)

    meta = {
        "emotion": emotion, "minutes": minutes, "bpm": bpm, "mode": prof["mode"], "key_midi": key,
        "progression_A": progression, "progression_B": new_prog, "drums": bool(inst_drm),
        "comment": prof["comment"], "bars_total": total_bars, "roles_used": roles, "sr": sr
    }
    return out_mid, wav_path, meta


# ---------------------------- ارزیابی موسیقی --------------------------
TARGETS = {
    "HAPPY":    dict(mode="major", tempo=(115,140), density=(6,10), want_dissonance=False, want_syncopation=True),
    "SAD":      dict(mode="minor", tempo=(60,85),   density=(2,5),  want_dissonance=False, want_syncopation=False),
    "ANGRY":    dict(mode="minor", tempo=(130,170), density=(8,14), want_dissonance=True,  want_syncopation=True),
    "FEAR":     dict(mode="minor", tempo=(50,75),   density=(1,4),  want_dissonance=True,  want_syncopation=False),
    "SURPRISE": dict(mode="major", tempo=(115,145), density=(6,12), want_dissonance=False, want_syncopation=True),
    "OTHER":    dict(mode="major", tempo=(85,110),  density=(4,8),  want_dissonance=False, want_syncopation=False),
}

def analyze_midi_struct(mid_path, emotion, build_params):
    pm = pretty_midi.PrettyMIDI(mid_path)
    non_drums = [ins for ins in pm.instruments if not ins.is_drum]
    note_count = sum(len(ins.notes) for ins in non_drums)
    bars = build_params["bars_total"]
    density = note_count / max(1,bars)
    bpm_guess = build_params["bpm"]

    # سینکوپ: نسبت نُت‌هایی که روی off-beat می‌افتند
    step = 60.0/bpm_guess/2
    sync_onsets, total_notes = 0, 0
    for ins in non_drums:
        for n in ins.notes:
            total_notes += 1
            frac = (n.start/step) % 2
            if 0.4 < frac < 1.6: sync_onsets += 1
    sync_ratio = (sync_onsets / max(1,total_notes)) if total_notes else 0.0

    # دیسونانس تقریبی: وجود نت‌های خیلی کوتاه (تزئینات تند)
    has_dim_like = any((n.end-n.start) < (60.0/bpm_guess)/4 for ins in non_drums for n in ins.notes)

    target = TARGETS[emotion]
    mode_match = 1.0 if (build_params["mode"] == target["mode"]) else 0.0
    tempo_fit  = 1.0 if (target["tempo"][0] <= bpm_guess <= target["tempo"][1]) else 0.0
    d_lo, d_hi = target["density"]
    if density<d_lo: density_fit = max(0.0, 1 - (d_lo-density)/max(1e-6,d_lo))
    elif density>d_hi: density_fit = max(0.0, 1 - (density-d_hi)/max(1e-6,d_hi))
    else: density_fit = 1.0
    sync_fit   = min(1.0, sync_ratio*1.5) if target["want_syncopation"] else (1.0 - min(1.0, sync_ratio*1.5))
    disson_fit = 1.0 if (target["want_dissonance"] and has_dim_like) or (not target["want_dissonance"] and not has_dim_like) else 0.5

    score = 0.25*mode_match + 0.25*tempo_fit + 0.25*density_fit + 0.15*disson_fit + 0.10*sync_fit
    return {
        "bpm": bpm_guess, "bars": bars, "note_density_per_bar": round(density,2),
        "mode_match": round(mode_match,2), "tempo_fit": round(tempo_fit,2),
        "density_fit": round(density_fit,2), "dissonance_fit": round(disson_fit,2),
        "syncopation_fit": round(sync_fit,2), "overall_structural_score": round(float(score),3),
        "build_params": build_params,
    }


def evaluate_with_audio_classifier(wav_path, target_emotion:str):
    try:
        out = audio_clf(wav_path)
        top = max(out, key=lambda x:x["score"])
        mapped = AUD_MAP.get(top["label"].lower()[:3], "OTHER")
        return {"predicted_emotion_audio": mapped,
                "predicted_confidence_audio": round(float(top["score"]),3),
                "target_emotion": target_emotion,
                "audio_match": 1.0 if mapped==target_emotion else 0.0}
    except Exception as e:
        print("[audio_eval error]", repr(e))
        return {"predicted_emotion_audio": None,"predicted_confidence_audio":0.0,
                "target_emotion": target_emotion,"audio_match":0.0}

def generate_and_evaluate_full(emotion:str, minutes:float=3.0, seed=None):
    mid_path, wav_path, meta = generate_full_track(emotion, minutes=minutes, seed=seed)
    struct = analyze_midi_struct(mid_path, emotion, meta)
    audio_eval = evaluate_with_audio_classifier(wav_path, emotion)
    final_score = round(0.75*struct["overall_structural_score"] + 0.25*audio_eval["audio_match"], 3)
    report = {"emotion": emotion, "structural": struct, "audio_eval": audio_eval, "final_score": final_score}
    return wav_path, report

# ----------------------------- Gradio UI ------------------------------
def analyze_and_make_music(text, image, audio_path, threshold, minutes, seed):
    label, score, emoji = analyze_all(text, image, audio_path, threshold)
    try:
        seed_int = None if (seed is None or str(seed).strip()=="") else int(seed)
    except:
        seed_int = None
    wav_path, rep = generate_and_evaluate_full(label, minutes=float(minutes), seed=seed_int)
    # ← rep از همون قبل
    # سیگنال را برای پخش داخل Gradio بخوان:
    y, sr = sf.read(wav_path, always_2d=False)
    if y.ndim > 1:  # استریو → مونو ملایم برای سازگاری مرورگر
        y = np.mean(y, axis=1)
    y = _normalize_limit(y)  # اطمینان از عدم کلیپینگ/نویز
    rep["detected_label"] = label
    rep["detected_confidence"] = score
    return label, score, emoji, (sr, y.astype(np.float32)), rep


with gr.Blocks(css="""
  #root {max-width:980px;margin:auto;padding:16px;}
  .out {font-size:1.1rem;text-align:center;}
""") as app:
    gr.Markdown("## 🎛️ تشخیص احساس (متن/تصویر/صوت) → 🎵 ساخت موسیقی چند‌دقیقه‌ای مطابق احساس + گزارش ارزیابی")
    with gr.Row():
        with gr.Column():
            txt = gr.Textbox(label="متن (فارسی/انگلیسی)", lines=3, placeholder="مثال: امروز خیلی خوشحالم!")
            img = gr.Image(label="تصویر چهره", type="pil")
            aud = gr.Audio(label="صوت (wav/mp3)", type="filepath")
        with gr.Column():
            thr  = gr.Slider(0,1,0.7,step=0.01, label="آستانه OTHER برای متن/صوت")
            mins = gr.Slider(1.0, 6.0, value=3.0, step=0.5, label="مدت موسیقی (دقیقه)")
            seed = gr.Textbox(value="", label="Seed (خالی = هر بار متفاوت)")
            go   = gr.Button("🔎 تحلیل و 🎵 ساخت موسیقی", variant="primary")

    with gr.Row():
        o1 = gr.Textbox(label="احساس",    interactive=False, elem_classes="out")
        o2 = gr.Textbox(label="اعتماد",   interactive=False, elem_classes="out")
        o3 = gr.Textbox(label="ایموجی",   interactive=False, elem_classes="out")

    # audio_out = gr.Audio(label="🎧 موسیقی خروجی", type="filepath")
    audio_out = gr.Audio(label="🎧 موسیقی خروجی", type="numpy")
    rep_out   = gr.JSON(label="📊 گزارش ارزیابی (ساختاری + کلاسیفایر صوت روی WAV)")

    go.click(analyze_and_make_music, [txt, img, aud, thr, mins, seed], [o1, o2, o3, audio_out, rep_out])

app.launch(share=False)
# =====================================================================


Extracting templates from packages: 100%
Selecting previously unselected package libqt5core5a:amd64.
(Reading database ... 126380 files and directories currently installed.)
Preparing to unpack .../00-libqt5core5a_5.15.3+dfsg-2ubuntu0.2_amd64.deb ...
Unpacking libqt5core5a:amd64 (5.15.3+dfsg-2ubuntu0.2) ...
Selecting previously unselected package libevdev2:amd64.
Preparing to unpack .../01-libevdev2_1.12.1+dfsg-1_amd64.deb ...
Unpacking libevdev2:amd64 (1.12.1+dfsg-1) ...
Selecting previously unselected package libmtdev1:amd64.
Preparing to unpack .../02-libmtdev1_1.1.6-1build4_amd64.deb ...
Unpacking libmtdev1:amd64 (1.1.6-1build4) ...
Selecting previously unselected package libgudev-1.0-0:amd64.
Preparing to unpack .../03-libgudev-1.0-0_1%3a237-2build1_amd64.deb ...
Unpacking libgudev-1.0-0:amd64 (1:237-2build1) ...
Selecting previously unselected package libwacom-common.
Preparing to unpack .../04-libwacom-common_2.2.0-1_all.deb ...
Unpacking libwacom-common (2.2.0-1) ...
Selecting 

Device set to use cuda:0


config.json:   0%|          | 0.00/768 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

...

preprocessor_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

Device set to use cuda:0


model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

In [7]:
# =======================  یک سلولِ یکپارچه  ==========================
# نصب وابستگی‌ها
!apt-get -qq -y install fluidsynth
!pip install -q gradio transformers[sentencepiece] soundfile pillow pyFluidSynth==1.3.3 \
                pretty_midi music21 librosa midi2audio langdetect tensorflow opencv-python-headless

# SoundFont پیش‌فرض (General MIDI با کیفیت خوب)
!wget -q https://github.com/pepaslabs/fluidsynth-soundfonts/raw/master/FluidR3_GM/FluidR3_GM.sf2 -O /content/FluidR3_GM.sf2 \
  || wget -q https://github.com/urish/cyberbotics/raw/master/projects/robotbenchmark/resources/fluidr3/FluidR3_GM.sf2 -O /content/FluidR3_GM.sf2

# مدل FER (mini_XCEPTION) برای تصویر
!wget -q https://raw.githubusercontent.com/oarriaga/face_classification/master/trained_models/emotion_models/fer2013_mini_XCEPTION.102-0.66.hdf5 -O /content/emotion_model.h5

# ------------------ Import & Config (TF → CPU) ----------------
import os, time, random, json
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"

import tensorflow as tf
try:
    tf.config.set_visible_devices([], 'GPU')
except Exception:
    pass

import numpy as np
import gradio as gr
import soundfile as sf
from PIL import Image

import torch
from transformers import pipeline
import cv2
from tensorflow.keras.models import load_model
import pretty_midi
from midi2audio import FluidSynth
from langdetect import detect

# ----------------------- Device برای Transformers ----------------------
DEVICE = 0 if torch.cuda.is_available() else -1
print("Device for Transformers:", "cuda:0" if DEVICE==0 else "cpu")
SF2_PATH_DEFAULT = "/content/FluidR3_GM.sf2"

# ---------- Utility: نرمال‌سازی/لیمیتِ خروجی برای پخش پایدار ----------
def _normalize_limit(x, peak=0.98):
    x = np.array(x)
    if x.ndim == 1:
        x = x[:, None]
    x = np.nan_to_num(x, nan=0.0, posinf=0.0, neginf=0.0)
    x = x - np.mean(x, axis=0, keepdims=True)
    mx = np.max(np.abs(x)) + 1e-9
    x = x / mx
    y = np.tanh(1.2 * x)
    y = y * (peak / (np.max(np.abs(y)) + 1e-9))
    return y.squeeze()

def _render_wav_robust(pm, out_mid, out_wav, sf2_path=None, sr=22050):
    """همیشه اول MIDI را می‌نویسیم تا ارزیابی ساختاری خراب نشود."""
    sf2_path = sf2_path or SF2_PATH_DEFAULT
    pm.write(out_mid)  # ✅ مهم: همیشه بنویس
    # تلاش ۱: pyfluidsynth
    try:
        audio = pm.fluidsynth(fs=sr, sf2_path=sf2_path)
        audio = _normalize_limit(audio)
        rms = np.sqrt(np.mean(audio**2))
        if (rms < 1e-4) or (not np.isfinite(rms)):
            raise ValueError("silent_or_invalid_audio")
        sf.write(out_wav, audio, sr)
        return out_wav, sr
    except Exception as e:
        print("[render] fallback to midi2audio:", e)
        FluidSynth(sound_font=sf2_path, sample_rate=sr).midi_to_audio(out_mid, out_wav)
        y, sr2 = sf.read(out_wav, always_2d=False)
        y = _normalize_limit(y)
        sf.write(out_wav, y, sr2)
        return out_wav, sr2

# ----------------------------- Emoji & Maps ---------------------------
EMOJI = {"ANGRY":"😠","FEAR":"😨","HAPPY":"😃","SAD":"😢","SURPRISE":"😲","OTHER":"😐"}

# ------------------------- Pipelines متن/صوت --------------------------
# پارس‌برت اختیاری—اگر نبود، انگلیسی کار می‌کند
try:
    pipe_fa = pipeline(
        "text-classification",
        model="/content/final_parsbert_emotion",
        tokenizer="/content/final_parsbert_emotion",
        device=DEVICE
    )
except Exception as e:
    pipe_fa = None
    print("[warn] Persian text model not found -> using English when needed")

pipe_en = pipeline(
    "text-classification",
    model="nateraw/bert-base-uncased-emotion",
    tokenizer="nateraw/bert-base-uncased-emotion",
    device=DEVICE,
    return_all_scores=False
)

def map_en(label):
    l = label.lower()
    if l in ["joy","love"]: return "HAPPY"
    if l == "sadness":      return "SAD"
    if l == "anger":        return "ANGRY"
    if l == "fear":         return "FEAR"
    if l == "surprise":     return "SURPRISE"
    return "OTHER"

audio_clf = pipeline("audio-classification", model="superb/wav2vec2-base-superb-er", device=DEVICE, top_k=None)
AUD_MAP = {"hap":"HAPPY","sad":"SAD","ang":"ANGRY","fea":"FEAR","sur":"SURPRISE","neu":"OTHER"}

# ------------------------- مدل تصویر (TF روی CPU) ---------------------
emotion_cnn = load_model("/content/emotion_model.h5", compile=False)
FER_LABELS = ['angry','disgust','fear','happy','sad','surprise','neutral']
IMG_MAP = {
    "angry":"ANGRY","disgust":"ANGRY","fear":"FEAR",
    "happy":"HAPPY","sad":"SAD","surprise":"SURPRISE","neutral":"OTHER"
}
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# -------------------------- تشخیص احساس‌ها ----------------------------
def analyze_text(text, threshold):
    if not text or not str(text).strip():
        return "OTHER", 0.0, EMOJI["OTHER"]
    try:
        lang = detect(text)
    except:
        lang = "fa"
    if lang == "fa" and pipe_fa is not None:
        out = pipe_fa(text)[0]
        idx = int(out["label"].split("_")[-1])
        FA = ['ANGRY','FEAR','HAPPY','HATE','OTHER','SAD','SURPRISE']
        label = FA[idx] if FA[idx]!="HATE" else "ANGRY"
        score = float(out["score"])
    else:
        out = pipe_en(text)[0]
        label = map_en(out["label"])
        score = float(out["score"])
    if score < threshold:
        label = "OTHER"
    return label, round(score,3), EMOJI[label]

def analyze_image(img, threshold):
    try:
        gray = cv2.cvtColor(np.array(img.convert("RGB")), cv2.COLOR_RGB2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)
        if len(faces)==0:
            h,w = gray.shape
            sz = min(h,w)//2
            x = (w-sz)//2; y = (h-sz)//2
            roi = gray[y:y+sz, x:x+sz]
        else:
            x,y,w,h = faces[0]
            roi = gray[y:y+h, x:x+w]
        face = cv2.resize(roi, (64,64), interpolation=cv2.INTER_AREA)
        face = face.astype("float32")/255.0
        face = face.reshape(1,64,64,1)
        preds = emotion_cnn.predict(face, verbose=0)[0]
        idx   = int(np.argmax(preds))
        score = float(preds[idx])
        raw   = FER_LABELS[idx]
        label = IMG_MAP.get(raw, "OTHER")
        return label, round(score,3), EMOJI[label]
    except Exception as e:
        print("[analyze_image error]", repr(e))
        return "OTHER", 0.0, EMOJI["OTHER"]

def analyze_audio(filepath, threshold):
    try:
        out = audio_clf(filepath)
        if not out:
            return "OTHER", 0.0, EMOJI["OTHER"]
        top = max(out, key=lambda x: x["score"])
        raw = top["label"].lower()
        score = float(top["score"])
        label = AUD_MAP.get(raw[:3], "OTHER")
        if score < threshold:
            label = "OTHER"
        return label, round(score,3), EMOJI[label]
    except Exception as e:
        print("[analyze_audio error]", repr(e))
        return "OTHER", 0.0, EMOJI["OTHER"]

def analyze_all(text, image, audio_path, threshold):
    if text and str(text).strip():
        return analyze_text(text, threshold)
    if image is not None:
        return analyze_image(image, threshold)
    if audio_path:
        return analyze_audio(audio_path, threshold)
    return "OTHER", 0.0, EMOJI["OTHER"]

# --------------------- تنظیمات ساخت موسیقی ----------------
GM = dict(
    piano=[0,1,4,5,6],
    guitar_ac=[24,25], guitar_el=[26,27,28],
    guitar_drive=[29,30],
    bass=[32,33,34,39],
    strings=[48,49,50],
    brass=[60,61],
    mallet=[11,12,13,14],
    pad=[88,89,91,94],
    lead=[80,81,82,84]
)

# سازهای ایرانی (تقریب با GM)
PERSIAN_GM = dict(
    santur=[15],               # Dulcimer ≈ Santur
    tar=[24,104,107],          # NylonGtr / Sitar / Koto
    setar=[24,104],
    oud=[24,25,104],
    kamancheh=[40],            # Violin
    ney=[77,75],               # Shakuhachi / Pan Flute
    qanun=[46,15,107],         # Harp / Dulcimer / Koto
)

# تنها سازهای ایرانی در حالت Persian
PERSIAN_ROLES = {
    "HAPPY":    ["santur","qanun","oud","ney","kamancheh","tar","setar"],
    "SAD":      ["kamancheh","ney","santur","qanun","setar","tar"],
    "ANGRY":    ["santur","tar","kamancheh","ney","oud"],   # خشم ایرانی بدون گیتار غربی
    "FEAR":     ["ney","kamancheh","santur","qanun"],
    "SURPRISE": ["santur","qanun","oud","ney","kamancheh"],
    "OTHER":    ["santur","qanun","ney","kamancheh","setar"]
}

EMO_PROFILE = {
    "HAPPY":    {"mode":"major","tempo_range":(120,140),"base_keys":[60,62,65,67],
                 "progressions":[["I","V","vi","IV"],["I","vi","IV","V"],["I","IV","V","I"]],
                 "melody_density_range":(0.65,0.9),"drums":True,"comment":"شاد و پرانرژی"},
    "SAD":      {"mode":"minor","tempo_range":(45,65),"base_keys":[57,55,52],
                 "progressions":[["i","VI","III","VII"],["i","iv","i","VII"]],
                 "melody_density_range":(0.15,0.35),"drums":False,"comment":"خیلی آرام و افسرده"},
    "ANGRY":    {"mode":"minor","tempo_range":(140,175),"base_keys":[50,48,53],
                 "progressions":[["i","bVI","bVII","i"],["i","bII°","bVII","i"]],
                 "melody_density_range":(0.75,0.95),"drums":True,"comment":"راک/متال تهاجمی"},
    "FEAR":     {"mode":"minor","tempo_range":(50,72),"base_keys":[57,58,55],
                 "progressions":[["i","bII°","i","bVI"],["i","iv","bII°","i"]],
                 "melody_density_range":(0.15,0.35),"drums":False,"comment":"دلهره‌آور"},
    "SURPRISE": {"mode":"major","tempo_range":(122,145),"base_keys":[65,67,69],
                 "progressions":[["I","V","IV","♭VII"],["I","IV","V","♭VII"]],
                 "melody_density_range":(0.6,0.95),"drums":True,"comment":"جهشی/سینکوپ"},
    "OTHER":    {"mode":"major","tempo_range":(90,110),"base_keys":[60,62],
                 "progressions":[["I","ii","IV","V"],["I","IV","ii","V"]],
                 "melody_density_range":(0.4,0.65),"drums":False,"comment":"خنثی/پس‌زمینه"},
}
MAJOR_ROMAN = {"I":(0,"maj"),"ii":(2,"min"),"iii":(4,"min"),"IV":(5,"maj"),"V":(7,"maj"),"vi":(9,"min"),"♭VII":(10,"maj")}
MINOR_ROMAN = {"i":(0,"min"),"iv":(5,"min"),"v":(7,"min"),"VI":(8,"maj"),"VII":(10,"maj"),"bVI":(8,"maj"),"bVII":(10,"maj"),"bII°":(1,"dim")}
QUAL2INTS = {"maj":[0,4,7], "min":[0,3,7], "dim":[0,3,6]}

def _scale_for(mode:str, emotion:str, style:str):
    if style != "persian":
        return [0,2,4,5,7,9,11] if mode=="major" else [0,2,3,5,7,8,10]
    # تقریبی از دستگاه‌ها (بدون ربع‌پرده)
    if emotion in ["HAPPY","SURPRISE"]: return [0,1,4,5,7,8,11]   # Hijaz / Phrygian-dominant
    if emotion in ["SAD"]:              return [0,1,3,5,7,8,10]   # Phrygian (حزن)
    if emotion in ["FEAR"]:             return [0,1,3,5,6,8,10]   # Saba-like (دلهره)
    return [0,2,3,5,7,8,10]             # Nahawand-like

def _add_chord(inst, start, end, root, qual, rng, spread=0):
    for i in QUAL2INTS[qual]:
        p = int(root+i + spread)
        vel = int(np.clip(72 + rng.randint(-8,8), 40, 105))
        inst.notes.append(pretty_midi.Note(velocity=vel, pitch=p, start=start, end=end))

def _add_persian_bed(inst, start, end, root, rng, add_fifth=True, octave_spread=True):
    base = int(root)
    p_list = [base]
    if add_fifth: p_list.append(base+7)
    if octave_spread and (random.random()<0.6):
        p_list += [base-12, base+12]
    vel = int(np.clip(78 + rng.randint(-6,6), 50, 105))
    for p in p_list:
        p = max(0, min(127, p))
        inst.notes.append(pretty_midi.Note(velocity=vel, pitch=p, start=start, end=end))

def _arp_pattern(inst, start, end, root, qual, rng, step):
    ints = QUAL2INTS[qual]; seq = ints + ints[::-1]
    t = start; idx = rng.randint(0,len(seq)-1)
    while t < end:
        p = int(root + seq[idx % len(seq)])
        dur = step * (1.0 if rng.random()<0.7 else 0.5)
        vel = int(np.clip(74 + rng.randint(-12,12), 40, 110))
        jit = rng.uniform(-0.015,0.015)
        inst.notes.append(pretty_midi.Note(velocity=vel, pitch=p, start=t+jit, end=min(t+jit+dur, end)))
        t += step; idx += 1

def _scale_run(inst, start, end, key, scale, rng, step):
    t = start; idx = rng.randint(0,len(scale)-1); direction = rng.choice([1,-1])
    last = key + scale[idx]
    while t < end:
        deg = scale[idx % len(scale)]
        base = key + deg + rng.choice([-12,0,12])
        pitch = int(np.clip(int(0.65*last + 0.35*base), key-7, key+19))
        dur = step if rng.random()<0.8 else step*2
        vel = int(np.clip(76 + rng.randint(-10,10), 45, 110))
        jit = rng.uniform(-0.015,0.015)
        inst.notes.append(pretty_midi.Note(velocity=vel, pitch=pitch, start=t+jit, end=min(t+jit+dur, end)))
        last = pitch; t += step; idx += direction
        if rng.random()<0.12: direction *= -1

def _bass_pattern(inst, start, end, root, bpm, rng, style="eighths"):
    beat = 60.0/bpm
    if style=="long":
        p = max(0, root-12)
        inst.notes.append(pretty_midi.Note(velocity=74, pitch=p, start=start, end=end))
    elif style=="syncop":
        for k in [0.5, 1.5, 2.5, 3.5]:
            t = start + k*beat + rng.uniform(-0.01,0.01)
            inst.notes.append(pretty_midi.Note(velocity=85+rng.randint(-6,6), pitch=max(0,root-12), start=t, end=min(t+0.30, end)))
    else:
        t = start
        while t < end:
            p = max(0, root-12)
            d = 0.45*beat
            inst.notes.append(pretty_midi.Note(velocity=82+rng.randint(-8,8), pitch=p, start=t, end=min(t+d, end)))
            t += 0.5*beat

def _lead_melody(inst, start, bars, bpm, mode, key, density, rng, scale_override=None):
    scale = scale_override if scale_override is not None else ([0,2,4,5,7,9,11] if mode=="major" else [0,2,3,5,7,8,10])
    step = 0.5*(60.0/bpm)       # 8th
    total_sub = int(bars*4*2)
    t = start; last_pitch = key + 7 + rng.choice([-12,0,12])
    lo, hi = key-7, key+19
    motif = [rng.choice(scale) for _ in range(rng.choice([3,4,5,6]))]
    for s in range(total_sub):
        if rng.random() < density:
            deg = motif[s % len(motif)] if rng.random()<0.75 else rng.choice(scale)
            base = key + deg + rng.choice([-12,0,12])
            pitch = int(np.clip(int(0.65*last_pitch + 0.35*base), lo, hi))
            dur = step if rng.random()<0.65 else step*2
            jit = rng.uniform(-0.02,0.02)
            vel = int(np.clip(80 + rng.randint(-14,14), 40, 115))
            inst.notes.append(pretty_midi.Note(velocity=vel, pitch=pitch, start=t+jit, end=t+jit+dur))
            last_pitch = pitch
        t += step

def _drum_bar(inst_drm, bar_start, beat_dur, rng, style="straight"):
    for b in range(4):
        t = bar_start + b*beat_dur
        inst_drm.notes.append(pretty_midi.Note(100 if b%2==0 else 92, pitch=36 if b%2==0 else 38, start=t, end=t+0.06))
        if style=="swing":
            inst_drm.notes.append(pretty_midi.Note(62, pitch=42, start=t, end=t+0.03))
            inst_drm.notes.append(pretty_midi.Note(62, pitch=42, start=t+beat_dur*2/3, end=t+beat_dur*2/3+0.03))
        else:
            inst_drm.notes.append(pretty_midi.Note(62, pitch=42, start=t, end=t+0.03))
            inst_drm.notes.append(pretty_midi.Note(62, pitch=42, start=t+beat_dur/2, end=t+beat_dur/2+0.03))

def _drum_fill(inst_drm, bar_start, beat_dur, rng):
    for i, pitch in enumerate([47,45,43,41]):
        t = bar_start + i*(beat_dur/2)
        inst_drm.notes.append(pretty_midi.Note(86+rng.randint(-6,6), pitch=pitch, start=t, end=t+0.12))

def _cc(inst, cc, value, time=0.0):
    inst.control_changes.append(pretty_midi.ControlChange(cc, int(value), time))

def _power_chord_riff(inst_L, inst_R, start, end, root, bpm, rng):
    beat = 60.0/bpm; t = start
    while t < end:
        dur = min(0.45*beat, end-t)
        velL = int(np.clip(104 + rng.randint(-8,8), 60, 127))
        velR = int(np.clip(102 + rng.randint(-8,8), 60, 127))
        for off in [0,7,12]:
            inst_L.notes.append(pretty_midi.Note(velocity=velL, pitch=int(root+off), start=t+rng.uniform(-0.005,0.005), end=min(t+dur, end)))
            inst_R.notes.append(pretty_midi.Note(velocity=velR, pitch=int(root+off), start=t+rng.uniform(-0.005,0.005), end=min(t+dur, end)))
        t += 0.5*beat

def generate_full_track(emotion: str, minutes: float = 3.0, seed: int | None = None, style: str = "global", sf2_path: str | None = None):
    assert emotion in EMO_PROFILE, f"unknown emotion {emotion}"
    rng = random.Random(seed if seed is not None else int(time.time()*1000) % 2_147_483_647)
    prof = EMO_PROFILE[emotion]
    bpm = rng.randint(*prof["tempo_range"])
    key = rng.choice(prof["base_keys"])
    progression = rng.choice(prof["progressions"])
    density = rng.uniform(*prof["melody_density_range"])
    is_persian = (style == "persian")
    sec_per_bar = 4*(60.0/bpm)
    total_bars = max(16, int((minutes*60)/sec_per_bar))
    bars_A = int(total_bars*0.4); bars_B = int(total_bars*0.4); bars_A2 = total_bars - (bars_A+bars_B)
    beat_dur = (60.0/bpm)
    ROM = MAJOR_ROMAN if prof["mode"]=="major" else MINOR_ROMAN
    scale_local = _scale_for(prof["mode"], emotion, style)

    pm = pretty_midi.PrettyMIDI()
    insts = {}

    if is_persian:
        roles_src = PERSIAN_ROLES[emotion][:]
        rng.shuffle(roles_src)
        roles = roles_src[:rng.randint(3, min(6, len(roles_src)))]
        bed_prog = rng.choice(PERSIAN_GM["kamancheh"] + PERSIAN_GM["santur"])
        inst_har = pretty_midi.Instrument(program=bed_prog); pm.instruments.append(inst_har)
        _cc(inst_har,7,96); _cc(inst_har,10,64)
        def _addp(name, progs, vol, pan):
            insts[name] = pretty_midi.Instrument(program=rng.choice(progs))
            _cc(insts[name],7,vol); _cc(insts[name],10,pan)
            pm.instruments.append(insts[name])
        if "santur"    in roles: _addp("santur", PERSIAN_GM["santur"],   100, 60)
        if "qanun"     in roles: _addp("qanun",  PERSIAN_GM["qanun"],     98, 68)
        if "tar"       in roles: _addp("tar",    PERSIAN_GM["tar"],       96, 56)
        if "setar"     in roles: _addp("setar",  PERSIAN_GM["setar"],     94, 72)
        if "oud"       in roles: _addp("oud",    PERSIAN_GM["oud"],       98, 52)
        if "kamancheh" in roles: _addp("kam",    PERSIAN_GM["kamancheh"], 96, 76)
        if "ney"       in roles: _addp("ney",    PERSIAN_GM["ney"],      100, 64)
        inst_drm = None  # درام غربی نه
    else:
        roles_pool = ["pad","piano","guitar_ac","guitar_el","bass","strings","lead","mallet","brass","guitar_drive"]
        rng.shuffle(roles_pool)
        roles = roles_pool[:rng.randint(3, 6)]
        # Harmony bed
        if "pad" in roles:
            inst_har = pretty_midi.Instrument(program=rng.choice(GM["pad"]+GM["strings"]))
        elif "strings" in roles:
            inst_har = pretty_midi.Instrument(program=rng.choice(GM["strings"]))
        else:
            inst_har = pretty_midi.Instrument(program=rng.choice(GM["piano"]))
        _cc(inst_har, 7, 92); _cc(inst_har, 10, 64)
        pm.instruments.append(inst_har)

        def _addi(name, progs, vol, pan):
            insts[name] = pretty_midi.Instrument(program=rng.choice(progs))
            _cc(insts[name],7,vol); _cc(insts[name],10,pan)
            pm.instruments.append(insts[name])

        if "piano" in roles:        _addi("piano", GM["piano"], 96, 64)
        if "guitar_ac" in roles:    _addi("gtr_ac", GM["guitar_ac"], 96, 52)
        if "guitar_el" in roles:    _addi("gtr_el", GM["guitar_el"], 96, 76)
        if "guitar_drive" in roles: _addi("gtrL", GM["guitar_drive"], 110, 32); _addi("gtrR", GM["guitar_drive"], 110, 96)
        if "bass" in roles:         _addi("bass", GM["bass"], 110, 64)
        if "strings" in roles:      _addi("str2", GM["strings"], 90, 84)
        if "brass" in roles:        _addi("brs", GM["brass"], 96, 70)
        if "mallet" in roles:       _addi("mlt", GM["mallet"], 92, 58)
        if "lead" in roles:         _addi("lead", GM["lead"], 104, 64)
        inst_drm = pretty_midi.Instrument(program=0, is_drum=True) if prof["drums"] else None
        if inst_drm: pm.instruments.append(inst_drm)

    def render_section(start_bar, num_bars, key_this, prog_this, dens_mul=1.0, final_fill=False):
        for i in range(num_bars):
            roman = prog_this[i % len(prog_this)]
            off, qual = (MAJOR_ROMAN if EMO_PROFILE[emotion]["mode"]=="major" else MINOR_ROMAN)[roman]
            root = (key_this + off) % 128
            start = (start_bar+i)*4*beat_dur
            end   = (start_bar+i+1)*4*beat_dur

            if is_persian:
                _add_persian_bed(inst_har, start, end, root, random, add_fifth=True, octave_spread=True)
                step = 0.5*beat_dur
                for name in ["santur","qanun","tar","setar","oud"]:
                    if name in insts and random.random()<0.85:
                        _scale_run(insts[name], start, end, key_this, scale_local, random, step=step)
                if "ney" in insts and random.random()<0.95:
                    _scale_run(insts["ney"], start, end, key_this, scale_local, random, step=step)
            else:
                _add_chord(inst_har, start, end, root, qual, random, spread=random.choice([0,12,-12,0]))
                has_drive = ("gtrL" in insts and "gtrR" in insts)
                if has_drive and emotion=="ANGRY":
                    _power_chord_riff(insts["gtrL"], insts["gtrR"], start, end, root, bpm, random)
                else:
                    gtr = insts.get("gtr_el") or insts.get("gtr_ac")
                    if gtr: _arp_pattern(gtr, start, end, root, qual, random, step=0.5*beat_dur)
                if "piano" in insts and (emotion!="ANGRY") and random.random()<0.85:
                    _arp_pattern(insts["piano"], start, end, root, qual, random, step=0.5*beat_dur)
                if "brs" in insts and random.random()<0.35 and emotion in ["HAPPY","SURPRISE"]:
                    _add_chord(insts["brs"], start, min(start+beat_dur, end), root+12, qual, random)
                if "mlt" in insts and random.random()<0.5:
                    _arp_pattern(insts["mlt"], start, min(start+2*beat_dur, end), root+12, qual, random, step=0.25*beat_dur)
                if "bass" in insts:
                    style_b = "eighths"
                    if emotion in ["SAD","FEAR"]: style_b = "long"
                    if emotion in ["HAPPY","SURPRISE"]: style_b = random.choice(["eighths","syncop"])
                    if emotion=="ANGRY": style_b = "eighths"
                    _bass_pattern(insts["bass"], start, end, root, bpm, random, style=style_b)
                if inst_drm:
                    groove = "swing" if (emotion=="SURPRISE" and random.random()<0.4) else "straight"
                    _drum_bar(inst_drm, start, beat_dur, random, style=groove)

            # FEAR (Global): ارگ کلیسا لانگ‌نوت
            if (not is_persian) and emotion=="FEAR":
                org = pretty_midi.Instrument(program=19)  # Church Organ
                _cc(org,7,98); _cc(org,10,64)
                org.notes.append(pretty_midi.Note(velocity=84, pitch=int(root), start=start, end=end))
                pm.instruments.append(org)

        # Lead
        if is_persian:
            lead_inst = insts.get("ney") or insts.get("kam") or inst_har
            dens = density*dens_mul*(0.55 if emotion=="SAD" else 1.0)
            _lead_melody(lead_inst, start_bar*4*beat_dur, num_bars, bpm, EMO_PROFILE[emotion]["mode"], key_this, dens, random, scale_override=scale_local)
        else:
            lead_inst = insts.get("lead") or insts.get("piano") or inst_har
            _lead_melody(lead_inst, start_bar*4*beat_dur, num_bars, bpm, EMO_PROFILE[emotion]["mode"], key_this, density*dens_mul, random)
        if final_fill and (not is_persian) and inst_drm and (emotion!="SAD"):
            _drum_fill(inst_drm, (start_bar+num_bars-1)*4*beat_dur, beat_dur, random)

    # ساختار A–B–A′
    render_section(0, bars_A, key, progression, dens_mul=1.0, final_fill=True)
    mod_key = key + random.choice([-2, 2, 0])
    new_prog = random.choice(prof["progressions"])
    render_section(bars_A, bars_B, mod_key, new_prog, dens_mul=random.uniform(0.9,1.25), final_fill=True)
    render_section(bars_A+bars_B, bars_A2, key, progression, dens_mul=random.uniform(0.85,1.15), final_fill=False)

    stamp = int(time.time())
    out_mid = f"{emotion.lower()}_{style}_{minutes:.1f}min_{stamp}.mid"
    out_wav = f"{emotion.lower()}_{style}_{minutes:.1f}min_{stamp}.wav"
    wav_path, sr = _render_wav_robust(pm, out_mid, out_wav, sf2_path=sf2_path, sr=22050)

    meta = {
        "emotion": emotion, "style": style, "minutes": minutes, "bpm": bpm, "mode": prof["mode"],
        "key_midi": key, "progression_A": progression, "progression_B": new_prog,
        "drums": (not is_persian) and bool(EMO_PROFILE[emotion]["drums"]),
        "comment": prof["comment"], "bars_total": total_bars, "sr": sr
    }
    return out_mid, wav_path, meta

# ---------------------------- ارزیابی موسیقی --------------------------
TARGETS = {
    "HAPPY":    dict(mode="major", tempo=(115,140), density=(6,10), want_dissonance=False, want_syncopation=True),
    "SAD":      dict(mode="minor", tempo=(45,65),   density=(1,4),  want_dissonance=False, want_syncopation=False),
    "ANGRY":    dict(mode="minor", tempo=(140,175), density=(8,14), want_dissonance=True,  want_syncopation=True),
    "FEAR":     dict(mode="minor", tempo=(50,75),   density=(1,4),  want_dissonance=True,  want_syncopation=False),
    "SURPRISE": dict(mode="major", tempo=(120,145), density=(6,12), want_dissonance=False, want_syncopation=True),
    "OTHER":    dict(mode="major", tempo=(85,110),  density=(4,8),  want_dissonance=False, want_syncopation=False),
}

def analyze_midi_struct(mid_path, emotion, build_params):
    pm = pretty_midi.PrettyMIDI(mid_path)
    non_drums = [ins for ins in pm.instruments if not ins.is_drum]
    note_count = sum(len(ins.notes) for ins in non_drums)
    bars = build_params["bars_total"]
    density = note_count / max(1,bars)
    bpm_guess = build_params["bpm"]

    step = 60.0/bpm_guess/2
    sync_onsets, total_notes = 0, 0
    for ins in non_drums:
        for n in ins.notes:
            total_notes += 1
            frac = (n.start/step) % 2
            if 0.4 < frac < 1.6: sync_onsets += 1
    sync_ratio = (sync_onsets / max(1,total_notes)) if total_notes else 0.0
    has_dim_like = any((n.end-n.start) < (60.0/bpm_guess)/4 for ins in non_drums for n in ins.notes)

    target = TARGETS[emotion]
    mode_match = 1.0 if (build_params["mode"] == target["mode"]) else 0.0
    tempo_fit  = 1.0 if (target["tempo"][0] <= bpm_guess <= target["tempo"][1]) else 0.0
    d_lo, d_hi = target["density"]
    if density<d_lo: density_fit = max(0.0, 1 - (d_lo-density)/max(1e-6,d_lo))
    elif density>d_hi: density_fit = max(0.0, 1 - (density-d_hi)/max(1e-6,d_hi))
    else: density_fit = 1.0
    sync_fit   = min(1.0, sync_ratio*1.5) if target["want_syncopation"] else (1.0 - min(1.0, sync_ratio*1.5))
    disson_fit = 1.0 if (target["want_dissonance"] and has_dim_like) or (not target["want_dissonance"] and not has_dim_like) else 0.5

    score = 0.25*mode_match + 0.25*tempo_fit + 0.25*density_fit + 0.15*disson_fit + 0.10*sync_fit
    return {
        "bpm": bpm_guess, "bars": bars, "note_density_per_bar": round(density,2),
        "mode_match": round(mode_match,2), "tempo_fit": round(tempo_fit,2),
        "density_fit": round(density_fit,2), "dissonance_fit": round(disson_fit,2),
        "syncopation_fit": round(sync_fit,2), "overall_structural_score": round(float(score),3),
        "build_params": build_params,
    }

def evaluate_with_audio_classifier(wav_path, target_emotion:str):
    try:
        out = audio_clf(wav_path)
        top = max(out, key=lambda x:x["score"])
        mapped = AUD_MAP.get(top["label"].lower()[:3], "OTHER")
        return {"predicted_emotion_audio": mapped,
                "predicted_confidence_audio": round(float(top["score"]),3),
                "target_emotion": target_emotion,
                "audio_match": 1.0 if mapped==target_emotion else 0.0}
    except Exception as e:
        print("[audio_eval error]", repr(e))
        return {"predicted_emotion_audio": None,"predicted_confidence_audio":0.0,
                "target_emotion": target_emotion,"audio_match":0.0}

def generate_and_evaluate_full(emotion:str, minutes:float=3.0, seed=None, style:str="global", sf2_path: str | None = None):
    mid_path, wav_path, meta = generate_full_track(emotion, minutes=minutes, seed=seed, style=style, sf2_path=sf2_path)
    struct = analyze_midi_struct(mid_path, emotion, meta)
    audio_eval = evaluate_with_audio_classifier(wav_path, emotion)
    final_score = round(0.75*struct["overall_structural_score"] + 0.25*audio_eval["audio_match"], 3)
    report = {"emotion": emotion, "style": style, "structural": struct, "audio_eval": audio_eval, "final_score": final_score}
    return wav_path, report

# ----------------------------- Gradio UI ------------------------------
def analyze_and_make_music(text, image, audio_path, threshold, minutes, seed, style_choice, sf2_upload):
    label, score, emoji = analyze_all(text, image, audio_path, threshold)
    try:
        seed_int = None if (seed is None or str(seed).strip()=="") else int(seed)
    except:
        seed_int = None
    style = "persian" if (str(style_choice).lower().startswith("pers")) else "global"
    sf2_path = str(sf2_upload) if (sf2_upload is not None and str(sf2_upload).strip()!="") else None

    # تولید + ارزیابی
    wav_path, rep = generate_and_evaluate_full(label, minutes=float(minutes), seed=seed_int, style=style, sf2_path=sf2_path)

    # تبدیل برای پخش آنلاین
    y, sr = sf.read(wav_path, always_2d=False)
    if hasattr(y, "ndim") and y.ndim > 1:
        y = np.mean(y, axis=1)
    y = _normalize_limit(y)

    rep["detected_label"] = label
    rep["detected_confidence"] = score
    rep["soundfont"] = sf2_path if sf2_path else SF2_PATH_DEFAULT
    return label, score, emoji, (sr, y.astype(np.float32)), rep

with gr.Blocks(css="""
  #root {max-width:980px;margin:auto;padding:16px;}
  .out {font-size:1.1rem;text-align:center;}
""") as app:
    gr.Markdown("## 🎛️ تشخیص احساس (متن/تصویر/صوت) → 🎵 ساخت موسیقی چند‌دقیقه‌ای (Global یا Persian) + گزارش ارزیابی")
    with gr.Row():
        with gr.Column():
            txt = gr.Textbox(label="متن (فارسی/انگلیسی)", lines=3, placeholder="مثال: امروز خیلی خوشحالم!")
            img = gr.Image(label="تصویر چهره", type="pil")
            aud = gr.Audio(label="صوت (wav/mp3)", type="filepath")
        with gr.Column():
            thr  = gr.Slider(0,1,0.7,step=0.01, label="آستانه OTHER برای متن/صوت")
            mins = gr.Slider(1.0, 6.0, value=3.0, step=0.5, label="مدت موسیقی (دقیقه)")
            seed = gr.Textbox(value="", label="Seed (خالی = هر بار متفاوت)")
            style_dd = gr.Dropdown(choices=["Global/Western","Persian/Traditional"], value="Global/Western", label="سبک/Palette")
            sf2_file = gr.File(label="SoundFont (sf2) اختیاری — اگر SF2 ایرانی داری آپلود کن", file_types=[".sf2"], type="filepath")
            go   = gr.Button("🔎 تحلیل و 🎵 ساخت موسیقی", variant="primary")

    with gr.Row():
        o1 = gr.Textbox(label="احساس",    interactive=False, elem_classes="out")
        o2 = gr.Textbox(label="اعتماد",   interactive=False, elem_classes="out")
        o3 = gr.Textbox(label="ایموجی",   interactive=False, elem_classes="out")

    audio_out = gr.Audio(label="🎧 موسیقی خروجی", type="numpy")
    rep_out   = gr.JSON(label="📊 گزارش ارزیابی (ساختاری + کلاسیفایر صوت روی WAV)")

    go.click(analyze_and_make_music, [txt, img, aud, thr, mins, seed, style_dd, sf2_file], [o1, o2, o3, audio_out, rep_out])

app.launch(share=False)
# =====================================================================


Extracting templates from packages: 100%
Selecting previously unselected package libqt5core5a:amd64.
(Reading database ... 126380 files and directories currently installed.)
Preparing to unpack .../00-libqt5core5a_5.15.3+dfsg-2ubuntu0.2_amd64.deb ...
Unpacking libqt5core5a:amd64 (5.15.3+dfsg-2ubuntu0.2) ...
Selecting previously unselected package libevdev2:amd64.
Preparing to unpack .../01-libevdev2_1.12.1+dfsg-1_amd64.deb ...
Unpacking libevdev2:amd64 (1.12.1+dfsg-1) ...
Selecting previously unselected package libmtdev1:amd64.
Preparing to unpack .../02-libmtdev1_1.1.6-1build4_amd64.deb ...
Unpacking libmtdev1:amd64 (1.1.6-1build4) ...
Selecting previously unselected package libgudev-1.0-0:amd64.
Preparing to unpack .../03-libgudev-1.0-0_1%3a237-2build1_amd64.deb ...
Unpacking libgudev-1.0-0:amd64 (1:237-2build1) ...
Selecting previously unselected package libwacom-common.
Preparing to unpack .../04-libwacom-common_2.2.0-1_all.deb ...
Unpacking libwacom-common (2.2.0-1) ...
Selecting 

Device set to use cuda:0


config.json:   0%|          | 0.00/768 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

...

preprocessor_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

Device set to use cuda:0


model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

In [8]:
# =======================  یک سلولِ یکپارچه  ==========================
# نصب وابستگی‌ها (به MP3 هم نیاز داریم)
!apt-get -qq -y install fluidsynth ffmpeg
!pip install -q gradio transformers[sentencepiece] soundfile pillow pyFluidSynth==1.3.3 \
                pretty_midi music21 librosa midi2audio langdetect tensorflow opencv-python-headless pydub

# SoundFont پیش‌فرض (General MIDI با کیفیت خوب)
!wget -q https://github.com/pepaslabs/fluidsynth-soundfonts/raw/master/FluidR3_GM/FluidR3_GM.sf2 -O /content/FluidR3_GM.sf2 \
  || wget -q https://github.com/urish/cyberbotics/raw/master/projects/robotbenchmark/resources/fluidr3/FluidR3_GM.sf2 -O /content/FluidR3_GM.sf2

# مدل FER (mini_XCEPTION) برای تصویر
!wget -q https://raw.githubusercontent.com/oarriaga/face_classification/master/trained_models/emotion_models/fer2013_mini_XCEPTION.102-0.66.hdf5 -O /content/emotion_model.h5

# ------------------ Import & Config (TF → CPU) ----------------
import os, time, random, json
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"

import tensorflow as tf
try:
    tf.config.set_visible_devices([], 'GPU')
except Exception:
    pass

import numpy as np
import gradio as gr
import soundfile as sf
from PIL import Image

import torch
from transformers import pipeline
import cv2
from tensorflow.keras.models import load_model
import pretty_midi
from midi2audio import FluidSynth
from langdetect import detect
from pydub import AudioSegment  # برای MP3

# ----------------------- Device برای Transformers ----------------------
DEVICE = 0 if torch.cuda.is_available() else -1
print("Device for Transformers:", "cuda:0" if DEVICE==0 else "cpu")
SF2_PATH_DEFAULT = "/content/FluidR3_GM.sf2"

# ---------- Utility: نرمال‌سازی/لیمیتِ خروجی برای پخش پایدار ----------
def _normalize_limit(x, peak=0.98):
    x = np.array(x)
    if x.ndim == 1:
        x = x[:, None]
    x = np.nan_to_num(x, nan=0.0, posinf=0.0, neginf=0.0)
    x = x - np.mean(x, axis=0, keepdims=True)
    mx = np.max(np.abs(x)) + 1e-9
    x = x / mx
    y = np.tanh(1.2 * x)
    y = y * (peak / (np.max(np.abs(y)) + 1e-9))
    return y.squeeze()

def _render_wav_robust(pm, out_mid, out_wav, sf2_path=None, sr=22050):
    """همیشه اول MIDI را می‌نویسیم تا ارزیابی‌ها fail نشن."""
    sf2_path = sf2_path or SF2_PATH_DEFAULT
    pm.write(out_mid)  # ✅ مهم: همیشه بنویس
    # تلاش ۱: pyfluidsynth
    try:
        audio = pm.fluidsynth(fs=sr, sf2_path=sf2_path)
        audio = _normalize_limit(audio)
        rms = np.sqrt(np.mean(audio**2))
        if (rms < 1e-4) or (not np.isfinite(rms)):
            raise ValueError("silent_or_invalid_audio")
        sf.write(out_wav, audio, sr)
        return out_wav, sr
    except Exception as e:
        print("[render] fallback to midi2audio:", e)
        FluidSynth(sound_font=sf2_path, sample_rate=sr).midi_to_audio(out_mid, out_wav)
        y, sr2 = sf.read(out_wav, always_2d=False)
        y = _normalize_limit(y)
        sf.write(out_wav, y, sr2)
        return out_wav, sr2

def _to_mp3(wav_path, mp3_path, bitrate="192k"):
    try:
        AudioSegment.from_file(wav_path).export(mp3_path, format="mp3", bitrate=bitrate)
        return mp3_path
    except Exception as e:
        print("[mp3] export error:", e)
        return None

# ----------------------------- Emoji & Maps ---------------------------
EMOJI = {"ANGRY":"😠","FEAR":"😨","HAPPY":"😃","SAD":"😢","SURPRISE":"😲","OTHER":"😐"}

# ------------------------- Pipelines متن/صوت --------------------------
# پارس‌برت اختیاری—اگر نبود، انگلیسی کار می‌کند
try:
    pipe_fa = pipeline(
        "text-classification",
        model="/content/final_parsbert_emotion",
        tokenizer="/content/final_parsbert_emotion",
        device=DEVICE
    )
except Exception as e:
    pipe_fa = None
    print("[warn] Persian text model not found -> using English when needed")

pipe_en = pipeline(
    "text-classification",
    model="nateraw/bert-base-uncased-emotion",
    tokenizer="nateraw/bert-base-uncased-emotion",
    device=DEVICE,
    return_all_scores=False
)

def map_en(label):
    l = label.lower()
    if l in ["joy","love"]: return "HAPPY"
    if l == "sadness":      return "SAD"
    if l == "anger":        return "ANGRY"
    if l == "fear":         return "FEAR"
    if l == "surprise":     return "SURPRISE"
    return "OTHER"

audio_clf = pipeline("audio-classification", model="superb/wav2vec2-base-superb-er", device=DEVICE, top_k=None)
AUD_MAP = {"hap":"HAPPY","sad":"SAD","ang":"ANGRY","fea":"FEAR","sur":"SURPRISE","neu":"OTHER"}

# ------------------------- مدل تصویر (TF روی CPU) ---------------------
emotion_cnn = load_model("/content/emotion_model.h5", compile=False)
FER_LABELS = ['angry','disgust','fear','happy','sad','surprise','neutral']
IMG_MAP = {
    "angry":"ANGRY","disgust":"ANGRY","fear":"FEAR",
    "happy":"HAPPY","sad":"SAD","surprise":"SURPRISE","neutral":"OTHER"
}
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# -------------------------- تشخیص احساس‌ها ----------------------------
def analyze_text(text, threshold):
    if not text or not str(text).strip():
        return "OTHER", 0.0, EMOJI["OTHER"]
    try:
        lang = detect(text)
    except:
        lang = "fa"
    if lang == "fa" and pipe_fa is not None:
        out = pipe_fa(text)[0]
        idx = int(out["label"].split("_")[-1])
        FA = ['ANGRY','FEAR','HAPPY','HATE','OTHER','SAD','SURPRISE']
        label = FA[idx] if FA[idx]!="HATE" else "ANGRY"
        score = float(out["score"])
    else:
        out = pipe_en(text)[0]
        label = map_en(out["label"])
        score = float(out["score"])
    if score < threshold:
        label = "OTHER"
    return label, round(score,3), EMOJI[label]

def analyze_image(img, threshold):
    try:
        gray = cv2.cvtColor(np.array(img.convert("RGB")), cv2.COLOR_RGB2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)
        if len(faces)==0:
            h,w = gray.shape
            sz = min(h,w)//2
            x = (w-sz)//2; y = (h-sz)//2
            roi = gray[y:y+sz, x:x+sz]
        else:
            x,y,w,h = faces[0]
            roi = gray[y:y+h, x:x+w]
        face = cv2.resize(roi, (64,64), interpolation=cv2.INTER_AREA)
        face = face.astype("float32")/255.0
        face = face.reshape(1,64,64,1)
        preds = emotion_cnn.predict(face, verbose=0)[0]
        idx   = int(np.argmax(preds))
        score = float(preds[idx])
        raw   = FER_LABELS[idx]
        label = IMG_MAP.get(raw, "OTHER")
        return label, round(score,3), EMOJI[label]
    except Exception as e:
        print("[analyze_image error]", repr(e))
        return "OTHER", 0.0, EMOJI["OTHER"]

def analyze_audio(filepath, threshold):
    try:
        out = audio_clf(filepath)
        if not out:
            return "OTHER", 0.0, EMOJI["OTHER"]
        top = max(out, key=lambda x: x["score"])
        raw = top["label"].lower()
        score = float(top["score"])
        label = AUD_MAP.get(raw[:3], "OTHER")
        if score < threshold:
            label = "OTHER"
        return label, round(score,3), EMOJI[label]
    except Exception as e:
        print("[analyze_audio error]", repr(e))
        return "OTHER", 0.0, EMOJI["OTHER"]

def analyze_all(text, image, audio_path, threshold):
    if text and str(text).strip():
        return analyze_text(text, threshold)
    if image is not None:
        return analyze_image(image, threshold)
    if audio_path:
        return analyze_audio(audio_path, threshold)
    return "OTHER", 0.0, EMOJI["OTHER"]

# --------------------- تنظیمات ساخت موسیقی ----------------
GM = dict(
    piano=[0,1,4,5,6],
    guitar_ac=[24,25], guitar_el=[26,27,28],
    guitar_drive=[29,30],
    bass=[32,33,34,39],
    strings=[48,49,50],
    brass=[60,61],
    mallet=[11,12,13,14],
    pad=[88,89,91,94],
    lead=[80,81,82,84]
)

# سازهای ایرانی (تقریب با GM)
PERSIAN_GM = dict(
    santur=[15],               # Dulcimer ≈ Santur
    tar=[24,104,107],          # NylonGtr / Sitar / Koto
    setar=[24,104],
    oud=[24,25,104],
    kamancheh=[40],            # Violin
    ney=[77,75],               # Shakuhachi / Pan Flute
    qanun=[46,15,107],         # Harp / Dulcimer / Koto
)

# تنها سازهای ایرانی در حالت Persian
PERSIAN_ROLES = {
    "HAPPY":    ["santur","qanun","oud","ney","kamancheh","tar","setar"],
    "SAD":      ["kamancheh","ney","santur","qanun","setar","tar"],
    "ANGRY":    ["santur","tar","kamancheh","ney","oud"],   # خشم ایرانی بدون گیتار غربی
    "FEAR":     ["ney","kamancheh","santur","qanun"],
    "SURPRISE": ["santur","qanun","oud","ney","kamancheh"],
    "OTHER":    ["santur","qanun","ney","kamancheh","setar"]
}

EMO_PROFILE = {
    "HAPPY":    {"mode":"major","tempo_range":(124,142),"base_keys":[60,62,65,67],
                 "progressions":[["I","V","vi","IV"],["I","IV","V","I"],["I","V","IV","I"]],
                 "melody_density_range":(0.7,0.95),"drums":True,"comment":"شاد و رقصان"},
    "SAD":      {"mode":"minor","tempo_range":(45,60),"base_keys":[57,55,52],
                 "progressions":[["i","VI","III","VII"],["i","iv","i","VII"],["i","VI","i","v"]],
                 "melody_density_range":(0.12,0.28),"drums":False,"comment":"خیلی آرام و افسرده"},
    "ANGRY":    {"mode":"minor","tempo_range":(150,175),"base_keys":[50,48,53],
                 "progressions":[["i","bVI","bVII","i"],["i","bII°","bVII","i"]],
                 "melody_density_range":(0.8,0.96),"drums":True,"comment":"راک/متال تهاجمی"},
    "FEAR":     {"mode":"minor","tempo_range":(50,68),"base_keys":[57,58,55],
                 "progressions":[["i","bII°","i","bVI"],["i","iv","bII°","i"]],
                 "melody_density_range":(0.12,0.3),"drums":False,"comment":"دلهره‌آور"},
    "SURPRISE": {"mode":"major","tempo_range":(122,145),"base_keys":[65,67,69],
                 "progressions":[["I","V","IV","♭VII"],["I","IV","V","♭VII"]],
                 "melody_density_range":(0.6,0.95),"drums":True,"comment":"جهشی/سینکوپ"},
    "OTHER":    {"mode":"major","tempo_range":(90,110),"base_keys":[60,62],
                 "progressions":[["I","ii","IV","V"],["I","IV","ii","V"]],
                 "melody_density_range":(0.4,0.65),"drums":False,"comment":"خنثی/پس‌زمینه"},
}

# ✅ فیکس اصلی: «III» در مینور
MAJOR_ROMAN = {"I":(0,"maj"),"ii":(2,"min"),"iii":(4,"min"),"IV":(5,"maj"),"V":(7,"maj"),"vi":(9,"min"),"♭VII":(10,"maj")}
MINOR_ROMAN = {"i":(0,"min"),"ii°":(2,"dim"),"III":(3,"maj"),"iv":(5,"min"),"v":(7,"min"),
               "VI":(8,"maj"),"VII":(10,"maj"),"bVI":(8,"maj"),"bVII":(10,"maj"),"bII°":(1,"dim")}
QUAL2INTS = {"maj":[0,4,7], "min":[0,3,7], "dim":[0,3,6]}

def _scale_for(mode:str, emotion:str, style:str):
    if style != "persian":
        return [0,2,4,5,7,9,11] if mode=="major" else [0,2,3,5,7,8,10]
    # تقریبی از دستگاه‌ها (بدون ربع‌پرده)
    if emotion in ["HAPPY","SURPRISE"]: return [0,1,4,5,7,8,11]   # Hijaz / Phrygian-dominant
    if emotion in ["SAD"]:              return [0,1,3,5,7,8,10]   # Phrygian (حزن)
    if emotion in ["FEAR"]:             return [0,1,3,5,6,8,10]   # Saba-like (دلهره)
    return [0,2,3,5,7,8,10]             # Nahawand-like

def _add_chord(inst, start, end, root, qual, rng, spread=0):
    for i in QUAL2INTS[qual]:
        p = int(root+i + spread)
        vel = int(np.clip(72 + rng.randint(-8,8), 40, 105))
        inst.notes.append(pretty_midi.Note(velocity=vel, pitch=p, start=start, end=end))

def _add_persian_bed(inst, start, end, root, rng, add_fifth=True, octave_spread=True):
    base = int(root)
    p_list = [base]
    if add_fifth: p_list.append(base+7)
    if octave_spread and (rng.random()<0.6):
        p_list += [base-12, base+12]
    vel = int(np.clip(78 + rng.randint(-6,6), 50, 105))
    for p in p_list:
        p = max(0, min(127, p))
        inst.notes.append(pretty_midi.Note(velocity=vel, pitch=p, start=start, end=end))

def _arp_pattern(inst, start, end, root, qual, rng, step):
    ints = QUAL2INTS[qual]; seq = ints + ints[::-1]
    t = start; idx = rng.randint(0,len(seq)-1)
    while t < end:
        p = int(root + seq[idx % len(seq)])
        dur = step * (1.0 if rng.random()<0.7 else 0.5)
        vel = int(np.clip(74 + rng.randint(-12,12), 40, 110))
        jit = rng.uniform(-0.015,0.015)
        inst.notes.append(pretty_midi.Note(velocity=vel, pitch=p, start=t+jit, end=min(t+jit+dur, end)))
        t += step; idx += 1

def _scale_run(inst, start, end, key, scale, rng, step):
    t = start; idx = rng.randint(0,len(scale)-1); direction = rng.choice([1,-1])
    last = key + scale[idx]
    while t < end:
        deg = scale[idx % len(scale)]
        base = key + deg + rng.choice([-12,0,12])
        pitch = int(np.clip(int(0.65*last + 0.35*base), key-7, key+19))
        dur = step if rng.random()<0.8 else step*2
        vel = int(np.clip(76 + rng.randint(-10,10), 45, 110))
        jit = rng.uniform(-0.015,0.015)
        inst.notes.append(pretty_midi.Note(velocity=vel, pitch=pitch, start=t+jit, end=min(t+jit+dur, end)))
        last = pitch; t += step; idx += direction
        if rng.random()<0.12: direction *= -1

def _bass_pattern(inst, start, end, root, bpm, rng, style="eighths"):
    beat = 60.0/bpm
    if style=="long":
        p = max(0, root-12)
        inst.notes.append(pretty_midi.Note(velocity=74, pitch=p, start=start, end=end))
    elif style=="syncop":
        for k in [0.5, 1.5, 2.5, 3.5]:
            t = start + k*beat + rng.uniform(-0.01,0.01)
            inst.notes.append(pretty_midi.Note(velocity=85+rng.randint(-6,6), pitch=max(0,root-12), start=t, end=min(t+0.30, end)))
    else:
        t = start
        while t < end:
            p = max(0, root-12)
            d = 0.45*beat
            inst.notes.append(pretty_midi.Note(velocity=82+rng.randint(-8,8), pitch=p, start=t, end=min(t+d, end)))
            t += 0.5*beat

def _lead_melody(inst, start, bars, bpm, mode, key, density, rng, scale_override=None):
    scale = scale_override if scale_override is not None else ([0,2,4,5,7,9,11] if mode=="major" else [0,2,3,5,7,8,10])
    step = 0.5*(60.0/bpm)       # 8th
    total_sub = int(bars*4*2)
    t = start; last_pitch = key + 7 + rng.choice([-12,0,12])
    lo, hi = key-7, key+19
    motif = [rng.choice(scale) for _ in range(rng.choice([3,4,5,6]))]
    for s in range(total_sub):
        if rng.random() < density:
            deg = motif[s % len(motif)] if rng.random()<0.75 else rng.choice(scale)
            base = key + deg + rng.choice([-12,0,12])
            pitch = int(np.clip(int(0.65*last_pitch + 0.35*base), lo, hi))
            dur = step if rng.random()<0.65 else step*2
            jit = rng.uniform(-0.02,0.02)
            vel = int(np.clip(80 + rng.randint(-14,14), 40, 115))
            inst.notes.append(pretty_midi.Note(velocity=vel, pitch=pitch, start=t+jit, end=t+jit+dur))
            last_pitch = pitch
        t += step

# --- درام‌ها
def _drum_bar(inst_drm, bar_start, beat_dur, rng, style="straight"):
    """درام ساده‌ی 4/4"""
    for b in range(4):
        t = bar_start + b*beat_dur
        inst_drm.notes.append(pretty_midi.Note(100 if b%2==0 else 92, pitch=36 if b%2==0 else 38, start=t, end=t+0.06))
        inst_drm.notes.append(pretty_midi.Note(62, pitch=42, start=t, end=t+0.03))
        if style=="swing":
            inst_drm.notes.append(pretty_midi.Note(62, pitch=42, start=t+beat_dur*2/3, end=t+beat_dur*2/3+0.03))
        else:
            inst_drm.notes.append(pretty_midi.Note(62, pitch=42, start=t+beat_dur/2, end=t+beat_dur/2+0.03))

def _dance_4onfloor(inst_drm, bar_start, beat_dur, rng):
    """چهار-به-کف برای شاد غربی"""
    for b in range(4):
        t = bar_start + b*beat_dur
        # Kick on every beat
        inst_drm.notes.append(pretty_midi.Note(108, pitch=36, start=t, end=t+0.08))
        # Snare/Clap on 2 & 4
        if b in [1,3]:
            inst_drm.notes.append(pretty_midi.Note(110, pitch=38, start=t, end=t+0.06))
            inst_drm.notes.append(pretty_midi.Note(120, pitch=39, start=t, end=t+0.03))  # clap
        # Hi-hat 8ths
        inst_drm.notes.append(pretty_midi.Note(70, pitch=42, start=t, end=t+0.03))
        inst_drm.notes.append(pretty_midi.Note(70, pitch=42, start=t+beat_dur/2, end=t+beat_dur/2+0.03))

def _metal_bar(inst_drm, bar_start, beat_dur, rng):
    """درام متال: دابل‌کیک + کرش آغاز هر میزان"""
    # Crash on bar start
    inst_drm.notes.append(pretty_midi.Note(100, pitch=49, start=bar_start, end=bar_start+0.5*beat_dur))
    # 16th grid
    for s in range(16):
        t = bar_start + s*(beat_dur/4)
        # Snare on beats 2 and 4
        if abs(t - (bar_start+beat_dur)) < 1e-6 or abs(t - (bar_start+3*beat_dur)) < 1e-6:
            inst_drm.notes.append(pretty_midi.Note(110, pitch=38, start=t, end=t+0.05))
        else:
            # Double kick dense (avoid exactly at snare)
            inst_drm.notes.append(pretty_midi.Note(100, pitch=36, start=t, end=t+0.04))
        # Hi-hat 8ths
        if s % 2 == 0:
            inst_drm.notes.append(pretty_midi.Note(72, pitch=42, start=t, end=t+0.03))

def _persian_68(inst_drm, bar_start, beat_dur, rng):
    """ریتم شش‌و‌هشت ایرانی شبه‌تنبک/دف با سازهای GM (کونگا/تمبورین/کِلَپ)"""
    step = beat_dur*(2/3)  # 6 پالس در هر میزان 4/4 → حس 6/8
    pulses = [bar_start + i*step for i in range(6)]
    # Dum روی 1 و 4 (کونگای بم)
    for i in [0,3]:
        inst_drm.notes.append(pretty_midi.Note(105, pitch=64, start=pulses[i], end=pulses[i]+0.08))  # Low Conga
    # Tek روی 3 و 6 (کونگای زیر/باز)
    for i in [2,5]:
        inst_drm.notes.append(pretty_midi.Note(96, pitch=63, start=pulses[i], end=pulses[i]+0.06))  # Open High Conga
    # تمبورین ریز
    for i in range(6):
        inst_drm.notes.append(pretty_midi.Note(70, pitch=54, start=pulses[i], end=pulses[i]+0.03))
    # دست‌زدن روی 4
    inst_drm.notes.append(pretty_midi.Note(120, pitch=39, start=pulses[3], end=pulses[3]+0.04))

def _drum_fill(inst_drm, bar_start, beat_dur, rng):
    for i, pitch in enumerate([47,45,43,41]):
        t = bar_start + i*(beat_dur/2)
        inst_drm.notes.append(pretty_midi.Note(86+rng.randint(-6,6), pitch=pitch, start=t, end=t+0.12))

def _cc(inst, cc, value, time=0.0):
    inst.control_changes.append(pretty_midi.ControlChange(cc, int(value), time))

def _power_chord_riff(inst_L, inst_R, start, end, root, bpm, rng):
    beat = 60.0/bpm; t = start
    while t < end:
        dur = min(0.45*beat, end-t)
        velL = int(np.clip(110 + rng.randint(-8,8), 60, 127))
        velR = int(np.clip(108 + rng.randint(-8,8), 60, 127))
        for off in [0,7,12]:
            inst_L.notes.append(pretty_midi.Note(velocity=velL, pitch=int(root+off), start=t+rng.uniform(-0.005,0.005), end=min(t+dur, end)))
            inst_R.notes.append(pretty_midi.Note(velocity=velR, pitch=int(root+off), start=t+rng.uniform(-0.005,0.005), end=min(t+dur, end)))
        t += 0.5*beat  # 8th push

def generate_full_track(emotion: str, minutes: float = 3.0, seed: int | None = None, style: str = "global", sf2_path: str | None = None):
    assert emotion in EMO_PROFILE, f"unknown emotion {emotion}"
    rng = random.Random(seed if seed is not None else int(time.time()*1000) % 2_147_483_647)
    prof = EMO_PROFILE[emotion]
    bpm = rng.randint(*prof["tempo_range"])
    key = rng.choice(prof["base_keys"])
    progression = rng.choice(prof["progressions"])
    density = rng.uniform(*prof["melody_density_range"])
    is_persian = (style == "persian")
    sec_per_bar = 4*(60.0/bpm)
    total_bars = max(16, int((minutes*60)/sec_per_bar))
    bars_A = int(total_bars*0.4); bars_B = int(total_bars*0.4); bars_A2 = total_bars - (bars_A+bars_B)
    beat_dur = (60.0/bpm)
    mode_here = prof["mode"]
    ROM = MAJOR_ROMAN if mode_here=="major" else MINOR_ROMAN
    scale_local = _scale_for(mode_here, emotion, style)

    pm = pretty_midi.PrettyMIDI()
    insts = {}

    if is_persian:
        roles_src = PERSIAN_ROLES[emotion][:]
        rng.shuffle(roles_src)
        roles = roles_src[:rng.randint(3, min(6, len(roles_src)))]
        bed_prog = rng.choice(PERSIAN_GM["kamancheh"] + PERSIAN_GM["santur"])
        inst_har = pretty_midi.Instrument(program=bed_prog); pm.instruments.append(inst_har)
        _cc(inst_har,7,96); _cc(inst_har,10,64)

        def _addp(name, progs, vol, pan):
            insts[name] = pretty_midi.Instrument(program=rng.choice(progs))
            _cc(insts[name],7,vol); _cc(insts[name],10,pan)
            pm.instruments.append(insts[name])

        if "santur"    in roles: _addp("santur", PERSIAN_GM["santur"],   100, 60)
        if "qanun"     in roles: _addp("qanun",  PERSIAN_GM["qanun"],     98, 68)
        if "tar"       in roles: _addp("tar",    PERSIAN_GM["tar"],       96, 56)
        if "setar"     in roles: _addp("setar",  PERSIAN_GM["setar"],     94, 72)
        if "oud"       in roles: _addp("oud",    PERSIAN_GM["oud"],       98, 52)
        if "kamancheh" in roles: _addp("kam",    PERSIAN_GM["kamancheh"], 96, 76)
        if "ney"       in roles: _addp("ney",    PERSIAN_GM["ney"],      100, 64)

        # فقط برای HAPPY ایرانی: ریتم 6/8 (daf/tonbak شبیه‌سازی با درام GM)
        inst_drm = pretty_midi.Instrument(program=0, is_drum=True) if emotion=="HAPPY" else None
        if inst_drm: pm.instruments.append(inst_drm)
    else:
        roles_pool = ["pad","piano","guitar_ac","guitar_el","bass","strings","lead","mallet","brass","guitar_drive"]
        rng.shuffle(roles_pool)
        roles = roles_pool[:rng.randint(3, 6)]

        # اجباری‌ها برای انرژی بهتر
        if emotion=="ANGRY":
            if "guitar_drive" not in roles: roles.append("guitar_drive")
            if "bass" not in roles: roles.append("bass")
            prof["drums"] = True
        if emotion=="HAPPY":
            if "bass" not in roles: roles.append("bass")
            if "brass" not in roles: roles.append("brass")
            prof["drums"] = True

        # Harmony bed
        if "pad" in roles:
            inst_har = pretty_midi.Instrument(program=rng.choice(GM["pad"]+GM["strings"]))
        elif "strings" in roles:
            inst_har = pretty_midi.Instrument(program=rng.choice(GM["strings"]))
        else:
            inst_har = pretty_midi.Instrument(program=rng.choice(GM["piano"]))
        _cc(inst_har, 7, 92); _cc(inst_har, 10, 64)
        pm.instruments.append(inst_har)

        def _addi(name, progs, vol, pan):
            if name in insts: return
            insts[name] = pretty_midi.Instrument(program=rng.choice(progs))
            _cc(insts[name],7,vol); _cc(insts[name],10,pan)
            pm.instruments.append(insts[name])

        if "piano" in roles:        _addi("piano", GM["piano"], 96, 64)
        if "guitar_ac" in roles:    _addi("gtr_ac", GM["guitar_ac"], 96, 52)
        if "guitar_el" in roles:    _addi("gtr_el", GM["guitar_el"], 96, 76)
        if "guitar_drive" in roles: _addi("gtrL", GM["guitar_drive"], 110, 32); _addi("gtrR", GM["guitar_drive"], 110, 96)
        if "bass" in roles:         _addi("bass", GM["bass"], 110, 64)
        if "strings" in roles:      _addi("str2", GM["strings"], 90, 84)
        if "brass" in roles:        _addi("brs", GM["brass"], 96, 70)
        if "mallet" in roles:       _addi("mlt", GM["mallet"], 92, 58)
        if "lead" in roles:         _addi("lead", GM["lead"], 104, 64)
        inst_drm = pretty_midi.Instrument(program=0, is_drum=True) if prof["drums"] else None
        if inst_drm: pm.instruments.append(inst_drm)

    def render_section(start_bar, num_bars, key_this, prog_this, dens_mul=1.0, final_fill=False):
        for i in range(num_bars):
            roman = prog_this[i % len(prog_this)]
            # ✅ فالسِیف در صورت درجه ناشناخته
            if roman not in ROM:
                off, qual = (0, "maj" if mode_here=="major" else "min")
            else:
                off, qual = ROM[roman]
            root = (key_this + off) % 128
            start = (start_bar+i)*4*beat_dur
            end   = (start_bar+i+1)*4*beat_dur

            if is_persian:
                _add_persian_bed(inst_har, start, end, root, rng, add_fifth=True, octave_spread=True)
                step = 0.5*beat_dur
                for name in ["santur","qanun","tar","setar","oud"]:
                    if name in insts and rng.random()<0.85:
                        _scale_run(insts[name], start, end, key_this, scale_local, rng, step=step)
                if "ney" in insts and rng.random()<0.95:
                    _scale_run(insts["ney"], start, end, key_this, scale_local, rng, step=step)
                if inst_drm:
                    _persian_68(inst_drm, start, beat_dur, rng)
            else:
                _add_chord(inst_har, start, end, root, qual, rng, spread=rng.choice([0,12,-12,0]))
                has_drive = ("gtrL" in insts and "gtrR" in insts)
                if has_drive and emotion=="ANGRY":
                    _power_chord_riff(insts["gtrL"], insts["gtrR"], start, end, root, bpm, rng)
                else:
                    gtr = insts.get("gtr_el") or insts.get("gtr_ac")
                    if gtr: _arp_pattern(gtr, start, end, root, qual, rng, step=0.5*beat_dur)
                if "piano" in insts and (emotion!="ANGRY") and rng.random()<0.85:
                    _arp_pattern(insts["piano"], start, end, root, qual, rng, step=0.5*beat_dur)
                if "brs" in insts and rng.random()<0.4 and emotion in ["HAPPY","SURPRISE"]:
                    _add_chord(insts["brs"], start, min(start+beat_dur, end), root+12, qual, rng)
                if "mlt" in insts and rng.random()<0.5:
                    _arp_pattern(insts["mlt"], start, min(start+2*beat_dur, end), root+12, qual, rng, step=0.25*beat_dur)
                if "bass" in insts:
                    style_b = "eighths"
                    if emotion in ["SAD","FEAR"]: style_b = "long"
                    if emotion in ["HAPPY","SURPRISE"]: style_b = rng.choice(["eighths","syncop"])
                    if emotion=="ANGRY": style_b = "eighths"
                    _bass_pattern(insts["bass"], start, end, root, bpm, rng, style=style_b)
                if inst_drm:
                    if emotion=="HAPPY":      _dance_4onfloor(inst_drm, start, beat_dur, rng)
                    elif emotion=="ANGRY":    _metal_bar(inst_drm, start, beat_dur, rng)
                    elif emotion=="SURPRISE": _drum_bar(inst_drm, start, beat_dur, rng, style="swing" if rng.random()<0.4 else "straight")
                    elif emotion not in ["SAD","FEAR"]:
                        _drum_bar(inst_drm, start, beat_dur, rng, style="straight")

            # FEAR (Global): ارگ کلیسا لانگ‌نوت
            if (not is_persian) and emotion=="FEAR":
                org = pretty_midi.Instrument(program=19)  # Church Organ
                _cc(org,7,98); _cc(org,10,64)
                org.notes.append(pretty_midi.Note(velocity=84, pitch=int(root), start=start, end=end))
                pm.instruments.append(org)

        # Lead
        if is_persian:
            lead_inst = insts.get("ney") or insts.get("kam") or inst_har
            dens = density*dens_mul*(0.5 if emotion=="SAD" else 1.0)
            _lead_melody(lead_inst, start_bar*4*beat_dur, num_bars, bpm, mode_here, key_this, dens, rng, scale_override=scale_local)
        else:
            lead_inst = insts.get("lead") or insts.get("piano") or inst_har
            dmul = density*dens_mul
            if emotion=="SAD": dmul *= 0.6
            _lead_melody(lead_inst, start_bar*4*beat_dur, num_bars, bpm, mode_here, key_this, dmul, rng)
        if final_fill and (not is_persian) and inst_drm and (emotion not in ["SAD","FEAR"]):
            _drum_fill(inst_drm, (start_bar+num_bars-1)*4*beat_dur, beat_dur, rng)

    # ساختار A–B–A′
    render_section(0, bars_A, key, progression, dens_mul=1.0, final_fill=True)
    mod_key = key + rng.choice([-2, 2, 0])
    new_prog = rng.choice(prof["progressions"])
    render_section(bars_A, bars_B, mod_key, new_prog, dens_mul=rng.uniform(0.9,1.25), final_fill=True)
    render_section(bars_A+bars_B, bars_A2, key, progression, dens_mul=rng.uniform(0.85,1.15), final_fill=False)

    stamp = int(time.time())
    out_mid = f"{emotion.lower()}_{style}_{minutes:.1f}min_{stamp}.mid"
    out_wav = f"{emotion.lower()}_{style}_{minutes:.1f}min_{stamp}.wav"
    wav_path, sr = _render_wav_robust(pm, out_mid, out_wav, sf2_path=sf2_path, sr=22050)

    # MP3
    out_mp3 = f"{emotion.lower()}_{style}_{minutes:.1f}min_{stamp}.mp3"
    _ = _to_mp3(wav_path, out_mp3, bitrate="192k")

    meta = {
        "emotion": emotion, "style": style, "minutes": minutes, "bpm": bpm, "mode": mode_here,
        "key_midi": key, "progression_A": progression, "progression_B": new_prog,
        "drums": (not is_persian) and bool(EMO_PROFILE[emotion]["drums"]) or (is_persian and emotion=="HAPPY"),
        "comment": prof["comment"], "bars_total": total_bars, "sr": sr
    }
    return out_mid, wav_path, out_mp3, meta

# ---------------------------- ارزیابی موسیقی --------------------------
TARGETS = {
    "HAPPY":    dict(mode="major", tempo=(124,142), density=(7,12), want_dissonance=False, want_syncopation=True),
    "SAD":      dict(mode="minor", tempo=(45,60),   density=(1,4),  want_dissonance=False, want_syncopation=False),
    "ANGRY":    dict(mode="minor", tempo=(150,175), density=(8,14), want_dissonance=True,  want_syncopation=True),
    "FEAR":     dict(mode="minor", tempo=(50,68),   density=(1,4),  want_dissonance=True,  want_syncopation=False),
    "SURPRISE": dict(mode="major", tempo=(122,145), density=(6,12), want_dissonance=False, want_syncopation=True),
    "OTHER":    dict(mode="major", tempo=(85,110),  density=(4,8),  want_dissonance=False, want_syncopation=False),
}

def analyze_midi_struct(mid_path, emotion, build_params):
    pm = pretty_midi.PrettyMIDI(mid_path)
    non_drums = [ins for ins in pm.instruments if not ins.is_drum]
    note_count = sum(len(ins.notes) for ins in non_drums)
    bars = build_params["bars_total"]
    density = note_count / max(1,bars)
    bpm_guess = build_params["bpm"]

    step = 60.0/bpm_guess/2
    sync_onsets, total_notes = 0, 0
    for ins in non_drums:
        for n in ins.notes:
            total_notes += 1
            frac = (n.start/step) % 2
            if 0.4 < frac < 1.6: sync_onsets += 1
    sync_ratio = (sync_onsets / max(1,total_notes)) if total_notes else 0.0
    has_dim_like = any((n.end-n.start) < (60.0/bpm_guess)/4 for ins in non_drums for n in ins.notes)

    target = TARGETS[emotion]
    mode_match = 1.0 if (build_params["mode"] == target["mode"]) else 0.0
    tempo_fit  = 1.0 if (target["tempo"][0] <= bpm_guess <= target["tempo"][1]) else 0.0
    d_lo, d_hi = target["density"]
    if density<d_lo: density_fit = max(0.0, 1 - (d_lo-density)/max(1e-6,d_lo))
    elif density>d_hi: density_fit = max(0.0, 1 - (density-d_hi)/max(1e-6,d_hi))
    else: density_fit = 1.0
    sync_fit   = min(1.0, sync_ratio*1.5) if target["want_syncopation"] else (1.0 - min(1.0, sync_ratio*1.5))
    disson_fit = 1.0 if (target["want_dissonance"] and has_dim_like) or (not target["want_dissonance"] and not has_dim_like) else 0.5

    score = 0.25*mode_match + 0.25*tempo_fit + 0.25*density_fit + 0.15*disson_fit + 0.10*sync_fit
    return {
        "bpm": bpm_guess, "bars": bars, "note_density_per_bar": round(density,2),
        "mode_match": round(mode_match,2), "tempo_fit": round(tempo_fit,2),
        "density_fit": round(density_fit,2), "dissonance_fit": round(disson_fit,2),
        "syncopation_fit": round(sync_fit,2), "overall_structural_score": round(float(score),3),
        "build_params": build_params,
    }

def evaluate_with_audio_classifier(wav_path, target_emotion:str):
    try:
        out = audio_clf(wav_path)
        top = max(out, key=lambda x:x["score"])
        mapped = AUD_MAP.get(top["label"].lower()[:3], "OTHER")
        return {"predicted_emotion_audio": mapped,
                "predicted_confidence_audio": round(float(top["score"]),3),
                "target_emotion": target_emotion,
                "audio_match": 1.0 if mapped==target_emotion else 0.0}
    except Exception as e:
        print("[audio_eval error]", repr(e))
        return {"predicted_emotion_audio": None,"predicted_confidence_audio":0.0,
                "target_emotion": target_emotion,"audio_match":0.0}

def generate_and_evaluate_full(emotion:str, minutes:float=3.0, seed=None, style:str="global", sf2_path: str | None = None):
    mid_path, wav_path, mp3_path, meta = generate_full_track(emotion, minutes=minutes, seed=seed, style=style, sf2_path=sf2_path)
    struct = analyze_midi_struct(mid_path, emotion, meta)
    audio_eval = evaluate_with_audio_classifier(wav_path, emotion)
    final_score = round(0.75*struct["overall_structural_score"] + 0.25*audio_eval["audio_match"], 3)
    report = {"emotion": emotion, "style": style, "structural": struct, "audio_eval": audio_eval, "final_score": final_score}
    return wav_path, mp3_path, report

# ----------------------------- Gradio UI ------------------------------
def analyze_and_make_music(text, image, audio_path, threshold, minutes, seed, style_choice, sf2_upload):
    label, score, emoji = analyze_all(text, image, audio_path, threshold)
    try:
        seed_int = None if (seed is None or str(seed).strip()=="") else int(seed)
    except:
        seed_int = None
    style = "persian" if (str(style_choice).lower().startswith("pers")) else "global"
    sf2_path = str(sf2_upload) if (sf2_upload is not None and str(sf2_upload).strip()!="") else None

    # تولید + ارزیابی
    wav_path, mp3_path, rep = generate_and_evaluate_full(label, minutes=float(minutes), seed=seed_int, style=style, sf2_path=sf2_path)

    # تبدیل برای پخش آنلاین
    y, sr = sf.read(wav_path, always_2d=False)
    if hasattr(y, "ndim") and y.ndim > 1:
        y = np.mean(y, axis=1)
    y = _normalize_limit(y)

    rep["detected_label"] = label
    rep["detected_confidence"] = score
    rep["soundfont"] = sf2_path if sf2_path else SF2_PATH_DEFAULT
    return label, score, emoji, (sr, y.astype(np.float32)), rep, (mp3_path if mp3_path else wav_path)

with gr.Blocks(css="""
  #root {max-width:980px;margin:auto;padding:16px;}
  .out {font-size:1.1rem;text-align:center;}
""") as app:
    gr.Markdown("## 🎛️ تشخیص احساس (متن/تصویر/صوت) → 🎵 ساخت موسیقی چند‌دقیقه‌ای (Global یا Persian) + گزارش ارزیابی + MP3")
    with gr.Row():
        with gr.Column():
            txt = gr.Textbox(label="متن (فارسی/انگلیسی)", lines=3, placeholder="مثال: امروز خیلی خوشحالم!")
            img = gr.Image(label="تصویر چهره", type="pil")
            aud = gr.Audio(label="صوت (wav/mp3)", type="filepath")
        with gr.Column():
            thr  = gr.Slider(0,1,0.7,step=0.01, label="آستانه OTHER برای متن/صوت")
            mins = gr.Slider(1.0, 6.0, value=3.0, step=0.5, label="مدت موسیقی (دقیقه)")
            seed = gr.Textbox(value="", label="Seed (خالی = هر بار متفاوت)")
            style_dd = gr.Dropdown(choices=["Global/Western","Persian/Traditional"], value="Global/Western", label="سبک/Palette")
            sf2_file = gr.File(label="SoundFont (sf2) اختیاری — اگر SF2 ایرانی داری آپلود کن", file_types=[".sf2"], type="filepath")
            go   = gr.Button("🔎 تحلیل و 🎵 ساخت موسیقی", variant="primary")

    with gr.Row():
        o1 = gr.Textbox(label="احساس",    interactive=False, elem_classes="out")
        o2 = gr.Textbox(label="اعتماد",   interactive=False, elem_classes="out")
        o3 = gr.Textbox(label="ایموجی",   interactive=False, elem_classes="out")

    audio_out = gr.Audio(label="🎧 موسیقی خروجی (پخش آنلاین)", type="numpy")
    rep_out   = gr.JSON(label="📊 گزارش ارزیابی (ساختاری + کلاسیفایر صوت روی WAV)")
    mp3_out   = gr.File(label="⬇️ دانلود MP3")

    go.click(analyze_and_make_music,
             [txt, img, aud, thr, mins, seed, style_dd, sf2_file],
             [o1, o2, o3, audio_out, rep_out, mp3_out])

app.launch(share=False)
# =====================================================================


Device for Transformers: cuda:0


Device set to use cuda:0
Device set to use cuda:0
/usr/local/lib/python3.11/dist-packages/transformers/pipelines/text_classification.py:111: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/configuration_utils.py:334: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Device set to use cuda:0


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>